## Прогнозирование на несколько шагов вперёд ансамблями над регрессионными деревьями

Регрессионные деревья и ансамбли над ними по умолчанию не предназначены для учёта временной структуры. Однако прогнозирование на несколько разных горизонтов одной и той же моделью, базирующейся на деревьях, можно осуществить следующим образом.

Возьмём обучающую выборку для задачи прогнозирования на один шаг вперёд. При группировке по идентификатору объекта применим операцию сдвига целевой переменной на $k$ шагов, где $k$ принимает значения от 0 до $(h-1)$, а $h$ — желаемый горизонт прогнозирования. Получим выборку, которая в $h$ раз больше по размеру, чем исходная. Добавим в эту выборку признак, принимающий значения от 1 до $h$ и равный $(k+1)$. Этот признак можно интерпретировать как то, на сколько шагов вперёд строится прогноз. Если теперь обучить модель на таких данных, то в зависимости от поданного на стадии предсказания значения нового признака модель сможет предсказывать на любое количество шагов вперёд от 1 до $h$.

Также стоит отметить, что популярные реализации случайного леса поддерживают возможность предсказания сразу нескольких целевых переменных (multi-output learning). В таком случае в качестве вектора целевых переменных можно брать вектор будущих значений с соответствующими горизонтами.

## Обратное распространение (backpropagation)

Обратное распространение — приём, позволяющий эффективно вычислять частные производные функции по её параметрам.

В контексте нейронных сетей обратное распространение пригождается при выполнении шага градиентного спуска. Пусть имеются:
* объект с вектором признаков $x \in \mathbb{R}^n$,
* целевая переменная на этом объекте $y$, являющаяся вещественным числом (в задаче регрессии) или вектором (например, one-hot вектором метки класса в задаче многоклассовой классификации),
* предсказание нейронной сети $\hat{y}$, могущее быть числом или вектором в зависимости от $y$,
* функция потерь $L$, по $y$ и $\hat{y}$ возвращающая вещественное число.

Тогда $i$-я компонента вектора обновления весов нейронной сети $w$ на этом объекте при шаге градиентного спуска с точностью до знака и умножения на темп обучения $\eta$ имеет вид:
$$\frac{\partial L(y, \hat{y})}{\partial w_i} = \nabla_{w_i}\hat{y} \: \nabla_\hat{y}L(y, \hat{y}),$$
где общности ради можно считать, что оператор $\nabla$ возвращает матрицу Якоби (матрицу, составленную из частных производных) размерности $k \times l$, где $k$ — размерность того, по чему дифференцируется, а $l$ — размерность того, что дифференцируется. При таком определении градиент (вектор частных производных) является частным случаем матрицы Якоби, и, на самом деле, в выписанной формуле фигурируют лишь скаляры и векторы: множитель $\nabla_{w_i}\hat{y}$ имеет размерность $1 \times m$, а множитель $\nabla_\hat{y}L(y, \hat{y})$ имеет размерность $m \times 1$, где $m$ — 1, если $y$ и $\hat{y}$ скаляры, или длина $y$ или $\hat{y}$, если это векторы. Так или иначе, множитель $\nabla_\hat{y}L(y, \hat{y})$ посчитать легко из явного вида функции $L$. Встаёт вопрос, как посчитать $\nabla_{w_i}\hat{y}$, и вот тут-то и пригождается обратное распространение.

На самом деле, вычислить $\nabla_{w_i}\hat{y}$ можно просто в лоб. Для этого достаточно вспомнить, что нейронная сеть является композицией дифференцируемых функций, а потом пройтись по всем путям от нейрона, к которому относится рассматриваемый вес $w_i$ (или от каждого из таких нейронов, если вес $w_i$ общий между несколькими нейронами), до любого выхода нейронной сети, применяя при этом формулу дифференцирования композиции:
$$\nabla_x (f \circ g) = \nabla_x g \nabla_g f.$$
Кстати, в скалярном виде эта формула известна ещё из школьной программы как:
$$(f \circ g)\prime(x) = f\prime(g(x))g\prime(x).$$

Так можно вычислить обновление какой-то одной компоненты вектора весов. Но если сделать это для всех значений $i$, то окажется, что значительное количество частных производных будет многократно посчитано заново. Например, для многослойного перцептрона частная производная значения активации какого-либо нейрона по значению активации нейрона предыдущего слоя будет посчитана столько раз, сколько существует путей из какого-либо входного или скрытого нейрона в какой-либо выходной нейрон, проходящих через связь между двумя рассматриваемыми нейронами.

Обратное распространение сводится к тому, что процесс вычисления частных производных становится организован таким образом, что все частные производные берутся лишь по одному разу. Для этого происходит движение по нейронной сети в обратную сторону (от выходов ко входам), и считаются частные производные выходов по значению активации текущего нейрона. В этом простом способе избежать повторов и заключается вся суть обратного распространения, а вектор градиента $L$ по $w$, который оно находит, является тем же самым вектором градиента $L$ по $w$, который был бы найден любым другим способом вычислить его (например, неэффективным способом прямого прохода, описывавшимся выше).

Между прочим, применимость обратного распространения не ограничивается нейронными сетями. Более общим понятием является граф дифференцируемых вычислений, то есть направленный граф, воспроизводящий некоторую дифференцируемую функцию, получающую на вход вещественный вектор и возвращающую вещественный вектор (возможно, другой длины). Вершины графа вычислений бывают трёх типов:
* входы — в них не ведут никакие рёбра, и в каждом из них задаётся соответствующая компонента входного вектора;
* операции — в них должно вести хотя бы одно ребро и из них должно выходить хотя бы одно ребро; каждой такой вершине сопоставлена дифференцируемая функция, принимающая на вход то, что отдают вершины, откуда ведут рёбра, и возвращающая вещественное число;
* выходы — в каждый из них ведёт ровно одно ребро, и результат операции, откуда ведёт ребро, становится соответствующей компонентой выходного вектора.

## Многозадачное обучение (multi-task learning)

Допустим, есть вспомогательная задача, для которой имеется много размеченных данных, и есть основная задача, для которой размеченных данных существенно меньше. Также допустим, что эти две задачи работают с одними и теми же объектами, но предсказывают на них разные целевые переменные. Например, во вспомогательной задаче нужно определить пол человека с портретного фото, а в основной задаче требуется определить цвет волос.

В описанной ситуации можно поступить следующим образом. Построим нейронную сеть для решения вспомогательной задачи и обучим её. Затем уберём сколько-то последних слоёв получившейся сети, и вместо них поставим новые, такие что размерность выходного слоя уже соответствует основной задаче.

Далее есть два варианта:

1. Зафиксировать веса оставленных слоёв нейронной сети, обученной под вспомогательную задачу, и просто прогонять объекты через оставленные слои как через статичный предобработчик;

2. Веса оставшихся от вспомогательной задачи слоёв обучать наравне с весами новых слоёв, то есть считать, что они просто были инициализированы по-другому, но больше отличий между ними и весами новых слоёв нет.

Какой из вариантов выбрать, зависит от числа данных в основной задаче. Если острого недостатка в них нет, то второй способ, как правило, даёт более высокий результат. Первый способ, однако, быстрее, и также он менее требователен к количеству данных (а если размеченных данных совсем мало, то можно даже использовать результат прогона через оставленные обученные слои в качестве признаков для более простого метода: например, метода ближайших соседей).

Новая нейронная сеть обучится лучше, чем если бы сеть с такой же архитектурой обучалась на данных для основной задачи с нуля. Дело в том, что при описанном подходе нейронная сеть будет опираться на скрытое представление, выученное по большому количеству размеченных данных оставленными слоями старой нейронной сети. Также подстройка весов под разные задачи способствует увеличению обобщающей способности. Подробности о том, почему многозадачное обучение работает, есть в разделе 3 статьи [Ruder, 2017](https://arxiv.org/pdf/1706.05098.pdf).

Ещё один подход к многозадачному обучению таков: обучать нейронную сеть параллельно то под вспомогательную задачу, то под основную, при каждом переключении задачи оставляя сколько-то первых слоёв, но заменяя последние слои на то, что было ранее в задаче, к которой происходит переход.

## Перечисление подходов к подбору гиперпараметров

Способы подбора гиперпараметов алгоритма машинного обучения бывают следующими:
* Перебор с экспоненциальной по числу гиперпараметров вычислительной сложностью:
    - Поиск по сетке (grid search) с кросс-валидацией относительно разбиения выборки на $k$ подвыборок;
    - Поиск по сетке с усреднением результатов $t$ кросс-валидаций относительно разбиения выборки на $k$ подвыборок;
* Перебор, подходящий и для большого числа гиперпараметров:
    - Поиск по случайному подмножеству узлов полной сетки (randomized grid search);
    - Поиск по подмножеству, составленному в соответствии с некотрым детерминированным планом эксперимента (design of experiment), которым может быть:
        - Латинский гиперкуб;
        - Оптимизированный латинский гиперкуб;
        - Последовательность Холтона;
        - Последовательность Соболя;
* Поиск оптимума в пространстве гиперпараметров на базе суррогатной модели:
    - Байесовская оптимизация с предсказаниями, сделанными [регрессией на основе гауссовских процессов](__home_url__/notes/Регрессия на основе гауссовских процессов) (реализована такая оптимизация, например, в `skopt.gp_minimize`);
* Некоторые теоретические оценки:
    - Для линейной регрессии оценка среднеквадратической ошибки при leave-one-out кросс-валидации считается в замкнутой форме без необходимости много раз настраивать веса;
    - [Информационные критерии](__home_url__/notes/Информационные критерии):
        - Акаике, AIC;
        - Шварца (также известен как байесовский), BIC;
    - Выбор гиперпараметров по байесовскому [принципу наибольшей обоснованности](__home_url__/notes/Принцип наибольшей обоснованности) (maximum evidence principle).

## Байесовский подход к дискриминативным моделям

Пусть имеется дискриминативная модель распределения матрицы объекты-признаки $X$, вектора целевой переменной $y$ и параметров модели $\theta$, а именно пусть дана условная плотность $p(y, \theta \, \vert \, X) = p(y \, \vert \, X, \theta) p(\theta \, \vert X) = p(y \, \vert \, X, \theta) p(\theta)$, где последнее равенство верно, потому что предполагается, что распределение параметров не зависит от данных, и где в правой части оба множителя известны в аналитическом виде.

В байесовском смысле задачей обучения дискриминативной модели является задача нахождения условной плотности $p(\theta \, \vert \, X_\mathrm{train}, y_\mathrm{train})$, где $X_\mathrm{train}$ и $y_\mathrm{train}$ образуют обучающую выборку. Искомая условная плотность находится по теореме Байеса, где "меняются местами" $\theta$ и $y_\mathrm{train}$ и где все вероятности являются условными относительно $X_\mathrm{train}$ (просто на заметку: в дискриминативном моделировании в отличие от генеративного моделирования не предполагается, что известна $p(X_\mathrm{train})$):
$$p(\theta \, \vert \, X_\mathrm{train}, y_\mathrm{train}) = \frac{p(y_\mathrm{train} \, \vert \, X_\mathrm{train}, \theta) p(\theta)}{\int p(y_\mathrm{train} \, \vert \, X_\mathrm{train}, \theta) p(\theta) d\theta}.$$
В предположении, что интеграл из знаменателя нижеследующего равенства берётся, задача обучения решена, а если он не берётся, то используются приближённые байесовские методы.

Применение же обученной модели в байесовском смысле является задачей нахождения распределения $p(y_\mathrm{new} \, \vert \, X_\mathrm{new}, X_\mathrm{train}, y_\mathrm{train})$ вектора целевой переменной $y_\mathrm{new}$ для известной матрицы объекты-признаки $X_\mathrm{new}$. На стадии применения возникает взвешенный ансамбль моделей с различными параметрами:
$$p(y_\mathrm{new} \, \vert \, X_\mathrm{new}, X_\mathrm{train}, y_\mathrm{train}) = \int p(y_\mathrm{new} \, \vert \, X_\mathrm{new}, \theta) p(\theta \, \vert \, X_\mathrm{train}, y_\mathrm{train}) d\theta,$$
где плотность $p(\theta \, \vert \, X_\mathrm{train}, y_\mathrm{train})$ была найдена при обучении, а если бы вместо неё стояла априорная плотность $p(\theta)$, то слева получилась бы плотность $p(y_\mathrm{new} \, \vert \, X_\mathrm{new})$.

## Принцип наибольшей обоснованности

Зададимся вопросом: если есть множество из $m$ [дискриминативных моделей](__home_url__/notes/Байесовский подход к дискриминативным моделям) $\{p_i(y \, \vert \, X, \theta_i) p_i(\theta_i)\}_{i=1}^m$, то какую из них выбрать? Один из ответов даёт принцип наибольшей обоснованности (evidence), предлагающий руководствоваться следующим правилом:
$$i_\mathrm{opt} = \arg \max_i \int p_i(y_\mathrm{train} \, \vert \, X_\mathrm{train}, \theta) p_i(\theta) d\theta,$$
где за $\theta$ обозначен вектор параметров из множества, являющегося объединением множеств параметров всех $m$ моделей (если у $i$-й модели нет какого-то параметра из $\theta$, то можно, например, считать, что в рамках $i$-й модели его распределение задаётся дельта-функцией Дирака).

С философской точки зрения, принцип наибольшей обоснованности является математической формализацией бритвы Оккама: из моделей, способных объяснить имеющиеся данные, выбирается наиболее простая, то есть наименее способная объяснять при другом выборе параметров данные, не похожие на имеющиеся. Если же говорить более предметно, то можно сказать, что принцип наибольшей обоснованности ищет баланс между двумя желательными, но не всегда совместимыми друг с другом свойствами модели. Эти свойства таковы:
* способность хорошо воспроизводить зависимость, присутствующую в обучающей выборке: чем выше в среднем $p_i(y_\mathrm{train} \, \vert \, X_\mathrm{train}, \theta)$, тем выше обоснованность;
* неспособность к переподгонке (overfitting): в общем случае, в зависимости от $\theta$ модель может воспроизводить различные зависимости и чем вероятнее $\theta$, при которых получается какая-то зависимость, не согласующаяся с данными, тем ниже обоснованность. Отметим, что усреднение происходит относительно априорного распределения $p(\theta)$, так что даже если есть значение вектора $\theta$, для которого высока $p_i(y_\mathrm{train} \, \vert \, X_\mathrm{train}, \theta)$ и которое поэтому стало бы наиболее вероятным после обучения, вклад такого $\theta$ будет внесён лишь с весом $p(\theta)$, а не с весом $p(\theta \, \vert \, X_\mathrm{train}, y_\mathrm{train})$.

Считая, что модели отличаются только гиперпараметрами, при помощи принципа наибольшей обоснованности можно подбирать гиперпараметры без кросс-валидации. Это особенно актуально тогда, когда гиперпараметров много, из-за чего поиск по хоть сколько-нибудь мелкой сетке становится вычислительно неподъёмным.

Более подробно про принцип наибольшей обоснованности можно прочесть в главе 28 книги [MacKay, 2003](http://www.inference.org.uk/mackay/itprnn/ps/343.355.pdf).

## Информационные критерии

Иногда в моделях из одного и того же семейства есть параметр, регулирующий сложность. Например, в семействе полиномиальных функций сложностью можно считать максимальную допустимую степень многочлена. Довольно часто сложность можно положить равной количеству оцениваемых по данным параметров — эта формулировка, в частности, включает в себя предыдущую, ведь при оценивании многочленом степени не выше $d$ оценивается $d+1$ параметр.

Модели из одного и того же семейства, отличающиеся друг от друга только сложностью, можно оценивать при помощи информационных критериев.

Информационный критерий Акаике имеет вид:
$$\mathrm{AIC} = 2k - 2l,$$
где $k$ — количество оценённых по данным параметров, а $l$ — значение функционала качества, достигнутое на тренировочной выборке (например, логарифм правдоподобия данных при условии того, что выборка порождается из распределения, задаваемого той моделью, которая была построена на тренировочной выборке).

Байесовский информационный критерий имеет вид:
$$\mathrm{BIC} = \log{(n)} \, k - 2l,$$
где $n$ — размер тренировочной выборки.

По сути, информационные критерии представляют собой оценку качества подгонки под данные с дополнительным штрафом за сложность модели. Интуитивно говоря, чем сложнее модель, тем больше ситуаций, под которые её можно подогнать и, значит, тем менее ценны уровни качества подгонки под данные.

Чем ниже значение какого-либо информационного критерия, тем более хорошей считается модель.

## Откуда может взяться недовольство заказчика машинно-обученной моделью?

Любая прикладная задача моделирования имеет заказчика, неважно, внутреннего или внешнего. Когда заказчик принимает модель, он решает, устраивает ли она его. Причины недовольства заказчика бывают такие:

1. Модель, и в самом деле, плохая;

2. Заказчик не умеет пользоваться моделью;

3. Ожидания заказчика неоправданно завышены.

Какой бы ни была истинная причина, заказчик, скорее всего, решит, что реализовался первый сценарий: модель плохая. Поэтому в интересах специалиста по моделированию избежать ситуаций, когда хорошая модель попадает во второй или третий сценарий.

Разберём эти два сценария подробнее.

Пример сценария, когда моделью не умеют пользоваться, таков: сервис на базе модели рассылает своевременные уведомления о появлении объектов положительного класса, но люди, которым они приходят, игнорируют их. Говоря более общо, можно сказать, что возможны осложнения с интеграцией в бизнес-процессы заказчика. Чтобы упредить их возникновение, полезно задать следующие вопросы:
* Что должно происходить с результатами работы сервиса?
* Как именно сервис приносит конечную ценность?
* Все ли исполнители знают и понимают ответы на два предыдущих вопроса?
* Можно ли оперативно отслеживать сбои в исполнении действий на базе сервиса и можно ли предложить количественные метрики оценки качества исполнения таких действий?
* Все ли признаки, использованные при обучении модели, будут доступны на стадии применения модели вовремя и в сопоставимом качестве?

Что касается второго сценария, проблему завышенных ожиданий проще всего решить, ещё до старта работ по проекту согласовав метрику успеха и её целевой уровень. Сделать это можно, проведя оценку конечного эффекта от внедрения сервиса. Этот конечный эффект может быть как финансовым, то есть выражающимся в деньгах, так и каким-либо более широким (скажем, социальным), но, так или иначе, измеримым.

Впрочем, бывают ситуации, когда заказчик рассуждает не в терминах конечного эффекта, а в терминах всестороннего описания зависимостей — в этом случае заказчик может потребовать, скажем, почти стопроцентную точность по положительному классу. Разумеется, модель, не способная точно описать все зависимости, тоже может быть полезна. Проиллюстрировать это заказчику можно при помощи такого сравнения: если есть лотерея, где вероятность выиграть равна $p$, стоимость участия составляет $c$, а выигрыш приносит $g$, то ответ на вопрос, рационально ли участвовать в такой лотерее, зависит не от того, насколько вероятность $p$ близка к 100%, а от того, как друг с другом соотносятся все три указанных параметра, ведь ожидаемый чистый выигрыш равен $pg - c$.

## Три фундаментальных источника ошибок в машинном обучении

Оптимальный в байесовском смысле регрессор (или классификатор) — это то, что задаётся следующим формальным выражением:
$$f_{opt} = \arg \min_{f \in F} \int_{(x, y) \sim P_{true}(x, y)}l(f(x), y),$$
где $F$ — множество всех возможных регрессоров (или классификаторов соответственно), $P_{true}$ — истинное совместное распределение признаков и целевой переменной, а $l$ — функция потерь, по значению регрессора (классификатора) на векторе признаков и целевой переменной на этом векторе признаков возвращающая соответствующий штраф.

В реальности есть ограничения, не позволяющие найти $f_{opt}$.

Во-первых, неизбежна ошибка приближения (approximation error): поиск проводится не по всему множеству возможных регрессоров (классификаторов), а лишь по какому-то подмножеству $\overline{F} \subset F$, например, по некому параметрическому семейству. Скажем, в случае с линейной регрессией $\overline{F}$ — это множество всех линейных регрессоров. Как следствие, ищется уже нечто другое:
$$\hat{f_1} = \arg \min_{f \in \overline{F}} \int_{(x, y) \sim P_{true}(x, y)}l(f(x), y).$$
Ошибка, вызываемая отличием $\hat{f_1}$ от $f_{opt}$, и называется ошибкой приближения.

Во-вторых, существует ошибка оценивания по данным (estimation error): невозможно получить истинную генеральную совокупность $P_{true}$, а вместо неё есть лишь выборка конечного размера $n$. С учётом такого ограничения задача принимает вид:
$$\hat{f_2} = \arg \min_{f \in \overline{F}} \frac{1}{n} \sum_{i=1}^{n} l(f(x_i), y_i) + \alpha R(f),$$
где $(x_i, y_i)$ — признаки и ответ на $i$-м объекте обучающей выборки, $R(f)$ — регуляризатор, а $\alpha$ — коэффициент, задающий силу регуляризации. Ошибка оценивания — ошибка, вызываемая отличием $\hat{f_2}$ от $\hat{f_1}$.

В-третьих, не всегда удаётся довести поиск минимума из предыдущего пункта до конца, и отсюда возникает ошибка оптимизации (optimization error). Если предположить, что вычислительные ресурсы ограничены и есть всего лишь $T$ итераций, то получится, что решается задача
$$\hat{f_3}^{(T)} = \arg \min_{(T); f \in \overline{F}} \frac{1}{n} \sum_{i=1}^{n} l(f(x_i), y_i) + \alpha R(f),$$
где запись $(T)$ под минимумом обозначает, что минимум ищется за ограниченное число шагов. По аналогии с определением предыдущих ошибок ошибка оптимизации — это ошибка, вызываемая отличием $\hat{f_3}^{(T)}$ от $\hat{f_2}$.

Можно считать, что машинное обучение на больших данных отличается от машинного обучения на малых данных тем, что ошибка оптимизации начинает доминировать над ошибкой оценивания по данным.

Подробности есть в статье [Bottou, Bousquet, 2007](https://papers.nips.cc/paper/3323-the-tradeoffs-of-large-scale-learning.pdf).

## Автоматическое обнаружение выбросов

Выброс (outlier) — объект, считающийся нетипичным для генеральной совокупности, фигурирующей в задаче. Выбросы могут искажать процесс обучения модели, а также результаты оценки качества модели (что создаёт риск неправильного подбора гиперпараметров). Строго говоря, является ли какой-то объект выбросом или нет, должен определять человек, отталкиваясь от своего понимания предметной области и от знаний о том, как именно собирались данные. Тем не менее на практике автоматическое удаление выбросов тоже может существенно улучшить конечный результат.

Способы автоматического обнаружения выбросов делятся на две группы в зависимости от того, на какие выбросы они реагируют:

1. Способы, позволяющие выявить одномерные выбросы, то есть выбросы, являющиеся аномальными значениями какого-либо одного признака. Пример одномерного выброса: среди точек на плоскости с координатами (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (100, 6) у последней точки подозрительно большая абсцисса.

2. Способы, позволяющие выявить многомерные выбросы, то есть выбросы, являющиеся аномальными объектами с точки зрения распределения данных. Пример многомерного выброса: среди точек на плоскости с координатами (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (1, 5) последняя точка лежит в стороне от прямой, на которой лежат все остальные точки, но при этом и её абсцисса, и её ордината не выбиваются из диапазона значений остальных точек.

К одномерным способам относятся:
* Стандартизированная оценка (Z-score) — из всех значений признака вычитается эмпирическое среднее этого признака, а полученные разности делятся на оценку стандартного отклонения признака; если получившееся отношение превышает 3, то объект считается выбросом. Порог 3 выбран, потому что в случае нормального распределения вероятность отклониться от среднего больше, чем на 3 стандартных отклонения, составляет 0,03% ([правило трёх сигм](https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D1%82%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5#%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%BE_%D1%82%D1%80%D1%91%D1%85_%D1%81%D0%B8%D0%B3%D0%BC)). 
* Устойчивая стандартизированная оценка (robust Z-score) — то же самое, что и стандартизированная оценка, но теперь вычитается медиана, а не среднее, и разности делятся на средний модуль отклонения от медианы, а не на стандартное отклонение. Предпочтительнее в тех ситуациях, где выбросов достаточно много для того, чтобы они могли исказить среднее и стандартное отклонение так, что часть из них будет принята за обычные объекты.
* Метод межквартильного размаха (interquartile range method) — берётся расстояние между 0,75-й и 0,25-й квантилями (так называемый межквартильный размах), а затем выбросами объявляются все наблюдения, отстоящие от медианы более чем на 1,5 межквартильных размаха. Численные параметры приведены в соответствии с классическим вариантом, но, разумеется, их можно поменять (например, вместо двух квартилей взять две другие квантили).

К многомерным способам относятся:
* Способы, где для каждого объекта считается расстояние Махалонобиса от него до некоторого оценённого по данным распределения, принимаемого за генеральную совокупность, а потом некой одномерной процедурой выбросами объявляются сколько-то объектов с наибольшим расстоянием Махалонобиса. Рассмотрим такой подход подробнее. Для многомерного вектора $x$ и многомерного вероятностного распределения $\tau(\mu, S)$ с вектором средних $\mu$ и ковариационной матрицей $S$ "расстоянием" от $x$ до $\tau(\mu, S)$ можно положить величину:
$$D_M(x, \tau(\mu, S)) = \sqrt{(x - \mu)^T S^{-1} (x - \mu)},$$
которая и называется расстоянием Махалонобиса. Интуитивно говоря, расстояние Махалонобиса отличается от евклидова расстояния между $x$ и $\mu$ тем, что каждое направление учитывается с весом, обратно пропорциональным разбросу распределения по этому направлению. В задаче обнаружения выбросов $\tau(\mu, S))$ можно единожды оценить по всем имеющимся данным, если их много, или же для каждого $x$ оценивать по всем объектам кроме $x$, если данных мало.
* Способы, где для каждого объекта считается оценка плотности генеральной совокупности в нём, а затем некой одномерной процедурой выбросами объявляются сколько-то объектов с наиболее низкой оценкой плотности. Оценку плотности можно получить, в частности, методом ядерного сглаживания (kernel density estimate). Как и в предыдущем пункте, если данных мало, то оценку ядерного сглаживания для каждого объекта лучше считать по всем объектам кроме него, а если данных много, то оценку ядерного сглаживания можно подсчитать один раз на всех объектах. Отдельно стоит отметить, что оценки ядерного сглаживания подвержены "проклятью размерности" и работают только в случае, когда размерность данных низкая. Если же размерность высокая, то для применения ядерного сглаживания можно предварительно снизить её, использовав PCA, t-SNE или MDS на отмасштабированных данных, однако как проводить масштабирование — нетривиальный вопрос, вносящий долю субъективности.

Подходом, занимающим промежуточное место между двумя выписанными подходами к обнаружению многомерных выбросов, являетя следующий. Распределение данных приближается смесью многомерных нормальных распределений, а потом выбросами, как и во втором варианте, объявляются сколько-то объектов с наименьшей оценкой плотности. Связь же с расстоянием Махалонобиса в том, что это расстояние даёт своего рода оценку плотности при приближении данных одним многомерным нормальным распределением.

## Автокодировщики с большей размерностью скрытого слоя

Рассмотрим автокодировщик (autoencoder) со входным слоем размерности $l$, одним скрытым слоем размерности $h$ и выходным слоем, размерность которого по определению должна быть равна размерности входного слоя, то есть $l$. Вопрос: могут ли для чего-то пригодиться автокодировщики, где $h > l$?

Без дополнительных модификаций такие автокодировщики не смогут обучаться чему-то нетривиальному, потому что всегда есть вырожденное решение: использовать лишь $l$ нейронов из $h$, чтобы просто передать вход в неизменном виде.

Есть следующие способы заставить скрытый слой выучивать нетривиальное представление:
* К ошибке восстановления, на входном векторе $x$ равной $\Vert x - \mathrm{autoencoder}(x)\Vert_2$, добавить регуляризатор, подталкивающий к разреженности скрытого слоя: например, $L_1$-норму $h$-мерного вектора, получающегося из $x$ в скрытом слое;
* Ко входу подмешивать шум, но при вычислении ошибки восстановления требовать близости к незашумлённому входу, то есть минимизировать математическое ожидание функции, на входном векторе $x$ равной $\Vert x - \mathrm{autoencoder}(x + \varepsilon)\Vert_2$, где $\varepsilon$ — шум.
* Каждую из компонент вектора входов с некоторой вероятностью $p$ обнулять, но при вычислении ошибки восстановления сверяться с вектором входов, где все компоненты имеют исходный вид — так автокодировщик будет побуждён выучивать взаимосвязи между признаками, позволяющие по части признаков восстановить остальные признаки. 

Полученный автокодировщик можно использовать в следующих целях:
* То, что получается в скрытом слое, можно брать в качестве признакового описания объекта; такие признаки могут содержать в себе более сложные взаимодействия, чем признаки, даваемые автокодировщиком с низким $h$; пригождается это в частичном обучении (semi-supervised learning), ведь объекты без меток тоже влияют на выучиваемый способ порождать признаки;
* До появлении [пакетной нормализации](__home_url__/notes/Пакетная нормализация) и остаточного обучения была популярна "жадная" (послойная от входов к выходам) инициализация весов глубокой однонаправленной нейронной сети, при которой инициализируемый на данный момент слой дополнялся до автокодировщика и обучался задаче автокодировщика, где кодировалось то, что подавал на вход предыдущий слой, — стало быть, если в каком-то месте ширина нейронной сети возрастала, то требовалось обучать автокодировщик, у которого $h > l$.

## Обучение метрик (metric learning)

В машинном обучении есть задача, похожая на задачу классификации, но тем не менее отличная от неё. Пусть множество рассматриваемых объектов — множество пар, элементами которого являются пары $(x_i, x_i^\prime)$, где $x_i$ и $x_i^\prime$ принадлежат одному и тому же пространству, а целевая переменная $y_i$ равна 1, если $x_i$ и $x_i^\prime$ похожи друг на друга, и равна 0, если $x_i$ и $x_i^\prime$ не похожи друг на друга. Требуется выучить отображение $f$, такое что евклидово расстояние между $f(x_i)$ и $f(x_i^\prime)$ мало для похожих объектов и велико для непохожих объектов. Поскольку после решения задачи появляется возможность находить расстояние между объектами, а не только предсказывать, похожи ли они, данная задача отличается от задачи бинарной классификации с признаковым описанием удвоенной длины.

Обучение метрик может быть полезно для решения задачи многоклассовой классификации с высоким числом классов, таких что каждый класс представлен малым числом объектов обучающей выборки. Положим все объекты одного и того же класса похожими друг на друга, а объекты разных классов отличающимися. Предсказывать классы новых объектов можно, применяя метод ближайшего соседа в пространстве, получающемся после применения отображения $f$.

Для обучения метрик иногда используют функцию потерь, называемую contrastive loss:
$$l(x_i, x_i^\prime, f, y_i) = y_i\Vert f(x_i) - f(x_i^\prime)\Vert_2 + (1 - y_i)\max(1 - \Vert f(x_i) - f(x_i^\prime)\Vert_2, 0).$$
Эта функция потерь состоит из двух слагаемых. Первое может быть отлично от нуля, только когда взята пара похожих объектов, и это слагаемое побуждает минимизировать евклидово расстояние между тем, во что переходят похожие объекты. Второе слагаемое может быть отлично от нуля, только когда взята пара непохожих объектов, и это слагаемое побуждает отображать непохожие объекты так, чтобы расстояние между их образами под действием $f$ было больше некоторого фиксированного порога.

Эмпирический риск для задачи обучения с contrastive loss, как и следовало ожидать, имеет вид:
$$E(f) = \frac{1}{n}\sum_{i = 1}^n  l(x_i, x_i^\prime, f, y_i) + \alpha R(f),$$
где через регуляризатор $R$ и силу регуляризации $\alpha$ можно наложить дополнительные ограничения.

Если считать, что отображение $f$ задаётся умножением слева на матрицу $M$ размера $k \times n$, то вышеуказанная функция потерь будет плоха тем, что задача её оптимизации по $M$ не является выпуклой из-за отрицательного знака перед вторым слагаемым. В таком случае берут немного другую функцию потерь, отталкивающуюся от того, что если обозначить матрицу $M^TM$ размера $n \times n$ за $S$, то $\Vert Mx_i - Mx_i^\prime\Vert_2 = (x_i - x_i^\prime)^T S (x_i - x_i^\prime)$:
$$l(x_i, x_i^\prime, S, y_i) = y_i(x_i - x_i^\prime)^T S (x_i - x_i^\prime) + (1 - y_i)\max(1 - (x_i - x_i^\prime)^T S (x_i - x_i^\prime), 0).$$
При минимизации эмпирического риска по матрице $S$ размера $n \times n$ стоит учитывать ограничение, что $S$ должна быть неотрицательно определённой, иначе её нельзя было бы представить в виде $S = M^TM$. Данное ограничение является выпуклым. Однако платой за превращение задачи оптимизации в выпуклую является то, что нельзя заранее задать размерность выходного пространства, ведь ограничения на ранг матрицы $S$ не являются выпуклыми.

## Искусственная целевая переменная

Бывают ситуации, когда достоверно целевая переменная неизвестна, но её можно восстановить оценочно. Например, в задаче обнаружения подозрительного трафика, приходящего на сайт, каких-то пользоваталей можно отнести к ботам или к живым людям на основании экспертного мнения.

Рассмотрим две ситуации:
* Специалисты по предметной области написали детерминированную программу, на базе свода правил и формул вычисляющую целевую переменную по входным признакам;
* Специалисты по предметной области не смогли составить список правил и формул, по которым можно вычислять целевую переменную, но внимательно изучили данные и разметили каждый пример вручную.

Вопрос: в каких из этих двух ситуаций стоит применять машинное обучение?

Ответ: только во второй. В первой ситуации построенная модель будет вести себя точно так же, как уже написанная программа, с точностью до ошибок, вызываемых нерепрезентативностью обучающей выборки, малым количеством данных и/или артефактами обучения. Иными словами, нет ничего, что позволило бы модели стать лучше, чем программа. А вот во второй ситуации модель, если хорошо обучится, станет автоматизированным воплощением интуиции и знаний экспертов.

Однако если немного изменить условие первой ситуации, то и в ней машинное обучение может помочь. Например, пусть программа в своих вычислениях использует также некоторые дополнительные внешние признаки, такие что измерять их дорого или долго, и которые поэтому не будут доступны на стадии регулярного применения. Тогда можно рассчитать целевую переменную по всем признакам включая трудноизмеримые, а потом обучить модель предсказывать полученную целевую переменную только по признакам, которые всегда будут доступны. Точно так же программа может использовать для разметки и информацию из более поздних моментов времени, которая на стадии применения станет неизвестна, поскольку будет относиться к будущему.

## Классификация с очень большим количеством классов

Пусть есть задача классификации, где различных меток много в сравнении с количеством объектов обучающей выборки (например, меток лишь в три раза меньше, чем объектов). В таком случае задача осложняется тем, что модель будет довольно редко угадывать правильную метку, и, значит, оптимизировать функции потерь, связанные с точными попаданиями, непрактично.

Предположим, что задача классификации с $q$ классами, где $q$ большое в вышеописанном смысле, решается путем настройки параметров $\theta$ некоторого классификатора, способного оценивать степени уверенности в принадлежности к классам (этим степеням даже необязательно давать в сумме 1). Обозначим за $p_\theta(x_i, c)$ предсказанную на $i$-м объекте $x_i$ степень уверенности в классе $c$.

Для $y_i$, истинного класса $i$-го объекта $x_i$, введём функцию $\mathrm{rank}_\theta(y_i)$. Эта функция вычисляется как порядковый номер класса $y_i$ при ранжировании классов по невозрастанию $f(c) = p_\theta(x_i, c)$. Формальное определение выглядит так:
$$\mathrm{rank}_\theta(y_i) = \#\!\left\{\left. c \in Q \: \right| \: p_\theta(x_i, c) > p_\theta(x_i, y_i)\right\} + 1,$$
где $Q$ — $q$-элементное множество классов, а решётка обозначает количество элементов в множестве. С учётом того, что предсказанные степени уверенности разных классов могут совпадать, стоит обратить внимание, что стоит знак >, а не $\ge$.

Для обучения $\theta$ можно использовать функции потерь из следующего параметрического семейства, зависящего от параметров $\alpha_1$, ..., $\alpha_q$:
$$l(x_i, y_i, \theta) = \sum_{k = 1}^{\mathrm{rank}_\theta(y_i) - 1}\alpha_k,$$
где если верхний предел суммы равен 0, то сумма по пустому множеству индексов полагается равной нулю, а зависимость правой части от $x_i$ заложена в определении $\mathrm{rank}_\theta(y_i)$.

Некоторые известные метрики оптимизируются при обучении с функциями потерь из данного семейства:
* точность (accuracy) будет оптимизирована при $\alpha_1 = 1$ и $\alpha_k = 0$ для $k \in \{2, ..., q\}$;
* вероятность угадать с $m$ попыток будет оптимизирована при $\alpha_m$ = 1 и остальных $\alpha_k = 0$;
* средний ранг будет оптимизирован при всех $\alpha_k$, равных друг другу.

## Внутреннее устройство LSTM-нейрона

#### Что означает LSTM?

Аббревиатура LSTM расшифровывается как Long Short-Term Memory (краткосрочная память произвольной длины). Так называется архитектура нейронной сети, часто используемая при решении задач, где объекты образуют последовательность, причём влияние некоторого объекта может распространяться на целевые переменные объектов, стоящих впереди него на произвольном расстоянии.

#### Содержимое LSTM-нейрона

Отличием LSTM является то, что в ней вводится специальный нейрон со сложной внутренней структурой, называемый LSTM-нейроном или LSTM-модулем (LSTM cell, LSTM unit, LSTM neuron). Рассмотрим устройство такого нейрона.

Для каждого индекса $t$, которым нумеруются позиции последовательности, с LSTM-нейроном ассоциированы следующие сущности:
* входы и выходы:
    - вектор входов $x_t$, берущийся из последовательности и имеющий длину $d$,
    - вектор возвращаемых нейроном выходов $y_t$, имеющий длину $h$;
* внутреннее состояние:
    - вектор "памяти" $c_t$ той же длины $h$, что и вектор выходов;
* три вектора, называемых вентилями и имеющих ту же длину $h$, что и выход:
    - входной вентиль (input gate), $i_t$,
    - вентиль забвения (forget gate), $f_t$,
    - выходной вентиль (output gate), $o_t$;
* восемь матриц весов и четыре векторных свободных члена:
    - $W_i$, $W_f$, $W_o$, $W_c$, их размер $h \times d$,
    - $U_i$, $U_f$, $U_o$, $U_c$, их размер $h \times h$,
    - $b_i$, $b_f$, $b_o$, $b_c$, их длина $h$.
    
Когда нейронная сеть уже обучена, последние двенадцать сущностей не меняются от шага к шагу, и именно поэтому индекс $t$ у них отсутствует.

#### Динамика при движении по последовательности

Начальные условия в позиции $t = 0$, предшествующей началу последовательности, задаются так:
* $c_0 = 0$, $y_0 = 0$,
* веса матриц и свободных членов как-то инициализированы, если происходит обучение, или уже обучены и зафиксированы, если речь идёт о применении,
* чему равны вентили, неважно.

При заданной последовательности $x_t$ для любой позиции $t \ge 1$ можно рекуррентно определить значения всех сущностей, ассоциированных с LSTM-нейроном, по формулам:
$$i_t = \sigma(W_i x_t + U_i y_{t-1} + b_i),$$
$$f_t = \sigma(W_f x_t + U_f y_{t-1} + b_f),$$
$$o_t = \sigma(W_o x_t + U_o y_{t-1} + b_o),$$
$$c_t = f_t \circ c_{t-1} + i_t \circ \tanh (W_c x_t + U_c y_{t-1} + b_c),$$
$$y_t = o_t \circ \tanh (c_t),$$
где $\sigma$ — сигмоидная функция активации, а $\circ$ — операция взятия адамарова произведения (поэлементное умножение).

#### Интерпретация

Почему данная конструкция, после обучения являющаяся некоторой динамической системой в дискретном времени, осмысленна?

Можно считать, что LSTM-нейрон в зависимости от значений вентилей способен демонстрировать различные режимы поведения (ниже 0 и 1 обозначают векторы из одних 0 или 1 соответственно):
* при $f_t = 0$, $i_t = o_t = 1$ получается обычный рекуррентный нейрон (нейрон Элмана);
* при $o_t = 0$, $i_t = f_t = 1$ происходит накопление входной информации (например, нейрон может работать счётчиком), а на выход информация не отдаётся;
* при $i_t = o_t = 0$, $f_t = 1$ происходит хранение того, что в памяти, без изменений;
* при $i_t = 0$, $f_t = o_t = 1$ происходит извлечение наружу информации, накопившейся в памяти;
* при $i_t = f_t = 0$ происходит сбрасывание памяти.

Таким образом, LSTM-нейрон обладает способностями к гибкому управлению памятью и способен проводить над ней все операции, кажущиеся необходимыми. Если данных много и закономерности в них устойчивые, то нейронная сеть с такими нейронами сможет выучить их, даже в случае отсутствия ограничения максимальной дальности влияния входов на выходы.

#### Обучение

Обучение LSTM-нейрона производится методом, по-английски называемым Truncated Back-Propagation Through Time, где первое слово обозначает, что в вычислениях следующие частные производные полагаются равными нулю:
$$\frac{\partial i_t}{\partial y_{t-1}} = \frac{\partial f_t}{\partial y_{t-1}} = \frac{\partial o_t}{\partial y_{t-1}} = \frac{\partial c_t}{\partial y_{t-1}} = 0.$$

#### Модификации

Помимо классического LSTM-нейрона, описанного выше, существует версия, называемая LSTM с глазками (peepholes). Её отличие в том, что вентили могут обращаться к памяти нейрона (подглядывать в неё через глазки). Вводятся ещё три матрицы размера $h \times h$, обозначаемые как $V_i$, $V_f$ и $V_o$. В рекуррентных соотношениях, задающих динамику при движении по последовательности, последние два равенства остаются неизменными, а первые три принимают вид:
$$i_t = \sigma(W_i x_t + U_i y_{t-1} + V_i c_{t-1} + b_i),$$
$$f_t = \sigma(W_f x_t + U_f y_{t-1} + V_f c_{t-1} + b_f),$$
$$o_t = \sigma(W_o x_t + U_o y_{t-1} + V_o c_{t} + b_o).$$
В последнем из соотношений используется $c_t$, а не $c_{t-1}$, потому что на момент обновления значения выходного вентиля значение памяти на текущем шаге уже известно.

Есть и версия LSTM с глазками, где матриц $U_i$, $U_f$, $U_o$ и $U_c$ нет (т.е. можно считать, что они тождественно нулевые).

## На пути к word2vec

Помимо word2vec есть некоторые более простые способы отобразить слово в непрерывный вектор.

#### Вложение на базе разбивки по коллекциям слов

* Построим матрицу, где строкам соответствуют слова, столбцам — некоторые коллекции слов (например, документы или предложения), а на пересечении строки и столбца может стоять:
    - бинарный индикатор вхождения слова в коллекцию;
    - количество вхождений слова в коллекцию;
    - TF-IDF;
    - BM-25.
* Применим к получившейся матрице какую-либо процедуру снижения размерности, такую как:
    - метод главных компонент, PCA;
    - многомерное шкалирование, MDS;
    - вложение на базе случайного соседства, t-SNE.
* Примем за векторное представление некоторого слова соответствующую этому слову строку матрицы со сниженной размерностью.

#### Вложение через переходные вероятности

* Пронумеруем все слова.
* Возьмём матрицу размера $\vert V \vert \times \vert V \vert$, где $\vert V \vert$ — количество слов, на пересечении $i$-й строки и $j$-го столбца которой стоит вероятность того, что сразу же после $i$-го слова в тексте встретится $j$-е слово. Казалось бы, вложением $i$-го слова можно считать $i$-ю строку такой матрицы или $i$-й столбец такой матрицы, однако у столь простого подхода есть два недостатка:
    - Большое количество оцениваемых по данным параметров, а именно $\vert V \vert^2$ элементов матрицы переходных вероятностей;
    - Размерность вектора вложения такая же, как у вектора, получаемого через one-hot encoding;
    - Подобное представление не очень информативно, хотя и более информативно, чем представление, получаемое через one-hot encoding.
* Попробуем получить более разумное вложение за счёт большей компактности представления. Предположим, что для некоторого фиксированного $d < \vert V \vert$ откуда-то даны две матрицы $W_1$ и $W_2$ размеров $d \times \vert V \vert$ и $\vert V \vert \times d$ соответственно, такие что матрицу переходных вероятностей удалось приблизить матрицей $W_2 W_1$. Последнее предположение формализуется и уточняется так: $\forall i, i \in \{1, ..., \vert V \vert\}$, после применения операции softmax к $\vert V \vert \times 1$ вектору-столбцу $W_2 W_1 x_i$, где $x_i$ — вектор-столбец размера $\vert V \vert \times 1$, такой что его $i$-я компонента равна 1, а остальные равны 0 (т.е. $x_i$ — one-hot представление $i$-го слова), получается вектор, близкий к вектору переходных вероятностей из $i$-го слова. В таком случае вложением слова в векторное пространство можно считать что угодно из этого списка:
    - $i$-й столбец матрицы $W_1$;
    - $i$-ю строку матрицы $W_2$;
    - сумму или полусумму $i$-го столбца матрицы $W_1$ и транспонированной $i$-й строки матрицы $W_2$ (обычно так делают для GloVe, а не для word2vec);
    - конкатенацию $i$-го столбца матрицы $W_1$ и транспонированной $i$-й строки матрицы $W_2$.
* Описанный в предыдущем пункте подход имеет следующие преимущества:
    - Всего $2 d \vert V \vert$ обучаемых параметров;
    - Размерность вектора вложения можно сделать маленькой;
    - Получаются более информативные векторные представления, поскольку теперь $\vert V \vert \times \vert V \vert$ матрица переходных вероятностей не может быть произвольной, а должна быть сводимой к виду $W_2 W_1$ с точностью до применения операции softmax к каждому столбцу указанного произведения двух матриц.

#### Связь word2vec с вложением через переходные вероятности

Нейронная сеть word2vec является способом найти описанные в предыдущем разделе матрицы $W_1$ и $W_2$, и в этом способе ради ускорения обучения возникают дополнительные трюки наподобие отрицательного сэмплирования. Главное отличие в постановке задачи машинного обучения заключается в том, что в word2vec выходное слово уже не обязано идти сразу же после входного слова, а должно лишь попасть в контекст определённой ширины.  

Впрочем, иногда считают, что также в word2vec присутствует зависимость структуры нейронной сети от выбранного размера контекстного окна. Обозначим размер контекстного окна за $C$. Тогда:
* в word2vec-CBoW (continuous bag of words):
    - входной слой имеет размер $C \vert V \vert$,
    - в скрытом слое усредняются $C$ произведений одной и той же матрицы $W_1$ со входными one-hot векторами, каждый из которых имеет длину $\vert V \vert$ (иногда в скрытом слое вместо усреднения используют суммирование),
    - выходной слой имеет размер $\vert V \vert$, а получается выход умножением слева вектора скрытого слоя на $W_2$ и применением softmax к произведению;
* в word2vec-SkipGram:
    - входной слой имеет размер $\vert V \vert$,
    - скрытый слой получается из входа умножением на матрицу $W_1$,
    - выход имеет длину $C \vert V \vert$, и для этих $C$ блоков, представляющих вероятности попадания слова в какую-либо из позиций контекста входного слова, происходит умножение одной и той же матрицы $W_2$ на вектор скрытого слоя с последующим применением softmax (то, что во всех $C$ копиях будет одно и то же вероятностное распределение, нестрашно, ведь задача не в том, чтобы предсказать контекст, а в том, чтобы обучить вложение).

Поскольку в SkipGram функция потерь (а именно [log loss](__home_url__/notes/Логарифмическая функция потерь и энтропия Шеннона)) вычисляется независимо для каждого из $C$ блоков, можно считать, что обучающих примеров в $C$ раз больше, чем было исходно, а вход и выход имеют длину $\vert V \vert$. В рамках такого [подхода к word2vec](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) размеры слоёв не зависят от размера контекстного окна.

Так или иначе, есть сходство между word2vec и представлением матрицы в виде произведения двух матриц меньшего ранга. Однако по сравнению с матричными разложениями у word2vec есть два преимущества:
* существуют приёмы, дающие существенное ускорение на этапе обучения, благодаря чему становится возможным обучение на больших объёмах реальных данных;
* поддерживается потоковое обучение, то есть уже обученную модель можно дообучить без обучения с нуля.

## Вероятностная интерпретация word2vec

Пусть есть выборка пар ($x_i$, $y_{i,c}$), где $i \in \{1, ..., L\}$, $L$ — размер выборки, $c \in \{1, ..., C\}$, $C$ — как-то заданное количество слов в контексте, $x_i$ — one-hot encoded вектор какого-то слова, а $y_{i,c}$ — one-hot encoded вектор слова, встретившегося в контексте слова $x_i$ на позиции $c$.

Допустим, эта выборка порождена из распределения, такого что условные вероятности выхода при условии входа имеют вид $$\mathbb{P}(y_{i,c} \vert x_i) = \frac{\exp(I(x_i)^TO(y_{i,c}))}{\sum_{v \in V}\exp(I(x_i)^TO(v))},$$
где $V$ — множество всех слов, а $I$ и $O$ — некоторые отображения из $\vert V \vert$-мерного в $d$-мерное пространство. Сделаем два наблюдения, касающихся выписанной формулы. Во-первых, в правой части применяется операция softmax. Во-вторых, поскольку все векторы $x_i$ и $y_{i,c}$ таковы, что в них на одной позиции стоит 1, а на всех остальных стоят 0, без ограничения общности можно считать, что $I$ и $O$ задаются умножением на матрицы размера $d \times \vert V \vert$.

Попытаемся найти эти матрицы, решив задачу максимизации правдоподобия имеющейся выборки:
$$\prod_{i = 1}^L\prod_{c = 1}^C \mathbb{P}(y_{i,c} \vert x_i) \rightarrow \max_{I, O}.$$

Оказывается, word2vec именно эту задачу и решает. Убедимся в этом.

Неоптимизированная (слишком долго обучающаяся) word2vec-SkipGram принимает на вход one-hot encoded вектор слова, умножает его слева на $d \times \vert V \vert$ матрицу $W_1$ (по сути, это операция взятия соответствующего слову столбца матрицы $W_1$) и делает результат вектором значений скрытого слоя, а этот вектор значений скрытого слоя умножает слева на $\vert V \vert \times d$ матрицу $W_2$, чтобы получить вектор размера $\vert V \vert \times 1$, после применения операции softmax к которому получаются вероятности попадания слов в контекст входного слова. Раз так, то отображением $I$, выучиваемым word2vec, является умножение слева на матрицу $W_1$, а отображением $O$ — умножение слева на транспонированную матрицу $W_2$. Обучается же word2vec под задачу $\vert V \vert$-классовой классификации с логарифмической функцией потерь, где по $x_i$ требуется предсказать $y_{i,c}$. Выкладкой проверяется, что задача обучения неоптимизированной word2vec совпадает с задачей максимизации выписанного правдоподобия.

Тут нелишне сделать небольшое отступление. Зададимся вопросом: почему для слова с идентификатором $j$ выучивается целых два вложения, а именно $j$-й столбец матрицы $W_1$ и $j$-я строка матрицы $W_2$? Иными словами, почему по аналогии с некоторыми автокодировщиками нельзя положить соответствующие пары весов по определению равными друг другу, чтобы при обучении они учитывались как один оптимизируемый параметр? Дело в том, что тогда word2vec получит неустранимый дефект: согласно её предсказаниям, почти всегда наиболее вероятным словом в контексте некого слова будет оно же само. Действительно, скалярное произведение $I(x_i)^TO(y_{i,c})$ тем больше, чем более коллинеарны множители, и при $I \equiv O$ больше, чем $I(x_i)^TO(x_i)$ может оказаться только аналогичное произведение для слова, образ которого под действием $I$ коллинеарен $I(x_i)$. Шансы, что такие слова найдутся, пренебрежимо малы.

Раз уж ищутся одновременно два вложения, уместно прокомментировать отличия в их интерпретации: $j$-й столбец матрицы $W_1$ задаёт значение скрытого слоя, а $j$-я строка матрицы $W_2$ показывает, как скрытый слой должен быть преобразован, чтобы получились вероятности попадания других слов в контекст. В принципе, пользоваться можно любым из этих вложений, но чаще под вложением, получаемым при помощи word2vec, имеют в виду $j$-й столбец матрицы $W_1$.

Вернёмся к основной теме заметки. Обнаруживается, что описанная выше архитектура word2vec обучается чрезвычайно медленно, так как:
* для вложения $\vert V \vert$ слов в $d$-мерное пространство требуется на каждом шаге обновлять $2 \vert V \vert d$ весов;
* на каждом объекте вычисляется softmax от вектора длины $\vert V \vert$.

Для ускорения обучения, конечно, можно выкинуть сколько-то обучающих примеров с вероятностью, пропорциональной частоте входного слова, потому что и оставшегося хватит для выучивания векторных представлений слов. Но есть и два способа принципиально ускорить обучение word2vec:
* Noise Contrastive Estimation. В нём применяется отрицательное сэмплирование. Сделать его можно двумя способами:
    - по-прежнему рассматривается задача $\vert V \vert$-классовой классификации, но при обучении считаются константами все веса от входов к скрытому слою кроме весов, исходящих от текущего входного слова, и все веса от скрытого слоя к выходному слою кроме весов, ведущих к текущему выходному слову, и весов, ведущих к скольки-то другим словам. Эти другие слова получаются процедурой так называемого отрицательного сэмплирования: они сэмплируются из эмпирического распределения слов, возведённого в степень 0,75, чтобы редкие слова выбирались чаще. Число 0,75, согласно [первоисточнику](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf), было подобрано эмпирически, причём [не исключено](https://arxiv.org/pdf/1804.04212.pdf), что для других корпусов этот гиперпараметр нужно подбирать заново. Сокращение числа обновляемых весов само по себе ускоряет обучение, но для большего эффекта операция softmax применяется только для ограничения выходного вектора на истинное $y_{i,c}$ и отрицательно просэмплированные к нему слова. 
    - задача $\vert V \vert$-классовой классификации заменяется на задачу бинарной классификации, где от нейронной сети требуется по паре слов, представленных как конкатенация их one-hot encoded векторов, сказать, встретились ли они в одном контекстном окне или нет, а для каждого примера положительного класса, взятого из реальных текстов, при том же первом слове пары сэмплируется несколько вторых слов по процедуре отрицательного сэмплирования. 
* Hierarchical Softmax. Здесь идея сводится к тому, чтобы изменить вероятностную модель на близкую к ней, но требующую меньшего количества вычислений. Выигрыш происходит за счёт того, что softmax заменяется на вычислительно более простое приближение. Для этого отображение $O$ берётся таким, что областью его определения являются не one-hot encoded векторы слов, а узлы дерева Хаффмана, листьями которого являются слова. В вероятностную модель при этом вносятся соответствующие правки:
$$\mathbb{P}(y_{i,c} \vert x_i) = \prod_{l \in \mathrm{Path}(y_{i,c})} \sigma\!\left(-\mathrm{Child}(l, y_{i,c}) I(x_i)^TO(l)\right),$$
где $\mathrm{Path}(y_{i,c})$ — путь от корня дерева Хаффмана до листа, соответствующего слову $y_{i,c}$, $\sigma$ — сигмоидная функуция, а $\mathrm{Child}(l, y_{i,c})$ равна 1, если в узле $l$ путь до вершины $y_{i,c}$ поворачивает направо, и -1 иначе. Если для того чтобы применением softmax получить из выходного слоя вероятностное распределение на словах требовалось просуммировать $\vert V \vert$ слагаемых в знаменателе, то для hierarchical softmax вычисление вероятностного распределения на словах требует лишь порядка $\log \vert V \vert$ операций.

## Метод Виолы-Джонса

#### Общее описание

Метод Виолы-Джонса применяется для детекции определённых объектов на изображениях. Чаще всего его используют для детекции лиц на фотографиях. При этом под детекцией имеется в виду обведение интересующего объекта в рамку (bounding box) без дальнейшего распознавания, какой именно из интересующих объектов представлен в этой рамке.

Хотя метод был предложен в [работе](http://citeseer.ist.psu.edu/viewdoc/summary?doi=10.1.1.10.6807) 2001-го года и с тех пор появились решения на базе нейронных сетей, он используется и поныне благодаря относительно высокой точности и масштабируемости.

Метод Виолы-Джонса предполагает, что изображения хранятся в чёрно-белом формате. Таким образом, с точки зрения метода Виолы-Джонса с каждым пикселем изображения ассоциировано ровно одно число, а именно число, кодирующее интенсивность чёрного.

#### Обучение

Детектор, получаемый методом Виолы-Джонса, нуждается в предварительной настройке (обучении) на данных.

Обучающая выборка собирается как для задачи бинарной классификации. Объектами положительного класса в ней являются изображения, на которых представлено непосредственно то, что впоследствии необходимо будет обводить в рамки (скажем, сами лица, а не люди в полный рост). При этом все такие изображения должны быть приведены к одному размеру $n \times m$ пикселей. В оригинальной работе изображения лиц сжимались до размера 24 пикселя на 24 пикселя, где число 24 выбрано по той причине, что имеет много целочисленных делителей (2, 3, 4, 6, 8, ...). Объектами отрицательного класса в такой выборке являются фрагменты того же размера $n \times m$, полученные из посторонних изображений, не содержащих то, что необходимо детектировать.

Признаковое описание объектов, используемое методом Виолы-Джонса, нетривиально. Входящие в него признаки называют вдохновлёнными преобразованием Хаара (Haar-like features), и, упрощённо говоря, можно считать, что эти признаки реагируют на наличие граней и линий в определённых местах. Интересно то, что в методе Виолы-Джонса признаки сами имеют параметры, настраиваемые по данным.

Даже для изображений размера 24 $\times$ 24 получается более 100 тысяч признаков. Любой из этих признаков может быть отнесён к одной из следующих групп:
* Вертикальная грань. Каждому признаку из этой группы сопоставлен некоторый прямоугольный фрагмент обучающего объекта размера $k \times l$, где $l$ должно быть чётным, $l = 2p$. Если сумма интенсивностей левой половины этого фрагмента меньше (или больше — есть и такой признак, и такой) суммы интенсивностей правой половины этого фрагмента на некоторый обучаемый порог $\theta$ (свой для каждого признака), то признак равен 1, а иначе он равен -1.
* Горизонтальная грань. Всё аналогично предыдущей группе, но теперь от обучающего изображения берутся фрагменты размером $k \times l$, где уже $k$ обязано быть чётным, а разность считается между суммами по верхней и нижней половинам.
* Вертикальная линия. Каждому признаку из этой группы сопоставлен некоторый прямоугольный фрагмент обучающего объекта размера $k \times l$, где $l$ представимо в виде $l = 2p + q$. Если средняя интенсивность по центральному прямоугольнику размера $k \times q$, вложенному в этот фрагмент, меньше (или больше) средней интенсивности по двум боковым прямоугольникам размера $k \times p$ на некоторый обучаемый порог $\theta$ (свой для каждого признака), то признак равен 1, а иначе он равен -1.
* Горизонтальная линия. Определяется аналогично вертикальной линии.
* Четыре прямоугольника крест-накрест. Каждому признаку из этой группы сопоставлен некоторый прямоугольный фрагмент обучающего объекта размера $k \times l$, где и $k$, и $l$ чётные, $k = 2p$, $l = 2q$. Если сумма интенсивностей по левому нижнему и правому верхнему прямоугольникам размера $p \times q$ меньше (или больше) суммы интенсивностей по левому верхнему и правому нижнему прямоугольникам размера $p \times q$ на некоторый обучаемый порог $\theta$ (свой для каждого признака), то признак равен 1, а иначе он равен -1.

Вычисление всех этих признаков, если его проводить в лоб по определению, окажется вычислительно затратным. Однако существует приём, позволяющий считать каждый из этих признаков за константное время вне зависимости от размеров прямоугольного фрагмента, по которому он считается. Этот приём связан с так называемым интегральным изображением (integral image), получаемым из исходного изображения. Строго говоря, интегральное изображение — это не изображение, потому что интенсивность чёрного в нём может выходить за максимально допустимое значение. Скорее, это таблица размера $n \times m$. В ней на пересечении $i$-й строки и $j$-го столбца стоит число, равное сумме интенсивностей по всем пикселям исходного изображения, находящимся на расстоянии не более $i$ от левого края и не более $j$ от верхнего края. Благодаря интегральному изображению можно вычислить сумму интенсивностей по какому-либо прямоугольному фрагменту, сложив или вычтя всего четыре числа.

Определим предсказанную метку на каком-либо объекте $x_i$ как:
$$\hat{y}_i = \mathrm{sign}\left(\sum_{j=1}^N \alpha_j f_j(x_i)\right),$$
где $N$ — количество признаков (оно зависит от $n$, $m$ и того, какие именно признаки включены: скажем, какая толщина линий рассматривается), $\alpha_j$ — обучаемый коэффициент при $j$-м признаке, $f_j(x_i)$ — значение $j$-го признака на $i$-м объекте. При таком определении модели обучить потребуется $2N$ параметров: $N$ коэффициентов при признаках и $N$ порогов для признаков. В методе Виолы-Джонса обучение проводится алгоритмом, близким к алгоритму AdaBoost.

#### Применение

На этапе детекции нужных объектов на изображении перебираются различные размеры рамки (но с отношением сторон, близким к отношению $n$ к $m$) и различные положения этой рамки относительно изображения. То, что попадает в текущую рамку, классифицируется детектором, и, если предсказывается положительная метка, рамка наносится, а иначе нет. Разумеется, может оказаться так, что один и тот же объект будет обведён в несколько немного сдвинутых друг относительно друга рамок.

Важно отметить, что при вычислении признаков размеры фрагментов, на которых они считаются, увеличиваются пропорционально тому, во сколько раз рамка больше, чем размер изображений, подававшихся на стадии обучения.

Не менее важно обсудить один приём, благодаря которому применение происходит быстро. Если бы вычислялись именно предсказания вышеописанной модели, для каждого нового положения текущей рамки потребовалось бы вычислять все признаки (а их сотни тысяч). Чтобы этого избежать, используется каскадирование (cascading). Оно заключается в том, что наряду с финальным классификатором, обученным на всех признаках, обучается также цепочка более слабых классификаторов, обученных на меньших подмножествах признаков. Все эти классификаторы должны иметь близкую к единице полноту, но их точность не обязана быть высокой. Если в каскаде поначалу легковесных и постепенно усложняющихся классификаторов текущее положение рамки будет отвергнуто каким-либо классификатором, оно отвергается и для всей процедуры. И только то, что через весь каскад дойдёт до финального классификатора и пройдёт его, принимается за участок с детектируемым объектом. Благодаря каскадированию большая часть рамок отвергается на ранних стадиях классификаторами, использющими небольшое число признаков.

## Метод подбора темпа обучения нейронной сети Adam

В классическом варианте градиентного спуска темп обучения один и тот же для всех настраиваемых весов. Исторически одним из первых методов, где это стало не так, был AdaGrad, главный недостаток которого оказался исправлен в методе под названием RMSProp. А метод, в полном виде называющийся Adaptive Moment Estimation и обычно сокращённо называемый Adam, считается дальнейшим развитием адаптивного управления темпом обучения.

Метод Adam имеет четыре гиперпараметра: $\eta$, $\beta_1$, $\beta_2$ и $\varepsilon$. Гиперпараметр $\eta$ задаёт относительную скорость обучения (все адаптивно подобранные темпы обучения содержат его как множитель). Гиперпараметр $\beta_1$ отвечает за сглаживание/угасание накопленного среднего градиентов, а гиперпараметр $\beta_2$ — за сглаживание/угасание накопленного второго момента градиентов, то есть накопленного среднего их квадратов. Наконец, $\varepsilon$ служит для численной стабильности.

Положим, что по аналогии с методом RMSProp формула обновления некого параметра $\theta$ (то есть одного из весов какого-либо слоя нейронной сети) на $t$-м шаге обучения имеет вид:
$$\theta_{t+1} := \theta_t - \frac{\eta}{\sqrt{v_t} + \varepsilon}m_t,$$
где $\theta_t$ — текущее значение параметра $\theta$, $\theta_{t+1}$ — его обновлённое значение, а $m_t$ и $v_t$ — текущие значения присущих алгоритму оптимизации внутренних параметров.

Эти два параметра алгоритма оптимизации $m_t$ и $v_t$ обновляются на каждом шаге по следующим правилам простого экспоненциального сглаживания:
$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_{\theta},$$
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g^2_{\theta},$$
где $g_{\theta}$ — градиент для параметра нейронной сети $\theta$ на $t$-м шаге обучения. Что касается формулы для $m_t$, экспоненциальное сглаживание в ней можно сравнить с инерцией, но только это уже инерция темпа обучения, а не инерция обновлений весов, часто используемая при обучении нейронных сетей. Что же касается формулы для $v_t$, в ней экспоненциальное сглаживание нужно не для повышения устойчивости, а для того чтобы накопленная сумма квадратов градиентов со временем могла и падать, а не только расти (главный недостаток AdaGrad как раз к тому и сводится, что там она никогда не падает).

Начальными условиями, с которых начинается эволюция во времени параметров алгоритма оптимизации $m_t$ и $v_t$, являются равенства их обоих нулю:
$$m_0 = 0,$$
$$v_0 = 0.$$
Раз так, то значения $m_t$ и $v_t$ при малых $t$ будут смещены в сторону нуля. Чтобы исправить это, вводят следующие скорректированные параметры:
$$\hat{m_t} = \frac{m_t}{1 - \beta_1^t},$$
$$\hat{v_t} = \frac{v_t}{1 - \beta_2^t}.$$

С учётом всего вышесказанного можно выписать окончательную формулу обновления весов методом Adam:
$$\theta_{t+1} := \theta_t - \frac{\eta}{\sqrt{\hat{v_t}} + \varepsilon}\hat{m_t}.$$

Напоследок же стоит заметить, что все методы наподобие AdaGrad, RMSProp или Adam предназначены для ускорения обучения, а не для улучшения качества обученной нейронной сети. Грубо говоря, они позволяют начать с высокого темпа обучения, чтобы побыстрее пройти начальную стадию. В статье же [Wilson et al, 2017](https://arxiv.org/abs/1705.08292) показывается, что иногда нейронные сети, обученные адаптивными методами, проигрывают нейронным сетям, обученным классическим градиентным спуском или стохастическим градиентным спуском. Таким образом, если данных мало, то стоит попробовать не только Adam, но и обычный градиентный спуск.

## Декомпозиция построения и обучения нейронной сети

В построении и обучении нейронных сетей есть три ключевых составляющих:

1. Модель, то есть архитектура и структура нейронной сети;
2. Функция потерь, то есть то, что оптимизируется при обучении;
3. Оптимизатор, то есть способ по данным настроить все параметры сети.

#### Модель

* Сколько слоёв?
* Какой ширины каждый из слоёв?
* Какие слои используются?
    - из обычных нейронов,
    - из [LSTM-модулей](__home_url__/notes/Внутреннее устройство LSTM-нейрона),
    - слои нормализации (например, [пакетной нормализации](__home_url__/notes/Пакетная нормализация)),
    - etc;
* Какие связи между слоями?
    - полные,
    - локальные (то есть каждый нейрон связан лишь с некоторой областью предыдущего слоя),
    - свёрточные (похожи на локальные, но веса в фильтре одинаковые для всего слоя),
    - рекуррентные,
    - остаточные (в обход одного или нескольких следующих слоёв),
    - etc;
* Какие функции активации используются в каждом из слоёв?
    - нет активации,
    - сигмоида,
    - гиперболический тангенс,
    - ReLU и его модификации, такие как Leaky ReLU или PReLU,
    - etc.
* Есть ли какие-либо связанные веса (shared weights)?

#### Функция потерь

* Для регрессии:
    - MSE, среднеквадратичная ошибка (ведёт к оптимизации правдоподобия обучающей выборки при гауссовском шуме в измерении целевой переменной),
    - MAE, средний модуль ошибки (ведёт к оптимизации правдоподобия обучающей выборки при лапласовском шуме в измерении целевой переменной),
    - etc.
* Для классификации:
    - [логарифмическая функция потерь](__home_url__/notes/Логарифмическая функция потерь и энтропия Шеннона) (ведёт к оптимизации правдоподобия обучающей выборки при отсутствии шумов в метках объектов обучающей выборки),
    - etc.
* Для других задач:
    - GAN loss,
    - style transfer loss,
    - etc.

#### Оптимизатор

* На базе градиентного спуска с таким приёмом, как обратное распространение ошибок (эта вычислительная техника сильно ускоряет подсчёт градиентов):
    - Когда обновлять веса?
        - После подсчёта коррекций на всех объектах обучающей выборки (классический градиентный спуск),
        - После подсчёта коррекций на всём пакете из случайно выбранных объектов обучающей выборки (пакетный градиентный спуск),
        - После подсчёта коррекций на каждом отдельно взятом объекте (стохастический градиентный спуск).
    - Какой темп обучения (learning rate) выбрать?
        - Постоянный,
        - С расписанием, то есть в виде заранее заданной невозрастающей функции от числа шагов,
        - Адаптивный:
            - AdaGrad,
            - RMSProp,
            - [Adam](__home_url__/notes/Метод подбора темпа обучения нейронной сети Adam).
    - Какую инерцию (momentum) обновлений весов выбрать?
        - никакую,
        - обычную,
        - инерцию Нестерова.
    - Как инициализировать веса каждого из слоёв перед началом обучения?
        - Случайно породить $I \times O$ весов, где $I$ — размер входа слоя, а $O$ — размер выхода слоя:
            - Из гауссовского распределения с нулевым средним и дисперсией:
                - 0.01 (или ещё меньше, если всё равно обучение не идёт),
                - $2 / (I + O)$ (Glorot Normal Initialization),
                - $1/I$ (часто используется для функции активации $\tanh$),
                - $2/I$ (He Normal Initialization, часто используется для функции активации ReLU);
            - Из равномерного распределения на отрезке $[-a, a]$, где $a$ равно:
                - $\sqrt{6 / (I + O)}$ (Glorot Uniform Initialization),
                - $\sqrt{3 / I}$ (LeCun Uniform Initialization).
        - Жадно послойно предобучить с дополнением до автокодировщика.
    - Какие техники регуляризации использовать?
        - $L_1$-регуляризацию (приводит к отбору весов),
        - $L_2$-регуляризацию (приводит к затуханию весов),
        - раннюю остановку, отталкивающуюся от оценки качества на отложенной выборке?
        - дропаут,
        - обогащение (augmentation) данных: в частности, подмешивание шума,
        - сглаживание целевой переменной (label smoothing).
* На базе генетических алгоритмов (не мэйнстрим в обучении нейронных сетей; что-то похожее применяется в обучении с подкреплением в [методе кросс-энтропии](https://github.com/udacity/deep-reinforcement-learning/blob/master/cross-entropy/CEM.ipynb) и в [эволюционных стратегиях](https://arxiv.org/pdf/1703.03864.pdf)).
* С использованием [вариационного вывода](__home_url__/notes/Вариационный байесовский вывод и mean-field approximation) (как правило, это вычислительно затратно, но зато поддерживается обучение дискретных мест, через которые не проходит градиент).

## Оценивание дисперсии целевой переменной

Если функцией потерь, с которой обучается регрессор, является среднеквадратичная ошибка (MSE), то регрессор будет пытаться предсказать условное среднее целевой переменной при условии признаков. Если взять средний модуль ошибки (MAE), то тогда получится оценка условной медианы целевой переменной при условии признаков. Оценка условной дисперсии целевой переменной при условии признаков тоже может быть получена, хотя это и не достигается за счёт выбора подходящей функции потерь. Нужна же оценка условной дисперсии, например, тогда, когда есть возможность дополнительно запрашивать примеры в обучающую выборку — из областей признакового пространства, где дисперсия выше, хочется иметь больше примеров, чем из областей признакового пространства, где дисперсия ниже.

Для получения оценки условной дисперсии применим следующий эвристический подход:
* С MSE в качестве функции потерь обучим модель с исходной целевой переменной;
* С MSE в качестве функции потерь обучим модель, для которой целевой переменной является квадрат исходной целевой переменной;
* Предсказанием дисперсии на некотором объекте назовём разность предсказания второй модели и квадрата предсказания первой модели, если эта разность больше 0, и 0 иначе.

Описанный подход опирается на определение дисперсии как разности математического ожидания квадрата и квадрата математического ожидания.

## Перенос модели из одной среды в другую

Предположим, что на некотором наборе признаков уже обучена модель, но все данные, из которых была составлена обучающая выборка, приходили из одной среды, а теперь хочется решать аналогичную задачу для объектов, приходящих из другой среды, где доступны только признаковые описания, но не целевые переменные.

Примеры подобных ситуаций таковы:
* Для решения задачи кредитного скоринга банк, работавший в некотором регионе, собрал достаточно размеченных примеров с жителями этого региона. Теперь банк хочет открыть отделение в другом регионе, однако там за короткое время можно собрать только заявки на кредит с признаками, но без целевой переменной, ведь для того, чтобы узнать, будет ли кредит возвращён, нужно ждать несколько лет.
* Мобильное приложение с моделью монетизации freemium (установка и базовое использование бесплатны, платны дополнительные возможности), долгое время выпускавшееся лишь под одну платформу, получило версию под другую платформу. Маркетологи хотят получать предсказание будущих поступлений от пользователей, привлекаемых в версию под новую платформу той или иной рекламной кампанией. Прямо сейчас доступны только параметры аудиторий и рекламных кампаний, но не конечный результ, ведь для его измерения необходимо ждать время, равное некой высокой квантили срока активности пользователя в приложении.

До тех пор, пока в новой среде не собрано достаточное количество размеченных данных, можно задействовать урезанную версию модели, обученную на данных из старой среды. Строится она так:
* Признаковые описания объектов из обеих сред вертикально конкатенируются, а новой целевой переменной становится бинарная переменная, равная 0, если объект пришёл из одной среды, и 1, если из другой.
* Обучается классификатор примерно той же природы, что и исходная модель, и смотрится, какие признаки обладают для него наибольшей важностью.
* Ищется подмножество признаков, такое что на нём классификатор не может показать хорошие метрики качества. Это подмножество признаков — подмножество таких признаков, что они с точки зрения моделей, похожих на выбранный классификатор, в совокупности выглядят одинаково в обеих средах.
* На отобранном подмножестве признаков только на объектах из старой среды обучается модель для исходной задачи, и именно эту модель можно использовать для предсказаний на объектах из новой среды.

Описанный приём позволяет избежать ошибок экстраполяции, связанных с тем, что модель применяется в ранее не виденных ей областях признакового пространства. В то же время использование лишь части признаков приводит к некоторому снижению качества модели. Стало быть, нужно найти оптимальный баланс между качеством модели и степенью неотличимости сред.

## Пакетная нормализация

#### Введение

Нейронные сети с нормализацией имеют то преимущество перед нейронными сетями без неё, что обучаются быстрее, а ещё их обучению сопутствует некоторая дополнительная регуляризация, благодаря которой порой не нужен дропаут.

Идея, стоящая за нормализацией, восходит к тому, что в машинном обучении бывает полезно центрировать данные и делать их дисперсию единичной. Однако в случае глубоких нейронных сетей классическое масштабирование (то есть вычитание из признаков их средних и деление на их среднеквадратичное отклонение) оказывается недостаточным, если его просто провести в рамках предобработки данных. Эффект от разовой предобработки может и не дойти до слоёв, расположенных далеко от входных. Значит, есть смысл делать масштабирование в каждом слое отдельно. Иными словами, есть смысл нормировать то, что какой-либо промежуточный слой подаёт на вход следующему слою. И помимо того, что это может быть полезно из общих соображений, в контексте нейронных сетей есть и ещё один довод за это: у функций активации, таких как сигмоида или гиперболический тангенс, градиенты заметно отличаются от нуля лишь в окрестности нуля, а поэтому подача на вход активации больших по модулю значений приведёт к затуханию распространения информации при обучении.

Обычное нормирование в каждом слое является далеко не лучшим решением, и поэтому были придуманы различные варианты более сложных процедур нормализации. Исторически первым вариантом нормализации была [пакетная нормализация](https://arxiv.org/pdf/1502.03167.pdf) (batch normalization), а впоследствии она была доработана до [пакетной ренормализации](https://arxiv.org/pdf/1702.03275.pdf), и, на самом деле, далее именно эта модифицированная версия будет называться пакетной нормализацией.

#### Перечень вносимых переменных

Со слоем пакетной (ре)нормализации ассоциированы следующие сущности:
* Два вектора, обучаемых пакетным градиентных спуском:
    - $\gamma$, вектор перемасштабирования, инициализируется вектором из одних единиц той же длины, что и размер предыдущего слоя,
    - $\beta$, вектор сдвига, инициализируется вектором из одних нулей той же длины, что и размер предыдущего слоя;
* Два вектора, существующих только во время обучения:
    - $\mu_\mathrm{batch}$, вектор средних по текущему пакету значений, подаваемых на вход слою пакетной нормализации,
    - $\sigma_\mathrm{batch}$, вектор среднеквадратичных отклонений значений, подаваемых на вход слою пакетной нормализации, подсчитанный по текущему пакету;
* Два вектора, арифметическими операциями обновляемые на каждом пакете во время обучения и остающиеся константами во время применения, нужны они только на стадии применения:
    - $\mu_\mathrm{cum}$, вектор экспоненциального среднего от векторов средних по пакетам значений, подаваемых на вход слою,
    - $\sigma_\mathrm{cum}$, вектор экспоненциального среднего от векторов внутрипакетных среднеквадратичных отклонений значений, подаваемых на вход слою.
    
К числу гиперпараметров пакетная нормализация добавляет только коэффициент сглаживания $\alpha$ для подсчёта экспоненциальных средних (или несколько таких коэффициентов для разных слоёв и отдельно для средних и среднеквадратичных отклонений).

#### Процесс обучения и применения

Рассмотрим слой пакетной нормализации, которому в пакете (mini-batch) подаются векторы-столбцы $u_i$, где $i$ принимает значения от 1 до размера пакета, а все $u_i$ имеют длину, равную размеру предыдущего слоя. Тогда обучение разбивается на такие шаги:
* Вычислить $\mu_\mathrm{batch}$ как среднее $u_i$ и вычислить $\sigma_\mathrm{batch}$ как среднеквадратичное отклонение $u_i$;
* Обновить значения $\mu_\mathrm{cum}$ и $\sigma_\mathrm{cum}$ по правилу простого экспоненциального сглаживания:
$$\mu_\mathrm{cum} := \alpha\mu_\mathrm{cum} + (1-\alpha)\mu_\mathrm{batch},$$
$$\sigma_\mathrm{cum} := \alpha\sigma_\mathrm{cum} + (1-\alpha)\sigma_\mathrm{batch};$$
* Вычислить нормализованные значения $v_i = (u_i - \mu_\mathrm{batch}) \: / \: (\sigma_\mathrm{batch} + \varepsilon)$, где $\varepsilon$ — некоторая малая константа, позволяющая избежать проблем с делением на ноль;
* Как вход для следующего слоя на соответствующем ($i$-м) объекте батча передать вектор $\gamma \circ v_i + \beta$, где $\circ$ обозначает поэлементное умножение;
* При обновлении параметров нейронной сети методом обратного распространения ошибок также считать градиенты по $\gamma$ и $\beta$, чтобы градиентный спуск сам мог решить, какими примерно должны быть средние и дисперсии, ведь это влияет, например, на то, какие участки нелинейности функции активации задействованы.

На стадии применения для нормализации всегда используются $\mu_\mathrm{cum}$ и $\sigma_\mathrm{cum}$, а $\gamma$ и $\beta$ полагаются выученными константными векторами.

#### Открытые вопросы и альтернативы

Связанные с пакетной нормализацией вопросы, на которые пока нет однозначных ответов, таковы:
* почему же всё-таки улучшается результат;
* стоит ли делать пакетную нормализацию сразу же перед функцией активации или же сразу же после неё (разные специалисты делают и так, и так).

Остановимся на первом вопросе. Попытаться объяснить, почему пакетная нормализация улучшает результаты, можно вот как. Когда в процессе обучения меняется распределение выходов некоторого слоя, следующий за ним слой оказывается обученным на предшествовавших этому входах, то есть на входах, имевших другое распределение, — это то, что называют внутренним сдвигом переменных (internal covariance shift), и это то, что якобы устраняется при помощи нормализации. Такое объяснение приводилось в оригинальной статье, но позже [было показано](https://arxiv.org/pdf/1805.11604.pdf), что это, в лучшем случае, вводная иллюстрация, но никак не настоящее объяснение. Новая гипотеза сводится к тому, что пакетная нормализация делает зависимость функции потерь от настраиваемых параметров более гладкой.

Помимо теоретических неясностей у пакетной нормализации есть и практические недостатки:
* нейронные сети с пакетной нормализацией могут обучаться только пакетным градиентным спуском;
* применение в рекуррентных нейронных сетях затруднено;
* при малом размере батчей качество может быть не на высоте.

В связи с этим появились многочисленные [альтренативы](https://mlexplained.com/2018/11/30/an-overview-of-normalization-methods-in-deep-learning/). Простой и в то же время эффективной и совместимой с реккурентными сетями альтернативой является послойная нормализация (layer normalization), где для каждого отдельного объекта происходит масштабирование по юнитам слоя. Реализовать это несложно, но вот теоретические объяснения, почему это работает, сложны.

## Субдискретизация

Субдискретизация (pooling) устроена так: по входу слоя проходит фильтр (filter, kernel) и каждый раз от всего, что попадает в него, берёт значение некоторой агрегирующей функции, становящееся значением в соответствующем узле (нейроне) (unit, neuron).

Например, в качестве агрегирующей функции можно взять максимум — тогда речь пойдёт о субдискретизации максимумом (max pooling). Её часто используют для задач, связанных с изображениями.

Какие желаемые свойства добавляет субдискретизация максимумом?
* Возникает устойчивость к сдвигам изображения на небольшое количество пикселей;
* Появляется возможность реализовать следующую логику: присутствие детектируемого объекта на объединении участков зависит от того, присутствует ли объект хотя бы на одном из этих участков, а не от доли участков, на которых он присутствует.

Для анализа же текстов порой используют так называемый максимум во времени (max over time pooling). Агрегирующей функцией по-прежнему является максимум, но фильтр одномерный и в длину имеет размер всего объекта (текста или предложения).

Также субдискретизация может увеличивать количество измерений в слое. Происходит это, если агрегирующая функция возвращает не одно значение, а несколько. Например, K-max pooling предполагает взятие K наибольших значений.

## Задача управления на базе машинно-обученной модели

Машинное обучение может использоваться для решения задачи предсказания, а может использоваться для решения задачи управления. Чтобы увидеть разницу между этими двумя задачами, набросаем общий сценарий и введём некоторое количество базовых сущностей (выделены курсивом). Итак, пусть машинно-обученная _модель_ возвращает предсказания, _пользователь_ модели, получив эти предсказания, принимает _решение_, связанное с каким-либо _процессом_, а конечный _результат_ для пользователя зависит как от его решения, так и от того, как пройдёт процесс. В рамках такого описания задача является задачей предсказания, если решение пользователя не способно повлиять на внутренние стороны процесса, и является задачей управления, если решение пользователя изменяет протекание процесса. Сценарий можно и расширить, допустив, что в задаче управления пользователь также может влиять на признаки объектов и выбирать, для объектов с какими признаками получать предсказания.

Примером задачи предсказания является задача кредитного скоринга. Здесь модель по признакам заёмщика предсказывает вероятность того, что кредит будет возвращён, пользователем является банк, решением является выдавать или нет кредит какому-то конкретному заёмщику, а процессом является процесс выплаты кредита этим заёмщиком. Поскольку платежеспособность заёмщика не зависит от того, в каком банке он получит кредит на условиях, идентичных условиям, предложенным банком, это именно задача предсказания.

Примером задачи управления является задача планирования промо-акций в розничных сетях. В одной из наиболее простых постановок рассматривается всего один товар, и модель по размеру скидки, прошлой истории продаж, дню недели, прогнозу погоды и некоторым другим факторам должна предсказать, сколько единиц товара купят завтра. Здесь пользователем модели является розничная сеть, решением является выставленная скидка, а процессом является поведение покупателей в магазине, а именно то, как они берут продвигаемый товар. Результатом становится валовая прибыль, то есть суммарное количество приобретённых единиц товара, умноженное на отпускную цену. Описанная задача не является задачей предсказания, потому что размер скидки напрямую влияет на покупательское поведение.

В задачах управления качество предсказательной модели иногда бывает сложно оценить на реальных данных. Например, менеджеры, выбрав изначальную скидку и увидев, что весь запас товара разберут, могут решить уменьшить скидку, но если условия сервиса с моделью, которым они пользуются, предполагают ограничение на то, что запросов на предсказание может быть не более одного в день, то предсказание спроса для новой скидки уже не будет получено, и фактический спрос будет не с чем сравнивать.

Однако обычно сценарий использования моделей для задачи управления другой. Предсказательная модель встраивается в функцию от управляемых и неуправляемых параметров, возвращаемое значение которой нужно оптимизировать по управляемым параметрам, соблюдая наложенные на них ограничения. Скажем, если в задаче планирования промо-акций предоставленная моделью функция $\mathrm{predict}$ предсказывает спрос по размеру скидки, температуре воздуха и вчерашнему спросу, то оптимизируемая функция валовой прибыли имеет вид:
$$f(\mathrm{discount}, \mathrm{temperature}, \mathrm{lag}) = (\mathrm{initial\_price} - \mathrm{discount}) * \mathrm{predict}(\mathrm{discount}, \mathrm{temperature}, \mathrm{lag}).$$
Эту функцию требуется оптимизировать по $\mathrm{discount} \in [0; \mathrm{initial\_price}]$, и это дополнительная задача оптимизации по суррогатной модели. Модель названа суррогатной, потому что была оценена по данным, а не выведена на основании законов и/или правил, относящихся к предметной области.

Задача управления на базе машинно-обученной модели имеет следующие нюансы:
* Может присутствовать ограничение на используемые методы машинного обучения (например, ансамбли над решающими деревьями возвращают функцию $\mathrm{predict}$, являющуюся кусочно-постоянной и потому имеющую неинформативную производную);
* Помимо ошибок, содержащихся в данных, и ошибок, вызываемых аппроксимацией целевой переменной, появляются ошибки оптимизации (скажем, вызванные локальными экстремумами), то есть нужно быть требовательнее к данным и стараться обеспечить запас качества модели;
* Наличие предсказательной силы не гарантирует наличие причинно-следственных связей; в частности, к истинному влиянию управляемого параметра моделью может быть ошибочно приписано влияние пропущенных параметров, скоррелированных с этим управляемым параметром. В эконометрике такие явления называют эндогенностью. Как следствие же, модель нужно обучать не только ради точного предсказания целевой переменной, но и ради достоверной оценки влияния факторов на эту целевую переменную.

## Логарифмическая функция потерь и энтропия Шеннона

Логарифмическая функция потерь (log loss) имеет вид:
$$\mathrm{log\_loss}\left(y, \hat{P}\right) = -\frac{1}{n}\sum_{i = 1}^n\sum_{j = 1}^c [y_i = j] \, \log \hat{P}_{ij},$$
где $n$ — количество объектов, $c$ — количество классов, $y$ — вектор длины $n$, такой что его $i$-я компонента $y_i$ равна верному ответу на $i$-м объекте выборки, $\hat{P}$ — матрица размера $n \times c$, такая что на пересечении $i$-й строки и $j$-го столбца стоит предсказанная вероятность принадлежности $i$-го объекта к $j$-му классу, а, наконец, квадратные скобки даны в нотации Айверсона, то есть выражение с ними равно 1, если то, что в скобках, верно, и равно 0, если то, что в скобках, неверно.

Логарифмическая функция потерь с точностью до константы является логарифмом от правдоподобия обучающей выборки. Если обозначить за $Y$ матрицу, $i$-й строкой которой является вектор, в котором везде стоят нули кроме содержащей единицу $j$-й позиции, где $j$ — истинная метка $i$-го объекта, то правдоподобие обучающей выборки относительно получившейся модели $\hat{P}$ записывается в виде:
$$\prod_{i=1}^n\prod_{j=1}^c \hat{P}_{ij}^{Y_{ij}}.$$
Обратить внимание стоит на то, что пронаблюдавшиеся метки объектов считаются детерминированными, то есть предполагается, что к меткам не подмешивается случайная ошибка, способная их исказить. Именно из-за детерминированности меток матрица $Y$ берётся в виде, где в каждой строке кроме нулей есть ровно одна единица, что после логарифмирования даёт множители вида $[y_i = j]$.

Помимо интерпретации, отталкивающейся от максимизации правдоподобия, логарифмическая функция потерь имеет интерпретацию, восходящую к теории информации.

Чтобы понять, в чём именно связь, обратимся к понятию энтропии Шеннона. Энтропия Шеннона — функционал на вероятностных распределениях. Поскольку речь идёт о $c$-классовой классификации, определим энтропию Шеннона только на вероятностных распределениях на множестве $c$ классов, хотя на самом деле она определена и для непрерывных распределений:
$$H(p) = - \sum_{i = 1}^c p_i \log_2{p_i},$$
где $p = (p_i)_{i=1}^c$ — вектор длины $c$, такой что его $i$-я компонента $p_i$ равна вероятности того, что просэмплированный из генеральной совокупности объект принадлежит к $i$-му классу.

Какой смысл в так определённой величине? Короткий ответ: она сообщает, чему равно математическое ожидание (относительно генеральной совокупности) количества бит информации, получаемой при измерении класса одного случайного объекта. Например, если все классы кроме одного имеют нулевую вероятность, то $H(p) = 0$, ведь никакой информации в измерении класса нет: и без измерений известно, что может быть только один класс.

Из короткого ответа непонятно, почему энтропия Шеннона, и впрямь, является ожидаемым количеством бит информации, даваемым замером класса на одном случайном объекте. Чтобы углубиться в это, придётся немного формализовать задачу. Допустим, метку каждого класса представили в виде последовательности нулей и единиц (т.е. последовательности бит) так, что у разных классов разные метки. При вышеописанной кодировке введём $l$ как вектор длины $c$, $l = (l_i)_{i=1}^c$, $i$-я компонента которого $l_i$ равна длине последовательности нулей и единиц, кодирующей метку $i$-го класса. Задача ставится как задача минимизации по возможным способам закодировать метки классов (так, чтобы разные классы имели разные метки) следующего функционала, являющегося математическим ожиданием длины последовательности, представляющей класс случайного объекта:
$$L = \sum_{i = 1}^c p_i l_i.$$
Шеннон показал, что оптимальное значение $L_{opt}$ не может быть меньше, чем $H(p)$. Более того, существуют схемы, позволяющие приблизиться к этому теоретическому минимуму, а именно коды Хаффмана и коды Шеннона-Фано. Таким образом, становится понятно, почему $H(p)$ можно интерпретировать как ожидаемое количество бит информации, ведь в вышеописанной тракторвке это ожидаемая длина битового представления метки класса при оптимальном кодировании меток.

Чтобы понять, как логарифмическая функция потерь связана с энтропией Шеннона, введём понятие кросс-энтропии между двумя задаваемыми векторами $p$ и $q$ вероятностными распределениями на метках классов:
$$H(p, q) = - \sum_{i=1}^c p_i \log_2{q_i}.$$
Эта величина может восприниматься следующим образом. Допустим, вектор $p$ содержит истинные вероятности классов, но при выборе способа закодировать метки классов думали, что истинные вероятности классов задаются вектором $q$, так что и кодирование получилось оптимальным для $q$. В таком случае $H(p, q)$ задаёт ожидаемое количество бит информации, которую потребуется передать, чтобы с указанным оптимальным для $q$ кодированием сообщить метку класса объекта, пришедшего из распределения, характеризуемого $p$. Только что кросс-энтропия $H(p, q)$ была описана с точки зрения разрастания количества бит, которое потребуется передать, но есть и взгляд с точки зрения потери информации: если разрешено передать лишь столько бит, сколько ровно хватило бы при оптимальном для $p$ кодировании, то при оптимальном для $q$ кодировании часть информации придётся потерять.

Так вот, логарифмическая функция потерь — средняя по выборке кросс-энтропия между распределением, сконцентрированным в истинном классе объекта, и распределением, задаваемым предсказанными для этого объекта вероятностями классов.

## Дивергенция Кульбака-Лейблера

Дивергенция Кульбака-Лейблера — асимметричное "расстояние" между двумя вероятностными распределениями. Из-за асимметричности и используется слово "дивергенция", а не слово "расстояние".

В случае распределений над дискретным множеством $c$ возможных классов любое вероятностное распределение можно представить в виде вектора длины $c$, сумма компонент которого равна 1. Тогда для распределений, характеризуемых векторами $p=(p_i)_{i=1}^c$ и $q=(q_i)_{i=1}^c$, дивергенция Кульбака-Лейблера равна:
$$D_{KL}\left(p \, \Vert \, q \right) = - \sum_{i=1}^c p_i \log \frac{q_i}{p_i}.$$

Смысл так введённой величины становится понятен из следующего соотношения:
$$D_{KL}\left(p \, \Vert \, q \right) = H(p, q) - H(p),$$
где $H(p, q)$ — кросс-энтропия между распределениями, описываемыми векторами $p$ и $q$, а $H(p)$ — [энтропия Шеннона](__home_url__/notes/Логарифмическая функция потерь и энтропия Шеннона) распределения, описываемого вектором $p$. Из выписанного равенства следует, что дивергенция Кульбака-Лейблера показывает, чему равно ожидаемое количество "лишних" бит, которые потребуется передать для описания исхода, приходящего из распределения, описываемого $p$, если для кодирования исходов была выбрана схема, оптимизированная для распределения, описываемого $q$. Также из выписанного равенства следует, что при использовании в качестве функции потерь дивергенции Кульбака-Лейблера между распределением, сконцентрированным в истинном классе объекта, и предсказанными вероятностями классов, получится то же самое, что получается при использовании логарифмической функции потерь, так как $H(p)$ не зависит от предсказанного $q$.

Свойства дивергенции Кульбака-Лейблера:
* Неотрицательна: $D_{KL}\left(p \, \Vert \, q \right) \ge 0$, причём равна 0 тогда и только тогда, когда $p = q$ (следует из неравенства Йенсена);
* С точностью до умножения на константу является предельным элементом параметрического семейства дивергенций Кресси-Рида в пределе при $\lambda \to 0$:
$$D_{KL}\left(p \, \Vert \, q \right) = -\frac{1}{2} \lim_{\lambda \to 0} \frac{2}{\lambda(\lambda + 1)} \sum_{i=1}^c p_i\left(\left(\frac{p_i}{q_i}\right)^\lambda - 1\right),$$
следует это из правила Лопиталя. К этому же параметрическому семейству с точностью до умножения на константу принадлежат расстояние Хеллингера и $\chi^2$-расстояние.

## Общий взгляд на EM-алгоритм

#### Введение

В задаче кластеризации EM-алгоритм может использоваться для разделения смеси гауссовских распределений. Он имеет ряд преимуществ перед методом K-средних. Во-первых, вместо бинарных индикаторов принадлежности к кластеру вводятся нечёткие степени принадлежности к кластеру. Во-вторых, ковариационные матрицы гауссиан не обязаны быть единичными (как это неявно предполагается при измерении евклидова расстояния до центров кластеров в алгоритме K-средних)— для отсутствия вычислительной неустойчивости достаточно, чтобы они были диагональными, хотя есть и версии с более слабыми ограничениями.

На самом же деле, EM-алгоритм имеет более общий вид, чем алгоритм для разделения смеси гауссовских распределений. Это алгоритм, позволяющий делать две вещи:
* оценивать параметры распределений тогда, когда часть переменных относится к ненаблюдаемым;
* решать задачу статистического вывода (statistical inference), то есть задачу оценивания значений ненаблюдаемых признаков объекта по его значениям наблюдаемых признаков.

Более того, EM-алгоритм может применяться и к выборкам, где векторы объектов не являются независимыми одинаково распределёнными: например, он может применяться к последовательностям (таким как марковские цепи).

#### Постановка задачи

Формализуем постановку задачи так.

Пусть есть следующие сущности:
* $X$ — $(l \times n)$-матрица наблюдаемых переменных, где $l$ — количество объектов в выборке, а $n$ — количество наблюдаемых признаков;
* $T$ — $(l \times h)$-матрица скрытых переменных, где $h$ — количество скрытых признаков, а $l$ то же самое, что и в матрице $X$. Для получения полных признаковых описаний объектов выборки матрицу $X$ нужно соединить с матрицей $T$ боковой конкатенацией;
* $\theta$ — неизвестный вектор параметров вероятностного распределения, породившего выборку; этот вектор требуется оценить по данным; в обычной постановке это детерминированный вектор, в байесовской же постановке (рассматриваемой лишь в самом конце заметки) это случайный вектор со своим собственным распределением.

Эти три сущности связаны между собой через функцию совместной условной плотности $p(X, T \, \vert \, \theta)$. В рассматриваемой задаче эта функция дана в аналитическом виде для любой длины $l$ матриц $X$ и $T$.

Тут уместно пояснить, почему выше используются матрицы $X$ и $T$ вместо векторов-строк $x$ и $t$, соответствующих какому-либо одному объекту. Дело в том, что различные строки матриц $X$ или $T$ могут не быть независимыми друг от друга, так что в общем случае функцию плотности нужно брать не от отдельных объектов выборки, а от всей выборки сразу. И, на самом деле, постановку можно ещё более обобщить, определив $X$ как кортеж (tuple) наблюдаемых переменных, $T$ — как кортеж скрытых переменных, а $\theta$ — как кортеж параметров модели.

Задача заключается в том, чтобы найти оценку для $\theta$ методом максимизации неполного правдоподобия, то есть правдоподобия только матрицы наблюдаемых переменных $X$, а не совместного правдоподобия $X$ и $T$: 
$$\hat{\theta} = \arg\max_\theta p(X \, \vert \, \theta) = \arg\max_\theta \int p(X, T \, \vert \, \theta) dT.$$
Неполное правдоподобие используется, потому что значение матрицы $T$ не дано, а дано только значение матрицы $X$.

Имея оценку $\hat{\theta}$, можно по матрице с наблюдаемой частью признакового описания объектов $X$ выводить матрицу ненаблюдаемой части признакового описания этих объектов $T$ следующим образом:
$$\hat{T} = \arg\max_T p(X, T \, \vert \, \hat{\theta}).$$
Это точечная оценка, а из описания E-шага станет понятно, как получать оценки распределения матрицы $T$.

#### Пример частной постановки

Задача разделения смеси гауссовских распределений является частным случаем описанной задачи, получающимся из неё, если положить, что:
* $X$ — $(l \times n)$-матрица наблюдаемых признаков объектов;
* $T$ — $(l \times 1)$-матрица (т.е. вектор-столбец) с дискретной скрытой переменной, задающей идентификатор (номер) компоненты смеси, откуда был порождён объект;
* $\theta$ — неизвестный вектор, составленный из:
    - вероятностей компонент смеси $p(i)$, где $i$ — скалярный идентификатор компоненты смеси,
    - центров гауссовских распределений $(\mu_i)_{i=1}^m$, где $m$ — количество компонент смеси,
    - ковариационных матриц гауссовских распределений $(S_i)_{i=1}^m$, развёрнутых из двумерного в одномерный вид, чтобы стать компонентами вектора $\theta$.

Чтобы показать, что задача разделения смеси гауссовских распределений является частным случаем поставленной задачи, необходимо и достаточно убедиться, что $p(X, T \, \vert \, \theta)$ дана в аналитическом виде. Поскольку в задаче разделения смеси гауссовских распределений объекты являются независимыми одинаково распределёнными, верно, что:
$$p(X, T  \, \vert \, \theta) = \prod_{i=1}^l p(x_{i\cdot}, t_{i\cdot}  \, \vert \, \theta),$$
где $x_{i\cdot}$ обозначает $i$-ю строку матрицы $X$, а $t_{i\cdot}$ обозначает $i$-ю строку матрицы $T$. Для векторов-строк $x_{i\cdot}$ и $t_{i\cdot}$ (т.е. матриц размера $1 \times n$ и $1 \times 1$ соответственно) плотность имеет следующий вид: 
$$p(x_{i\cdot}, t_{i\cdot} \, \vert \, \theta) = p(t_{i\cdot}) \mathcal{N}_{\mu_{t_{i\cdot}}, S_{t_{i\cdot}}}(x_{i\cdot}),$$
где в $(1 \times 1)$-матрице $t_{i\cdot}$ хранится скалярный идентификатор компоненты смеси, $\mathcal{N}_{\mu_{t_{i\cdot}}, S_{t_{i\cdot}}}(x_{i\cdot})$ — плотность гауссовского распределения с соответствующими параметрами в точке с координатами $x_{i\cdot}$, а $p(t_{i\cdot})$, $\mu_{t_{i\cdot}}$ и $S_{t_{i\cdot}}$ извлечены из $\theta$. Подстановкой этого равенства в предыдущее и получается аналитический вид для $p(X, T  \, \vert \, \theta)$.

#### Обозначения

Обозначим за $q(T)$ плотность некоторого (не обязательно известного) вероятностного распределения матрицы $T$. В дальнейшем будет считаться, что эта плотность приближает плотность апостериорной вероятности $p(T \, \vert \, X, \theta)$. Например, в задаче разделения смеси гауссовских распределений $q(T)$ хранит информацию о сформировавшихся к текущей итерации EM-алгоритма нечётких степенях принадлежности к кластерам.

Наконец, для определённости в том, использовать ли знак интеграла или знак суммы, будем считать, что $T$ является матрицей с элементами из $\mathbb{R}$. Если элементы $T$ дискретны, то интегралы заменятся на суммы, но содержательно ничего не изменится.

#### Нижняя вариационная оценка

Для решения общей задачи предпримем следующие шаги, на каждом из которых будем преобразовывать логарифм правдоподобия наблюдаемой выборки:
* $\log p(X \, \vert \, \theta) = \int q(T) \log p(X \, \vert \, \theta) dT$, потому что левая часть не зависит от $T$, а справа её просто внесли как константу в интеграл, равный 1;
* правую часть только что выписанного равенства представим в виде $\int q(T) \log \frac{p(X, T \, \vert \, \theta)}{p(T \, \vert \, X, \theta)} dT$ — сделать это можно по определению условной вероятности;
* домножим выражение под логарифмом на единицу, домножив числитель и знаменатель на одно и то же: $\int q(T) \log \frac{p(X, T \, \vert \, \theta) \, q(T)}{p(T \, \vert \, X, \theta) \, q(T)} dT$;
* воспользуемся тем, что логарифм произведения равен сумме логарифмов, а множители сгруппируем крест-накрест: $\int q(T) \log \frac{p(X, T \, \vert \, \theta)}{q(T)} dT + \int q(T) \log \frac{q(T)}{p(T \, \vert \, X, \theta)} dT$.

Первое слагаемое обозначим за $\mathcal{L}(q, \theta)$:
$$\mathcal{L}(q, \theta) = \int q(T) \log \frac{p(X, T \, \vert \, \theta)}{q(T)} dT.$$
Отметим, что в правой части присутствует $X$, но он не выписан как аргумент, от которого зависит $\mathcal{L}(q, \theta)$. Опустить $X$ можно по следующей причине: это уже заданная константная матрица пронаблюдавшихся значений. Таким образом, $\mathcal{L}(q, \theta)$ является функционалом, зависящим от функции плотности $q$ и вектора $\theta$.

Кроме того заметим, что второе слагаемое в получившемся на последнем шаге выражении равно дивергенции Кульбака-Лейблера между некоторым распределением $q(T)$ и распределением $p(T \, \vert \, X, \theta)$, которое им приближается по предположению, сделанному при описании $q(T)$ (впрочем, порядок аргументов дивергенции Кульбака-Лейблера такой, как если бы, наоборот, распределение $q(T)$ приближалось распределением $p(T \, \vert \, X, \theta)$):
$$D_{KL}(q(T) \, \Vert \, p(T \, \vert \, X, \theta)) = \int q(T) \log \frac{q(T)}{p(T \, \vert \, X, \theta)} dT.$$

Итак, путём вышеописанных преобразований было получено, что:
$$\log p(X \, \vert \, \theta) = \mathcal{L}(q, \theta) + D_{KL}(q(T) \, \Vert \, p(T \, \vert \, X, \theta)).$$
Второе слагаемое правой части неотрицательно в силу соответствующего свойства дивергенции Кульбака-Лейблера. Из этих соображений первое слагаемое называют нижней вариационной оценкой для $\log p(X \, \vert \, \theta)$.

#### Описание EM-алгоритма

Теперь всё готово, для того чтобы описать EM-алгоритм, пошагово максимизирующий то по $q$, то по $\theta$ прологарифмированное правдоподобие $\log p(X \, \vert \, \theta)$:
* E-шаг: при фиксированном $\theta = \theta^{(n)}$ решается задача $\mathcal{L}(q, \theta^{(n)}) \to \max_{q \in Q}$, где $Q$ — некоторое заранее выбранное множество вероятностных распределений. Для получения общего представления о том, как проводится такая максимизация, рассмотрим случай, когда $Q$ — множество всех вероятностных распределений. В этом случае делаются точные E-шаги, а решение оптимизационной задачи получается тогда, когда обнуляется дивергенция Кульбака-Лейблера, поскольку $\mathcal{L}(q, \theta^{(n)})$ не может быть больше $\log p(X \, \vert \, \theta^{(n)})$, в рамках текущего шага являющегося константой. Значит, решением задачи максимизации является распределение $q^{(n+1)}(T) = p(T \, \vert \, X, \theta^{(n)}) = \frac{p(X, T \, \vert \, \theta^{(n)})}{\int p(X, T \, \vert \, \theta^{(n)}) dT}$, где правая часть (с точностью до взятия интеграла из знаменателя) известна, потому что $X$ известен, $\theta^{(n)}$ известен и по предоположению, сделанному в формулировке задачи, известен аналитический вид $p(X, T \, \vert \, \theta)$. Так как параметр $\theta = \theta^{(n)}$ зафиксирован, можно сказать, что из параметрического семейства распределений $p_{\theta}(X, T) = p(X, T \, \vert \, \theta)$ выбрано какое-то одно $p(X, T \, \vert \, \theta^{(n)})$ и то, что делается на E-шаге, является решением задачи [статистического вывода](__home_url__/notes/Байесовский статистический вывод) для этого распределения;
* М-шаг: при фиксированном $q = q^{(n)}$ решается задача $\mathcal{L}(q^{(n)}, \theta) \to \max_{\theta}$. Поскольку логарифм частного равен разности логарифмов делимого и делителя, а распределение $q^{(n)}$ зафиксировано, задача эквивалентна задаче $\mathbb{E}_{T \sim q^{(n)}} \log p(X, T \, \vert \, \theta) \to \max_{\theta}$, где матрица $T$ случайна и именно по её распределению берётся математическое ожидание. Вообще говоря, выписанная оптимизационная задача может быть многоэкстремальной, но если $Q$ — экспоненциальный класс распределений, то задача выпуклая (максимизируется вогнутая функция).

#### Различные версии EM-алгоритма

В зависимости от вида ограничения $q \in Q$ возникают различные версии EM-алгоритма:
* Точный EM-алгоритм. В нём $Q$ — множество всех вероятностных распределений. Этот пример уже разбирался в описании EM-алгоритма. Осталось лишь заметить, что на E-шаге при вычислении решения требуется взять интеграл $\int p(X, T \, \vert \, \theta^{(n)}) dT$, но он может не браться. Проблема может возникнуть, если распределения $p(X \, \vert \, T, \theta^{(n)})$ и $p(T \, \vert \, \theta^{(n)})$ не образуют пару сопряжённых друг к другу. Поэтому иногда приходится брать как $Q$ нечто более узкое, чем множество всех вероятностных распределений, и проводить приближённые E-шаги.
* Жёсткий (crisp) EM-алгоритм. В нём $Q$ — множество всех атомарных распределений, сконцентрированных в одной точке (в данном случае, в одной матрице): $Q = \{\delta(T - T_0) \, \vert \, T_0 \in \mathbb{R}^{l \times h}\}$, где $\delta$ — дельта-функция Дирака. При таком ограничении E-шаг всегда можно провести: из определения дивергенции Кульбака-Лейблера получится, что решение — это распределение, сконцентрированное в точке моды по $T$ распределения $p(T \, \vert \, X, \theta^{(n)}))$, то есть в точке $\arg \max_{T} p(T \, \vert \, X, \theta^{(n)}))$. Говоря абстрактно, можно сказать, что в общем случае задача E-шага является задачей функциональной оптимизации (оптимум ищется по вероятностному распределению $q$), а такие задачи чаще всего не могут быть решены средствами современной математики, однако сужением $Q$ до параметрического семейства задачу можно свести к задаче оптимизации по вещественным параметрам (например, все элементы семейства атомарных распределений биективно сопоставляются координатам точки/матрицы, где сконцентрирована вся масса).
* Вариационный EM-алгоритм. В нём $Q$ — множество распределений, допускающих факторизацию по строкам: $Q = \{ q(T)\, \vert \, q(T) = \prod_{j=1}^l q_j(t_{j\cdot})\}$, где $q_j(t_{j\cdot})$ — некоторое распределение уже только одного вектора-строки, соответствующего $j$-му наблюдению (на самом деле, такое предположение приводит к naive mean-field approximation, но можно ослабить ограничение, вместо отдельных строк перейдя к непересекающимся множествам строк, что приведёт к generalized mean-field approximation). Заметим, что дельта-функция от матрицы раскладывается в произведение дельта-функций от своих строк, так что вариационный EM-алгоритм охватывает все варианты, доступные жёсткому EM-алгоритму. На E-шаге решение $q^{(n+1)}$ ищется с использованием вариационного подхода (покоординатного подъёма), и при соблюдении некоторых условий это решение можно найти.

Наконец, отдельно стоит выделить байесовский EM-алгоритм, где $\theta$ рассматривается как случайный вектор, имеющий своё собственное известное априорное распределение $p(\theta)$, а оценить требуется апостериорные распределения $p(\theta \, \vert \, X)$ и $p(T \, \vert \, X, \theta)$. Введение $p(\theta)$ влечёт за собой то, что вместо $p(X, T \, \vert \, \theta)$ в выкладках фигурирует $p(X, T, \theta) = p(X, T \, \vert \, \theta) p(\theta)$. E-шаг при этом не изменится, потому что $p(\theta^{(n)})$ сократится из выражения для $p(T \, \vert \, X, \theta^{(n)})$, а вот на M-шаге будет решаться следующая задача:
$$\mathbb{E}_{T \sim q^{(n)}} \log p(X, T, \theta) \to \max_{\theta}.$$
Иными словами, на M-шаге по сравнению с версией без априорного распределения на $\theta$ к оптимизируемому выражению просто добавится слагаемое $\log p(\theta)$.

#### Преимущества EM-алгоритма

Вспомним поставновку задачи, для решения которой был предложен EM-алгоритм и зададимся вопросом, почему нельзя было просто максимизировать неполное правдоподобие так же, как максимизируют обычное правдоподобие. Ответ заключается в следующем:
* Хотя подынтегральное выражение $p(X, T \, \vert \, \theta)$ и дано в аналитическом виде, часто бывает так, что сам интеграл $\int p(X, T \, \vert \, \theta) dT$ в аналитическом виде не берётся, так что брать градиент не от чего.
* Даже если неполное правдоподобие удаётся вычислить аналитически, EM-алгоритм, как правило, работает быстрее, чем градиентный спуск. Конечно, если $l$ велико, то используют стохастический градиентный спуск, но существует и стохастический EM-алгоритм, где на каждой итерации используется лишь часть выборки.
* На выходе также получается информация о $T$, а это может помочь лучше понять данные.
* Если часть элементов матрицы $X$ пропущена, не нужно применять эвристики, не имеющие под самой математического обоснования (например, такие как замена пропусков на среднее по столбцу). Просто нужно немного изменить постановку, к наблюдаемым переменным отнеся только то, что известно, и включив пропуски в кортеж скрытых переменных.

## Байесовский статистический вывод

#### Задача статистического вывода

Пусть имеются следующие сущности:
* кортеж (tuple) наблюдаемых переменных $X$, индексируемый одним индексом $i$; чаще всего будем считать, что этот кортеж составлен из известной выборки, то есть из матрицы объекты-признаки, содержащей только наблюдаемые признаки объектов и имеющей размеры $l \times n$, где $l$ — количество объектов, а $n$ — количество наблюдаемых признаков, в таком случае запись $X_i$ будет обозначать $i$-ю строку этой матрицы;
* кортеж скрытых переменных $Z$, индексируемый одним индексом $i$; в $Z$ может входить матрица скрытых признаков объектов $T$ размера $l \times h$, где $h$ — количество скрытых признаков, и тогда запись $Z_i$, где $1 \le i \le l$, обозначает $i$-ю строку матрицы $T$, то есть вектор-строку всех скрытых признаков $i$-го объекта; также в $Z$ могут входить неизвестные параметры модели, и тогда запись $Z_i$, где $l < i$, обозначает $(i-l)$-й параметр модели;
* вероятностная модель, то есть аналитический вид совместного распределения $p(X, Z)$.

Плотность вероятностного распределения дана именно для кортежей $X$ и $Z$, а не для одного произвольного объекта и параметров модели по ряду причин: в том числе по той, что в общем случае объекты могут не быть независимы друг от друга. Например, ими могут быть наблюдения элементов одной и той же последовательности с влиянием прошлого на будущее.

Задача заключается в нахождении оценки $\hat{Z}$ для неизвестного кортежа $Z$, соответствующего известному кортежу $X$. Также задачу можно расширить до такой: оценить апостериорное распределение $p(Z \, \vert \, X)$, а не только найти точечную оценку $\hat{Z}$.

#### Пример

Иллюстративным примером заадчи статистического вывода является задача байесовского обучения линейной регрессии, сформулированная в терминах нижеследующих сущностей:
* $X$ — кортеж из обучающей выборки, то есть из строк матрицы объекты-признаки, где помимо собственно признаков есть и целевая переменная;
* $Z$ — кортеж $(w, \alpha, \beta)$, где $w$ — $(n \times 1)$-вектор весов линейной модели, $\alpha$ — скалярный гиперпараметр регуляризации, а $\beta$ — единица, делённая на дисперсию шума, присутствующего в наблюдениях;
* вероятностная модель имеет вид $p(X, Z) = p(X \, \vert \, Z) p(Z)$, где:
    - $p(X \, \vert \, Z) = \prod_{i=1}^l \mathcal{N}_{x_i w, \beta^{-1}}(y_i)$, где $\mathcal{N}_{x_i w, \beta^{-1}}(y_i)$ — плотность в точке $y_i$ нормального распределения со средним $x_i w$ и дисперсией $\beta^{-1}$, $x_i$ — $(1 \times n)$-вектор только признаков $i$-го объекта, $y_i$ — значение целевой переменной на $i$-м объекте;
    - $p(Z) = p(w \, \vert \, \alpha) p(\alpha) p(\beta)$,
    - $p(w \, \vert \, \alpha) = \mathcal{N}_{0, \alpha^{-1}I}(w)$, где $I$ — единичная матрица размера $n \times n$,
    - $p(\alpha) = \Gamma_{a, b}(\alpha)$, $p(\beta) = \Gamma_{c, d}(\beta)$, где $\Gamma$ — плотность гамма-распределения, а $a$, $b$, $c$ и $d$ — константные гиперпараметры априорных распределений.

#### Байесовский подход к задаче статистического вывода

При байесовском подходе предполагается, что $p(X, Z) = p(X \, \vert \, Z) p(Z)$, где в аналитическом виде даны и $p(X \, \vert \, Z)$, и $p(Z)$. При этом $p(X \, \vert \, Z)$ называется правдоподобием выборки $X$ при условии $Z$, а $p(Z)$ является априорным распределением $Z$ (взятым на основании знаний о предметной области или полученным как апостериорное распределение $Z$ при условии какой-то ранее наблюдавшейся выборки $X_\mathrm{old}$).

Решение задачи, данной в байесовской постановке, описывается так:
* Апостериорное распределение находится по теореме Байеса:
$$p(Z \, \vert \, X) = \frac{p(X, Z)}{p(X)} = \frac{p(X, Z)}{\int p(X, Z) dZ},$$
* Точечная оценка $\hat{Z}$ получается как мода или как математическое ожидание полученного апостериорного распределения:
    - $\hat{Z} = \arg \max_Z p(Z \, \vert \, X)$,
    - $\hat{Z} = \mathbb{E}_{p(Z \, \vert \, X)}[Z]$.
    
Основная сложность, связанная с байесовским статистическим выводом, как правило, заключается во взятии интеграла из знаменателя правой части формулы для апостериорного распределения.

#### Способы решения задачи

Существуют следующие способы найти апостериорное распределение $p(Z \, \vert \, X)$:
* Если это возможно, явно взять интеграл из знаменателя в выражении для апостериорного распределения. Данный подход примением, например, тогда, когда $Z$ принимает дискретные значения и интеграл, на самом деле, является суммой, или тогда, когда $p(X \, \vert \, Z)$ и $p(Z)$ — пара сопряжённых (conjugate) распределений;
* [Вариационный байесовский вывод](__home_url__/notes/Вариационный байесовский вывод и mean-field approximation). Для произвольного зафиксированного $X$ проводится приближённое вычисление апостериорного распределения $p(Z \, \vert \, X)$ методом покоординатного подъёма, где на всех итерациях приближения ищутся в классе распределений, факторизующихся на непересекающиеся группы переменных;
* [Методы Монте-Карло по схеме марковской цепи](__home_url__/notes/Методы Монте-Карло над марковскими цепями), MCMC. Это численные методы взятия интеграла из знаменателя в выражении для апостериорного распределения, позволяющие сэмплировать значения $Z$ из распределения $p(Z \, \vert \, X)$.

## Вариационный байесовский вывод и mean-field approximation

#### Описание и обоснование

Пусть рассматривается задача [байесовского статистического вывода](__home_url__/notes/Байесовский статистический вывод), где по кортежу наблюдаемых переменных $X$ требуется найти апостериорное распределение кортежа скрытых переменных $p(Z \, \vert \, X)$, имея явный вид вероятностной модели $p(X, Z) = p(X \, \vert \, Z) p(Z)$, где обе плотности из правой части даны аналитически. Более детальное описание задачи и, в частности, то, как интерпретируются $X$ и $Z$, можно найти в заметке про байесовский статистический вывод.

Вариационный байесовский вывод применяется в ситуации, когда апостериорное распределение $p(Z \, \vert \, X) = \frac{p(X, Z)}{p(X)} = \frac{p(X, Z)}{\int p(X, Z) dZ}$ не получается найти явно из-за того, что не вычисляется аналитически $p(X) = \int p(X, Z) dZ$. Вместо искомого распределения $p(Z \, \vert \, X)$ ищется его некоторое приближение $q(Z)$, тоже являющееся вероятностным распределением кортежа $Z$ (причём для разных $X$, вообще говоря, будут найдены разные $q(Z)$). Итак, цель заключается в том, чтобы найти $q(Z) \approx p(Z \, \vert \, X)$, где близость понимается в смысле минимизации дивергенции Кульбака-Лейблера между распределениями $q(Z)$ и $p(Z \, \vert \, X)$:
$$D_{KL}(q(Z) \, \Vert \, p(Z \, \vert \, X)) \to \min_{q},$$
причём порядок аргументов дивергенции Кульбака-Лейблера такой, какой есть, потому что именно для такой дивергенции Кульбака-Лейблера удаётся найти решение сформулированной оптимизационной задачи.

Для решения поставленной задачи сведём её к эквивалентной ей оптимизационной задаче.

Каким бы ни было распределение $q(Z)$, всегда можно переписать логарифм правдоподобия наблюдаемой части выборки следующим образом:
$$\log p(X) = \log p(X) \int q(Z) dZ = \int q(Z) \log p(X) dZ = \int q(Z) \log \frac{p(X, Z)}{p(Z \, \vert \, X)} dZ = \int q(Z) \log \frac{p(X, Z)q(Z)}{p(Z \, \vert \, X)q(Z)} dZ \\ = \int q(Z) \log \frac{p(X, Z)}{q(Z)} dZ + \int q(Z) \log \frac{q(Z)}{p(Z \, \vert \, X)} dZ.$$

Обозначим первое слагаемое последнего выражения за $\mathcal{L}(q)$ и заметим, что второе слагаемое является той самой дивергенцией Кульбака-Лейблера $D_{KL}(q(Z) \, \Vert \, p(Z \, \vert \, X))$, которую необходимо минимизировать. Значит,
$$\log p(X) = \mathcal{L}(q) + D_{KL}(q(Z) \, \Vert \, p(Z \, \vert \, X)).$$

Благодаря такому равенству можно взглянуть на поиск приближения $q(Z)$ для неизвестного распределения $p(Z \, \vert \, X)$, которое, собственно говоря, неизвестно из-за невозможности вычислить $p(X)$, с двух точек зрения:
* Исходная оптимизационная задача. Минимизацией $D_{KL}(q(Z) \, \Vert \, p(Z \, \vert \, X))$ по $q$ можно получить приближение для $p(Z \, \vert \, X)$, потому что дивергенция Кульбака-Лейблера неотрицательна и, более того, обнуляется тогда и только тогда, когда два распределения идентичны.
* Эквивалентная ей оптимизационная задача. Максимизацией $\mathcal{L}(q)$ по $q$ можно получить приближение (а именно оценку снизу) для $p(X)$, потому что $\mathcal{L}(q)$ не может быть больше $\log p(X)$ в силу неотрицательности дивергенции Кульбака-Лейблера;

Раз две выписанных оптимизационных задачи эквивалентны (имеют одинаковое решение), остановимся на второй:
$$\mathcal{L}(q) \to \max_{q}.$$
Она выбрана, потому что удобнее для применения вариационного подхода. Функционал $\mathcal{L}(q)$ называется нижней вариационной оценкой (evidence lower bound, ELBO), и он имеет также следующую интерпретацию:
$$\mathcal{L}(q) = \int q(Z) \log(p(X \, \vert \, Z)p(Z)) - \log q(Z) \, dZ = \int q(Z) \log p(X \, \vert \, Z) - \log \frac{q(Z)}{p(Z)} \, dZ = \mathbb{E}_{Z \sim q(Z)}[\log p(X \, \vert \, Z)] - D_{KL}(q(Z)\, \Vert \, p(Z)),$$
что с точки зрения общей теории машинного обучения выглядит как сумма функционала качества и регуляризатора, штрафующего отклонения оценки $q(Z)$ апостериорного распределения $p(Z \, \vert \, X)$ от априорного распределения $p(Z)$.

Заметим, что до сих пор не вводилось никаких дополнительных предположений по сравнению с теми, что есть в общей постановке задачи байесовского статистического вывода. Вовлечение вариационного подхода опирается на то, что на распределение $q$ накладывается ограничение факторизуемости. Оно означает, что существует такая разбивка элементов кортежа $Z$ на $m$ непересекающихся множеств $s_i$, что
$$q(Z) = \prod_{i=1}^m q_i(Z_{s_i}),$$
где $Z_{s_i}$ — кортеж, получающийся из кортежа $Z$ оставлением тех и только тех элементов, которые попадают в множество $s_i$, а $q_i(Z_i)$ — произвольное распределение кортежа $Z_{s_i}$. Стало быть, теперь ищется приближение распределения $p(Z \, \vert \, X)$ в классе факторизуемых вероятностных распределений.

Будем решать оптимизационную задачу итеративно методом покоординатного подъёма. Инициализируем как-то все $q_i$, а затем на каждом шаге будем по очереди обновлять каждое из них в соответствии со следующим правилом:
$$q_i(Z_{s_i}) := \frac{\exp(\int \log p(X, Z) \prod_{j \ne i} q_j(Z_{s_j}) dZ_{s_{-i}})}{\int \exp(\int \log p(X, Z) \prod_{j \ne i} q_j(Z_{s_j}) dZ_{s_{-i}}) dZ_{s_i}},$$
где $Z_{s_{-i}}$ обозначает кортеж, получающийся из $Z$ выкидыванием элементов, принадлежащих множеству $s_i$. Выписанная формула обновления $q_i(Z_{s_i})$ называется основной формулой вариационного байесовского вывода, а её получение здесь ради краткости опущено. Вывод же назван вариационным, потому что при оптимизации используется взятие вариации оптимизируемого функционала $\mathcal{L}(q)$ вдоль направления, соответствующего $i$-му множеству скрытых переменных. На самом деле, существуют и другие подходы к вариационному байесовскому выводу, поэтому для большей конкретики описанный здесь подход называют mean-field approximation. Название пришло из физики магнитных полей, где есть раздел, именуемый теория среднего поля.

Может показаться, что интеграл из знаменателя правой части основной формулы вариационного байесовского вывода ничуть не проще исходного интеграла из формулы для апостериорного распределения. Однако взять этот интеграл иногда удаётся. В следующем разделе будет описано, что и при каких условиях возможно.

#### Применимость в зависимости от имеющихся свойств

Чтобы описывать различные ситуации, введём ряд определений.

Начнём с понятия условной сопряжённости, являющегося ослаблением понятия сопряжённости. Пусть распределение кортежа $Z$ факторизуется в разбивке элементов кортежа на $m$ непересекающихся множеств. Примем за $\{Z_{s_i}\}_{i=1}^m$ множество из $m$ кортежей, $i$-й из которых получается из $Z$ оставлением только соответствующих $i$-му множеству элементов, а через $Z_{s_{-i}}$ обозначим кортеж, получающийся из $Z$ выкидыванием всех элементов, соответствующих $i$-му множеству. Будем говорить, что присутствует условная сопряжённость, если одновременно выполнены $m$ условий, $i$-е из которых имеет вид:
* Для любого зафиксированного значения кортежа $Z_{s_{-i}}$ (то есть для любых зафиксированных значений всех кортежей $Z_{s_j}$, где берутся все $j$ от 1 до $m$ кроме $i$), распределения $p(X \, \vert \, Z_{s_i}, Z_{s_{-i}})$ и $p(Z_{s_i} \, \vert \, Z_{s_{-i}})$ сопряжены друг к другу.

Теперь предположим, что кортеж $Z$ может быть разделён на два кортежа, обозначаемые далее как $T$ и $\Theta$. Будем считать, что $p(Z) = p(T, \Theta) = p(T)p(\Theta)$, то есть имеется факторизация при разбивке элементов $Z$ на те, которые относятся к $T$, и на те, которые относятся к $\Theta$. Для интерпретируемости далее можно считать, что кортеж $T$ составлен из строк матрицы скрытых признаков объектов, а $\Theta$ — кортеж неизвестных параметров вероятностной модели.

Определим $T$-сопряжённость как ослабление условной сопряжённости до того, что при факторизации $Z$ в разбивке на $T$ и $\Theta$ условие выполнено только для $Z_{s_i} = T$, то есть как то, что пару сопряжённых распределений образуют $p(X \, \vert T, \Theta)$ и $p(T \, \vert \, \Theta)$. Аналогично, определим $\Theta$-сопряжённость как то, что сопряжены друг к другу распределения $p(X \, \vert \Theta, T)$ и $p(\Theta \, \vert \, T)$.

Далее, ослабим понятия $T$-сопряжённости и $\Theta$-сопряжённости. Назовём условной $T$-сопряжённостью свойство, при котором распределение кортежа $T$ само как-то факторизуется и для факторизации распределения всего $Z$ на $\Theta$ и на то, на что разбивается $T$, выполнены все условия из определения условной сопряжённости кроме того условия, где фиксируется всё, на что разбивается $T$ (то есть условия $\Theta$-сопряжённости). Аналогично, условной $\Theta$-сопряжённостью назовём свойство, при котором распределение кортежа $\Theta$ как-то факторизуется и для факторизации распределения всего $Z$ на $T$ и на то, на что разбивается $\Theta$, есть все условия из определения условной сопряжённости кроме того условия, где фиксируется всё, на что разбивается $\Theta$ (то есть условия $T$-сопряжённости).  

В зависимости от присутствия того или иного свойства применимы разные методы статистического вывода. При этом с установлением того, какое свойство присутствует, есть одна тонкость: факторизуемости может и не быть, но всегда можно предполагать, будто она есть. После такого предположения будет искаться лишь приближение в классе соответствующим образом факторизованных распределений, и поэтому чем меньше искусственно разбивается кортеж $Z$, тем выше точность вывода (с другой стороны, тем выше и вычислительные затраты).

Итак, соответствие между присутствующими свойствами и доступными методами вывода таково:
* сопряжённость (полная, т.е. $Z$-сопряжённость): полный байесовский вывод, без каких бы то ни было приближений оценивающий $p(Z \, \vert X) = p(T, \Theta \, \vert \, X)$;
* условная сопряжённость в разбивке $Z$ на $T$ и $\Theta$: вариационный байесовский вывод, оценивающий $q(Z) = q(T)q(\Theta) \approx p(T, \Theta \, \vert \, X)$;
* $T$-сопряжённость: точный EM-алгоритм, являющийся частным случаем вариационного байесовского вывода, где дополнительно предполагается, что $q(\Theta) = \delta(\Theta - \Theta_0)$ и в этой записи $\delta$ — дельта-функция Дирака, а $\Theta_0$ — константный кортеж, который требуется найти; точный EM-алгоритм оценивает $q(Z) = q(T)\delta(\Theta - \Theta_0) \approx p(T, \Theta \, \vert \, X)$;
* $\Theta$-сопряжённость: точный ME-алгоритм, являющийся частным случаем вариационного байесовского вывода, где дополнительно предполагается, что $q(T) = \delta(T - T_0)$, а $T_0$ — константный кортеж, который требуется найти; точный ME-алгоритм оценивает $q(Z) = \delta(T - T_0)q(\Theta) \approx p(T, \Theta \, \vert \, X)$;
* условная $T$-сопряжённость: вариационный EM-алгоритм, оценивающий $q(Z) = \left(\prod_{i=1}^m q_i(T_i)\right) \delta(\Theta - \Theta_0) \approx p(T, \Theta \, \vert \, X)$;
* условная $\Theta$-сопряжённость: вариационный ME-алгоритм, оценивающий $q(Z) = \delta(T - T_0) \left(\prod_{i=1}^m q_i(\Theta_i)\right) \approx p(T, \Theta \, \vert \, X)$;
* никаких свойств нет: жёсткий EM-алгоритм, оценивающий $q(Z) = \delta(T - T_0) \delta(\Theta - \Theta_0) \approx p(T, \Theta \, \vert \, X)$, то есть дающий только точечные оценки.

Получившийся список можно прокомментировать так. Чем более слабое свойство имеется, тем более "простым" распределением приходится полагать то распределение, которое ищется в задаче статистического вывода. "Простота" здесь понимается как факторизуемость и атомарность.

Напоследок разберём какой-нибудь пример, показывающий, почему указанных свойств достаточно для соответствующих методов. Скажем, остановимся на достаточности $T$-сопряжённости для точного EM-алгоритма. В точном EM-алгоритме сложное место находится на E-шаге, где ищется распределение $$q^{(n+1)}(Z) = p(Z \, \vert \, X, \Theta^{(n)}) = \frac{p(X, Z \, \vert \,  \Theta^{(n)})}{\int p(X, Z \, \vert \, \Theta^{(n)}) dZ} = \frac{p(X \, \vert \, Z, \Theta^{(n)})p(Z \, \vert \, \Theta^{(n)})}{\int p(X \, \vert \, Z, \Theta^{(n)})p(Z \, \vert \, \Theta^{(n)}) dZ} = \frac{p(X \, \vert \, T, \Theta^{(n)})p(T \, \vert \, \Theta^{(n)})\delta(\Theta - \Theta^{(n)})}{\int p(X \, \vert \, T, \Theta^{(n)})p(T \, \vert \, \Theta^{(n)})\delta(\Theta - \Theta^{(n)}) dT d\Theta} = \frac{p(X \, \vert \, T, \Theta^{(n)})p(T \, \vert \, \Theta^{(n)})}{\int p(X \, \vert \, T, \Theta^{(n)})p(T \, \vert \, \Theta^{(n)}) dT} = \frac{p(X, T \, \vert \, \Theta^{(n)})}{\int p(X, T \, \vert \, \Theta^{(n)}) dT},$$
и тут интеграл в полученном в конце выражении берётся явно в силу $T$-сопряжённости.

## Методы Монте-Карло над марковскими цепями

#### Введение

Пусть с точностью до константы известна в аналитическом виде плотность $p(x)$ распределения случайного вектора $x$. Требуется научиться сэмплировать выборки из этого распределения, то есть получать объекты $x_i \sim p(x)$, или хотя бы научиться приближённо интегрировать по этому распределению, то есть оценивать интегралы $\int f(x)p(x) dx$ для любой достаточно регулярной функции $f$. Решение второй задачи получается из решения первой задачи, ведь выписанный интеграл аппроксимируется средним $f(x)$ по просэмплированной выборке достаточного размера.

Умение решать так поставленную задачу пригождается, в частности, при проведении [байесовского статистического вывода](__home_url__/notes/Байесовский статистический вывод). Апостериорное распределение $p(Z \, \vert \, X)$ может быть получено в аналитическом виде с точностью до нормировочной константы, и затем при помощи сэмплирования из него можно будет по-байесовски ансамблировать предсказания, выдаваемые моделями с различными значениями неизвестных параметров.

В случае, когда $p(x)$ является распределением над низкоразмерным вектором, существуют алгоритмы, позволяющие сэмплировать независимые объекты в точности из распределения $p(x)$. К таким алгоритмам относятся сэмплирование с отклонением (rejection sampling) и сэмплирование с перевзвешиванием (importance sampling). Однако оба упомянутых алгоритма подвержены проклятию размерности, так что для данных высокой размерности нужны другие методы. Далее речь пойдёт о методах, которые, с одной стороны, более устойчивы к проклятью размерности, но, с другой стороны, сэмплируют так, что результаты соседних сэмплирований не являются независимыми (этот недостаток можно смягчить, оставляя лишь результаты каждого $m$-го испытания, где $m$ достаточно велико).

#### Схема Метрополиса-Хастингса

Введём следующие сущности:
* $\overline{p}(x)$ — функция, с точностью до константы равная функции плотности $p(x)$ интересующего вероятностного распределения;
* $q(x \, \vert \, y)$, где $y \in \mathrm{Domain}(x)$ — некоторая плотность вероятностного распределения, называемого предложным распределением и обладающего такими свойствами:
    - $\forall y$ есть возможность сэмплировать из распределения $q(x \, \vert \, y)$,
    - $\forall x, y$, таких что $p(x) > 0$ и $p(y) > 0$, также и $q(x \, \vert \, y) > 0$.
    
В качестве предложного распределения может быть использовано, например, $q(x \, \vert \, y) = \mathcal{N}_{y, \sigma^2I}(x)$. Такое распределение симметрично, $q(x \, \vert \, y) = q(y \, \vert \, x)$, и поэтому с ним может работать даже схема Метрополиса, то есть предшественница схемы Метрополиса-Хастингса, которую Хастингс обобщил на случай несимметричных предложных распределений.

Идея заключается в том, что хочется построить эргодическую марковскую цепь, инвариантным распределением которой будет $p(x)$. Если такая цепь будет построена, начиная с какого-то момента сэмплировать из $p(x)$ можно будет за счёт порождения следующего элемента этой цепи.

Сначала опишем конструкцию схемы Метрополиса-Хастингса, а потом докажем, что она работает так, как заявлено.

Итак, пусть текущий элемент цепи равен $x_n$. Следующий за ним элемент цепи $x_{n+1}$ порождается по представленному ниже алгоритму:
* Сэмплируется $x \sim q(x \, \vert x_n)$;
* Вычисляется отношение Хастингса $H(x, x_n) = \min\left(1, \frac{\overline{p}(x)q(x_n \, \vert \, x)}{\overline{p}(x_n)q(x \, \vert \, x_n)}\right) = \min\left(1, \frac{p(x)q(x_n \, \vert \, x)}{p(x_n)q(x \, \vert \, x_n)}\right)$;
* С вероятностью, равной $H(x, x_n)$, $x_{n+1} := x$, а иначе $x_{n+1} := x_n$.

Начало цепи $x_1$ может быть выбрано произвольно, но от его выбора может зависеть, как долго придётся ждать, пока марковская цепь сойдётся к инвариантному распределению.

Теперь перейдём к доказательству работоспособности конструкции.

Существует теорема, гласящая, что любая однородная марковская цепь с переходной вероятностью $p(x \, \vert \, y)$, такой что $p(x \, \vert \, y) > 0$ для всех $x$ и $y$, является эргодической, то есть имеет ровно одно инвариантное распределение, к которому непременно сходится. В случае со схемой Метрополиса-Хастингса при $x_{n+1} \ne x_n$ переходная вероятность равна $q(x_{n+1} \, \vert \, x_{n})H(x_{n+1}, x_n) > 0$, и при $x_{n+1} = x_n$ она тоже больше нуля, так что эта теорема применима.

Осталось показать, что инвариантное распределение имеет плотность $p(x)$. Чтобы сделать это, прибегнем к тривиальному факту, гласящему, что для марковской цепи с переходной вероятностью $r(x \, \vert \, y)$ из того, что $r(y \, \vert \, x)p(x) = r(x \, \vert y)p(y)$, следует, что $p(x)$ является инвариантным распределением. Предпосылку сделанного утверждения называют уравнением детального баланса, это уравнение говорит, что, грубо говоря, сколько вероятности переходит из $x$ в $y$ при переходе к следующему элементу марковской цепи, столько же переходит и из $y$ в $x$. Но это и означает, что $p(x)$ задаёт инвариантное распределение. Итак, убедимся, что для схемы Метрополиса-Хастингса уравнение детального баланса для $p(x)$ соблюдается (строго говоря, выкладка, приведённая ниже, применима только к случаю $x_{n+1} \ne x_n$, но случай $x_{n+1} = x_n$ тривиален):
$$r(x_{n+1} \, \vert \, x_n)p(x_n) = q(x_{n+1} \, \vert \, x_{n})H(x_{n+1}, x_n)p(x_n) = p(x_n)q(x_{n+1} \, \vert \, x_{n}) \min\left(1, \frac{p(x_{n+1})q(x_n \, \vert \, x_{n+1})}{p(x_n)q(x_{n+1} \, \vert \, x_n)}\right) = \min\left(p(x_n)q(x_{n+1} \, \vert \, x_{n}), p(x_{n+1})q(x_n \, \vert \, x_{n+1})\right) = p(x_{n+1})q(x_n \, \vert \, x_{n+1})\min\left(1, \frac{p(x_n)q(x_{n+1} \, \vert \, x_n)}{p(x_{n+1})q(x_n \, \vert \, x_{n+1})}\right) = r(x_n \, \vert \, x_{n+1})p(x_{n+1}).$$
Тем самым выше было показано, что для построенной марковской цепи среди инвариантных распределений есть $p(x)$, но поскольку инвариантное распределение ровно одно, только $p(x)$ и может быть им.

Неформально говоря, $H(x, x_n)$ для того и вводится, чтобы добиться соблюдения уравнения детального баланса. Также можно интерпретировать $H(x, x_n)$ как то, что побуждает марковскую цепь реже уходить из какой-либо точки, если в новой точке $p(x)$ ниже.

#### Схема Гиббса

Схема Гиббса является частным случаем схемы Метрополиса-Хастингса, имеющим то свойство, что всегда $H(x, x_n) = 1$. Достигается это за счёт того, что теперь шаг от $x_n$ к $x_{n+1}$ делается поэтапно, а именно для каждой координаты по порядку начиная с первой сначала проводится её обновление, затем происходит обновление следующей координаты, и так далее вплоть до последней координаты включительно.

Если обозначить за $x_n^i$ $i$-ю компоненту вектора $x_n$, то тогда формула для её обновления в схеме Гиббса имеет вид:
$$x_{n+1}^k \sim \overline{p}(x^k \, \vert \, x^1 = x_{n+1}^1, \, \ldots, \, x^{k-1} = x_{n+1}^{k-1}, \, x^{k+1} = x_n^{k+1}, \, \ldots, \, x^d = x_n^d),$$
где $d$ — размерность векторов $x_n$ и $x_{n+1}$. Знак $\sim$ в формуле обозначает, что $x_{n+1}^k$ сэмплируется из распределения, стоящего справа от этого знака. Сэмплировать из такого распределения можно, потому что оно является распределением одномерной случайной величины, известным с точностью до константы. Обоснуем это утверждение. Во-первых, с точностью до константы это распределение известно, потому что оно является ограничением известного с точностью до константы распределения $\overline{p}$. Во-вторых, для известных с точностью до константы распределений одномерных случайных величин можно применять техники, такие как сэмплирование с перевзвешиванием, потому что проклятие размерности нестрашно.

При предложном распределении $q(x^k \, \vert \, y) = \overline{p}(x^k \, \vert \, y^1, ..., y^{k-1}, y^{k+1}, ..., y^d)$ выкладка показывает, что $H(x, x_n) = 1$. Если выписанное предложное распределение развернуть от распределения одной $k$-й компоненты до распределения всего $d$-мерного вектора, то и получится, что схема Гиббса является схемой Метрополиса-Хастингса, в которой выбрано развёрнутое предложное распределение.

## Масштабирование марковских цепей Монте-Карло и схема Ланжевена

#### Введение

Пусть решается задача [байесовского статистического вывода](__home_url__/notes/Байесовский статистический вывод) и требуется оценить апостериорное распределение
$$p(Z \, \vert \, X) = \frac{p(X \, \vert \, Z)p(Z)}{\int p(X \, \vert \, Z)p(Z) dZ}.$$
Числитель по предположению задачи байесовского статистического вывода известен в аналитическом виде, а вот знаменатель является константой, вычислить которую часто бывает невозможно из-за того, что интеграл не берётся. Приходится прибегать к численным методам, одним из которых являются [марковские цепи Монте-Карло](__home_url__/notes/Методы Монте-Карло над марковскими цепями). Они позволяют сэмплировать $Z$ из $p(Z \, \vert \, X)$.

Недостатком классических схем для марковских цепей Монте-Карло, то есть схемы Метрополиса-Хастингса и схемы Гиббса, является то, что они работают слишком медленно при большом количестве объектов в выборке (при большом количестве строк в матрице $X$).

Так, в схеме Метрополиса-Хастингса отношение Хастингса для некоторого предложного распределения $q(Z \, \vert \, Z_n)$ и просэмплированного из него кандидата $Z$ имеет вид:
$$H(Z, Z_n) = \min\left(1, \frac{p(X \, \vert \, Z)p(Z) \, q(Z_n \, \vert \, Z)}{p(X \, \vert \, Z_n)p(Z_n) \, q(Z \, \vert \, Z_n)}\right).$$
Тут плохо масштабируется вычисление $p(X \, \vert \, Z)$ и $p(X \, \vert \, Z_n)$. Скажем, если все объекты независимы, придётся перемножить столько слагаемых, сколько строк в $X$. Поскольку подобное перемножение делается при сэмплировании каждого нового значения $Z_{n+1}$, это существенно замедляет схему.

В схеме же Гиббса проблема с масштабированием возникает из-за того, что каждая компонента вектора $Z_{n+1}$ сэмплируется из распределения, для вычисления которого с точностью до константы тоже требуется обработать всю матрицу $X$.

Существуют модификации марковских цепей Монте-Карло с улучшенной масштабируемостью на большие данные, отталкивающиеся от следующей идеи. Можно заметить, что схемы Метрополиса-Хастингса и Гиббса пытаются найти $Z$, для которых высока $p(Z \, \vert \, X)$, используя для этого только значения $p(Z \, \vert \, X)$. Однако что, если также использовать градиент $p(Z \, \vert \, X)$ или, точнее, градиент $\log p(Z \, \vert \, X)$, в случае независимых объектов распадающийся на сумму градиентов? Если так поступить, вместо градиента можно будет использовать стохастический градиент, оцениваемый только по текущему батчу, что и снимет проблему большого размера $X$.

#### Схема Ланжевена

Схема Ланжевена опирается на концепцию из физики, называемую динамика Ланжевена (Langevin dynamics). На базе такой динамики может быть постоена схема Метрополиса-Хастингса с определённым предложным распределением. Сама по себе эта схема не является масштабируемой, но её можно переделать в масштабируемую по аналогии с тем, как из классического градиентного подъёма получается стохастический градиентный подъём.

Итак, сначала опишем схему Метрополиса-Хастингса на базе динамики Ланжевена. Чтобы её описать, достаточно задать предложное распределение. Зададим его так:
$$q(Z \, \vert \, Z_n) = \mathcal{N}_{\mu, \varepsilon}(Z),$$
где $\mu = Z_n + \frac{\varepsilon}{2}\nabla_Z\left(\log(p(X \, \vert \, Z_n)p(Z_n))\right)$, $\varepsilon \in \mathbb{R}$, а $\nabla_Z$ обозначает градиент по $Z$ (в данном случае, вычисленный в точке $Z_n$). Подчеркнём, что $\varepsilon$ фигурирует и как дисперсия предложного распределения, и как множитель перед градиентом в выражении для среднего. Также отметим, что выписанное предложное распределение не является симметричным, и поэтому с ним работает именно схема Метрополиса-Хастингса, а не схема Метрополиса.

Теперь представим, что почему-то хочется проигнорировать коррекцию Метрополиса-Хастингса и, не вычисляя отношение Хастингса, всегда принимать просэмплированные из предложного распределения объекты. Тогда получится следующая схема (марковская цепь):
$$Z_{n+1} = Z_n + \frac{\varepsilon}{2}\nabla_Z\left(\log(p(X \, \vert \, Z_n)p(Z_n))\right) + \eta_{n},$$
где $\eta_{n} \sim \mathcal{N}_{0, \varepsilon}(\eta_{n})$. Если бы не последнее слагаемое, это была бы формула поиска моды распределения $p(Z \, \vert \, X)$ методом покоординатного подъёма, ведь
$$\nabla_Z\left(\log p(Z_n \, \vert \, X)\right) = \nabla_Z\left(\log(p(X \, \vert \, Z_n)p(Z_n)) - \log(\int p(X \, \vert \, Z)p(Z) dZ)\right) = \nabla_Z\left(\log(p(X \, \vert \, Z_n)p(Z_n))\right).$$
Однако благодаря добавлению шума, вносимого слагаемым $\eta_{n}$, схема описывает некоторое блуждание, как и в случае схемы Метрополиса-Хастингса склонное задерживаться в регионах с высокой плотностью $p(Z \, \vert \, X)$.

Через стохастические дифференциальные уравнения можно доказать, что если константу $\varepsilon$ заменить на элемент последовательности $\varepsilon_n$, такой что $\varepsilon_n \rightarrow 0$, то тогда описанная марковская цепь тоже будет порождать $Z_{n}$ из распределения $p(Z \, \vert \, X)$. Достигается это за счёт того, что множитель перед градиентом и дисперсия шума соотносятся друг с другом (а именно первое в два раза меньше второго).

Избавление от необходимости вычислять отношение Хастингса ускоряет вычисления — правда, ценой того, что из-за стремящихся к нулю $\varepsilon_n$ с какого-то момента соседние просэмплированные значения практически не будут отличаться друг от друга (т.е. будут сильно скоррелированы), из-за чего потребуется прореживание, становящееся всё более сильным.

Если объекты выборки независимы друг от друга, то можно получить второй существенный выигрыш. Разобъём выборку на батчи размера $b$, $n$-й из которых имеет вид $x_{n_1}$, ..., $x_{n_b}$. Тогда рассмотрим такую схему:
$$Z_{n+1} = Z_n + \frac{\varepsilon_n}{2}\left(\frac{N}{b}\sum_{j=1}^b \nabla_Z\log(p(x_{n_j} \, \vert \, Z_n)p(Z_n))\right) + \eta_{n},$$
где $N$ — количество строк в матрице $X$, а $\eta_{n} \sim \mathcal{N}_{0, \varepsilon_n}(\eta_{n})$. По построению новая схема на каждом шаге использует лишь батч ограниченного размера, а не всю выборку.

Чтобы новая схема сэмплировала $Z$ из распределения $p(Z \, \vert \, X)$, достаточно наложить такие ограничения на последовательность $\varepsilon_n$: $\sum_{n=1}^{+\infty} \varepsilon_n = +\infty$ и $\sum_{n=1}^{+\infty} \varepsilon_n^2 < +\infty$. В качестве примера подходящей последовательности подойдёт, например, $\varepsilon_n = \frac{1}{n^d}$, где $0.5 < d \le 1$. Доказательство того, что наложенных ограничений достаточно, опирается на то, что при вычислении $Z_{n+1}$ есть два источника случайности: выбор объектов в батч и добавление $\eta_{n}$. В силу наложенных ограничений дисперсия, вносимая первым из них, со временем станет пренебрежимо малой в сравнении с дисперсией, вносимой вторым из них. Грубо говоря, если оставить только второй источник случайности, то получится то же самое, что было в предыдущей схеме, то есть в схеме без разбивки на батчи. А у такой схемы, как уже упоминалось, работоспособность доказывается через стохастические дифференциальные уравнения. 

#### Альтернативы

Марковские цепи Монте-Карло можно строить и на динамике Гамильтона. Такая конструкция имеет то преимущество, что в случае мультимодального распределения $p(Z \, \vert \, X)$ переключение между модами, возле которых сэмплируются значения $Z$, происходит чаще.

## Скрытые марковские модели

#### Введение

Скрытая марковская модель с $m$ скрытыми состояниями и $n$ наблюдаемыми состояниями — это вероятностная графическая модель, описываемая совокупностью сущностей из следующего списка:
* Вероятностное распределение $\pi$ на множестве $\{1, ..., m\}$, представляющее собой распределение скрытого состояния в начальный момент времени;
* Матрица переходов между скрытыми состояниями $A$, её размер $m \times m$, а на пересечении $i$-й строки и $j$-го столбца стоит вероятность из скрытого состояния $i$ перейти в скрытое состояние $j$; как следствие сумма элементов любой строки матрицы $A$ равна 1;
* Матрица эмиссии наблюдаемых значений $B$, её размер $m \times n$, а на пересечении $i$-й строки и $j$-го столбца стоит условная вероятность того, что текущее наблюдаемое состояние равно $j$ при условии, что текущее скрытое состояние есть $i$; сумма элементов любой строки матрицы $B$ тоже равна 1.

Для переноса на случай, когда наблюдаемые значения непрерывны, можно считать, что вместо матрицы $B$ имеются $m$ вероятностных распределений.

В контексте скрытых марковских моделей существуют следующие задачи:
* Зная $\pi$, $A$ и $B$, вычислить вероятность данной последовательности наблюдаемых значений $(x_t)_{t=1}^T$, где $T$ — длина последовательности;
* Зная $\pi$, $A$ и $B$, по имеющейся последовательности наблюдаемых значений $(x_t)_{t=1}^T$ восстановить последовательность скрытых значений $(h_t)_{t=1}^T$;
* Имея множество последовательностей наблюдаемых значений $\left\{\left. (x_t)_{t=1}^{T_k} \; \right\vert \; k \in \{1, ..., N\}\right\}$, где $N$ — количество обучающих последовательностей, оценить $\pi$, $A$ и $B$, то есть обучить скрытую марковскую модель.

#### Задача вычисления вероятности наблюдаемой последовательности

Эта задача является задачей вероятностного (прямого) вывода, а не задачей статистического (обратного) вывода. На самом деле, подсчитать искомую вероятность можно просто в лоб по определению скрытой марковской модели:
$$p\!\left((x_t)_{t=1}^T\right) = \sum_{(h_t)_{t=1}^T \in \{1, ..., m\}^T} \left(\pi(h_1) B_{h_1x_1} \prod_{t=2}^T A_{h_{t-1}h_t} B_{h_tx_t}\right),$$
однако такой подход имеет экспоненциальную от длины наблюдаемой последовательности $T$ сложность, потому что суммирование проводится по $m^T$ возможным вариантам последовательностей скрытых состояний.

Для того чтобы задача была вычислительно подъёмной, используется алгоритм прохода вперёд (forward algorithm), восходящий к динамическому программированию.

Введём индексируемые позицией в последовательности $t$ и скрытым состоянием $i$ переменные
$$\alpha_{ti} = p\!\left((x_s)_{s=1}^t, h_t = i\right).$$
Иными словами, $\alpha_{ti}$ есть совместная вероятность появления подпоследовательности $(x_s)_{s=1}^t$ на первых $t$ наблюдаемых позициях и события, заключающегося в том, что в $t$-й позиции скрытое состояние равнялось $i$.

Теперь можно описать алгоритм прохода вперёд:
1. Для всех $i \in \{1, ..., m\}$ вычислить начальные значения:
$$\alpha_{1i} = \pi(i)B_{ix_1}.$$
2. Для всех $(t, i) \in \{2, ..., T\} \times \{1, ..., m\} $ в порядке неубывания $t$ провести индуктивный шаг вперёд:
$$\alpha_{ti} = \sum_{j=1}^m \alpha_{(t-1)j}A_{ji}B_{ix_t}.$$
3. Вычислить искомую вероятность:
$$p\!\left((x_t)_{t=1}^T\right) = \sum_{i=1}^m \alpha_{Ti}.$$

Поскольку в алгоритме прохода вперёд возникают произведения длины $T$, все сомножители которых являются вероятностями, при хоть сколько-нибудь больших $T$ возникает вычислительная проблема округления слишком малых чисел до нуля (underflow). Для решения этой проблемы вводится нормализация, при которой вместо $\alpha_{ti}$ подсчитываются отмасштабированные переменные $\hat{\alpha}_{ti}$, задаваемые следующими рекуррентными соотношениями:
$$\hat{\alpha}_{1i} = \frac{\pi(i)B_{ix_1}}{\sum_{j=1}^m \pi(j)B_{jx_1}},$$
$$\hat{\alpha}_{ti} = \frac{\sum_{j=1}^m \hat{\alpha}_{(t-1)j}A_{ji}B_{ix_t}}{\sum_{k=1}^m \left( \sum_{j=1}^m \hat{\alpha}_{(t-1)j}A_{jk}B_{kx_t}\right)}.$$
При фиксированном $t$ знаменатель из правой части для всех $\hat{\alpha}_{ti}$ один и тот же. Обозначим его за $c_t$. Тогда искомая вероятность равна $\prod_{t=1}^T c_t$, потому что
$$\prod_{t=1}^T c_t = \prod_{t=1}^T c_t \cdot \sum_{i=1}^m \hat{\alpha}_{Ti} = \prod_{t=1}^{T-1} c_t \cdot \sum_{i=1}^m \left(\sum_{j=1}^m \hat{\alpha}_{(T-1)j}A_{ji}B_{ix_T}\right) = ... = \sum_{i=1}^m \alpha_{Ti} = p\!\left((x_t)_{t=1}^T\right).$$

#### Задача восстановления последовательности скрытых состояний

Эта задача решается алгоритмом Витерби, который, как и алгоритм прохода вперёд, тоже восходит к динамическому программированию.

Введём индексируемые позицией в последовательности $t$ и скрытым состоянием $i$ переменные
$$\psi_{ti} = \max_{(h_s)_{s=1}^t \in \{1, ..., m\}^t}p\!\left((x_s)_{s=1}^t, h_t = i, (h_s)_{s=1}^t\right),$$
каждая из которых интерпретируется как максимальная совместная вероятность появления известной наблюдаемой подпоследовательности $(x_s)_{s=1}^t$, появления произвольной скрытой подпоследовательности $(h_s)_{s=1}^t$ и события, заключающегося в том, что в $t$-й позиции скрытое состояние равнялось $i$.

Также введём индексируемые позицией в последовательности $t$ и скрытым состоянием $i$ переменные
$$\phi_{ti} = \left(\arg\max_{(h_s)_{s=1}^t \in \{1, ..., m\}^t}p\!\left((x_s)_{s=1}^t, h_t = i, (h_s)_{s=1}^t\right)\right)_{t-1},$$
каждая из которых интерпретируется как скрытое состояние в позиции $t-1$ в подпоследовательности скрытых состояний, максимизирующей выражение из правой части определения $\psi_{ti}$.

Алгоритм Витерби устроен так:
1. Для всех $i \in \{1, ..., m\}$ вычислить начальные значения:
$$\psi_{1i} = \pi(i)B_{ix_1},$$
$$\phi_{1i} = 0,$$
где 0 взят как произвольный некорректный номер состояния (номера состояний начинаются с 1). Иными словами, ни одну $\phi_{1i}$ инициализировать не обязательно.
2. Для всех $(t, i) \in \{2, ..., T\} \times \{1, ..., m\} $ в порядке неубывания $t$ провести индуктивный шаг вперёд:
$$\psi_{ti} = B_{ix_t} \max_{j \in \{1, ..., m\}}A_{ji}\psi_{(t-1)j},$$
$$\phi_{ti} = \arg\max_{j \in \{1, ..., m\}}A_{ji}\psi_{(t-1)j}.$$
3. Восстановить последнее скрытое состояние:
$$h_T = \arg\max_k \psi_{Tk}.$$
4. Восстановить все остальные скрытые состояния индуктивными шагами назад:
$$h_t = \phi_{(t+1)h_{t+1}}.$$

Полученная алгоритмом Витерби последовательность $(h_t)_{t=1}^T$ является последовательностью с наибольшей совместной вероятностью $p\!\left((h_t)_{t=1}^T, (x_t)_{t=1}^T\right)$ и, как следствие, с наибольшей условной вероятностью $p\!\left(\left.(h_t)_{t=1}^T \right| (x_t)_{t=1}^T\right)$.

Алгоритм Витерби делается устойчивым к проблеме округления малых чисел до нуля путём перехода к логарифмам, из-за которого произведения заменяются на суммы. Поскольку в выражениях, фигурирующих в описании алгоритма, операция сложения не участвует, а есть только операция умножения, более сложные приёмы (такие как, например, в алгоритме прохода вперёд) не нужны.

#### Задача обучения скрытой марковской модели

Есть несколько подходов к обучению скрытых марковских моделей:
* Алгоритм Баума-Уэлша, являющийся частным случаем [EM-алгоритма](__home_url__/notes/Общий взгляд на EM-алгоритм), применяемого во многих задачах обучения вероятностных графических моделей;
* Градиентный спуск в пространстве троек $(\pi, A, B)$, после каждого шага которого строки матриц нормируются, чтобы их сумма, как это и требуется, равнялась 1.

Помимо батчевых версий указанных алгоритмов обучения, сразу обучающихся на всём множестве имеющихся последовательностей без возможности дальнейшего дообучения, существуют и онлайновые версии, способные дообучаться на одной или более последовательностях.

Что касается того, какой алгоритм обучения выбрать, оба алгоритма являются численными и оба не имеют гарантий того, что при оптимизации не произойдёт застревание около точки локального экстремума. По умолчанию алгоритм Баума-Уэлша предпочтительнее, однако встраивание в него регуляризации, необходимой для борьбы с присущим этому алгоритму переобучением, требует дополнительных математических изысканий, тогда как в случае с градиентным спуском достаточно просто добавить штрафное слагаемое к минимизируемой функции потерь.

Для алгоритма Баума-Уэлша требуется в том числе делать проход назад при условии как-то зафиксированных параметров $\pi$, $A$ и $B$. По аналогии с алгоритмом прохода вперёд введём следующие индексируемые позицией в последовательности $t$ и скрытым состоянием $i$ переменные:
$$\beta_{ti} = p\!\left(\left. (x_s)_{s=t+1}^T \right| h_t = i \right).$$
Вычисляются эти переменные индуктивными шагами назад от известного конца:
$$\beta_{Ti} = 1,$$
$$\beta_{ti} = \sum_{j=1}^m A_{ij}B_{jx_{t+1}}\beta_{(t+1)j}.$$

Теперь можно описать, как устроен алгоритм Баума-Уэлша для обучения на множестве, состоящем ровно из одной последовательности.

Сначала оценки параметров $\pi$, $A$ и $B$ инициализируются случайно или на базе априорных знаний, а затем поочерёдно до сходимости проводятся следующие итерации EM-алгоритма:
* E-шаг:
    - при текущих оценках $\pi$, $A$ и $B$ вычислить все $\alpha_{ti}$ алгоритмом прохода вперёд и все $\beta_{ti}$ алгоритмом прохода назад;
    - вычислить все условные вероятности того, что в позиции $t$ скрытое состояние равнялось $i$ при условии пронаблюдавшейся последовательности $(x_t)_{t=1}^T$ и при текущих оценках $\pi$, $A$ и $B$:
$$\gamma_{ti} = p\!\left(h_t = i \, \left| \, (x_t)_{t=1}^T\right.\right) = \frac{p\!\left(h_t = i, (x_t)_{t=1}^T\right)}{p\!\left((x_t)_{t=1}^T\right)} = \frac{\alpha_{ti}\beta_{ti}}{\sum_{j=1}^m \alpha_{tj}\beta_{tj}}.$$
    - вычислить все условные вероятности перехода в позиции с $t$-й на $(t+1)$-ю из состояния $i$ в состояние $j$ при условии пронаблюдавшейся последовательности $(x_t)_{t=1}^T$ и при текущих оценках $\pi$, $A$ и $B$:
$$\xi_{tij} = p\!\left(h_t = i, h_{t+1} = j \, \left| \, (x_t)_{t=1}^T\right.\right) = \frac{p\!\left(h_t = i, h_{t+1} = j, (x_t)_{t=1}^T\right)}{p\!\left((x_t)_{t=1}^T\right)} = \frac{\alpha_{ti}A_{ij}\beta_{(t+1)j}B_{jx_{t+1}}}{\sum_{k=1}^m\sum_{l=1}^m \alpha_{tk}A_{kl}\beta_{(t+1)l}B_{lx_{t+1}}}.$$
* M-шаг:
    - Обновить $\pi$ так:
$$\pi(i) := \gamma_{1i},$$
потому что справа стоит рассчитанная вероятность начать из состояния $i$;
    - Обновить матрицу $A$ по следующей формуле обновления её элементов:
$$A_{ij} := \frac{\sum_{t=1}^{T-1} \xi_{tij}}{\sum_{t=1}^{T-1} \gamma_{ti}},$$
потому что справа стоит отношение рассчитанного числа переходов из $i$ в $j$ к рассчитанному числу всех переходов из скрытого состояния $i$ (именно потому что состояние $i$ должно быть покинуто, сумма берётся лишь до $T-1$);
    - Обновить матрицу $B$ по следующей формуле обновления её элементов:
$$B_{io} := \frac{\sum_{t=1}^T[x_t = o]\,\gamma_{ti}}{\sum_{t=1}^T\gamma_{ti}},$$
где квадратные скобки обозначают нотацию Айверсона, а справа стоит отношение рассчитанного числа нахождений в скрытом состоянии $i$, таких что было выпущено наблюдаемое значение $o$, к рассчитанному числу всех нахождений в скрытом состоянии $i$.

Если же есть $N$ последовательностей в обучающей выборке, то:
* на E-шаге для всех $k$ от 1 до $N$ для $k$-й последовательности считаются свои значения $\alpha_{ti}^{(k)}$, $\beta_{ti}^{(k)}$, $\gamma_{ti}^{(k)}$ и $\xi_{tij}^{(k)}$, а также алгоритмом прохода вперёд вычисляется вероятность этой $k$-й последовательности, далее обозначаемая как $P_k$;
* на M-шаге вклад каждой последовательности учитывается с весом, обратно пропорциональным вероятности этой последовательности; например, формула обновления матрицы переходов между скрытыми состояниями будет выглядеть так:
$$A_{ij} := \frac{\sum_{k=1}^N \left(\frac{1}{P_k} \sum_{t=1}^{T_k-1} \xi_{tij}^{(k)}\right)}{\sum_{k=1}^N \left(\frac{1}{P_k}\sum_{t=1}^{T_k-1} \gamma_{ti}^{(k)}\right)}.$$
Более сильная подгонка под те последовательности, которые кажутся маловероятными относительно текущих оценок $\pi$, $A$ и $B$, позволяет обучать параметры так, чтобы все последовательности обучающей выборки выглядели вероятными.

## Линейные динамические системы

#### Введение

Линейная динамическая система (ЛДС) — это вероятностная графическая модель, задаваемая сущностями из следующего списка:
* Последовательность $(h_t)_{t=1}^{\infty}$ скрытых переменных, являющихся вещественными векторами, $h_t \in \mathbb{R}^m$;
* Последовательность $(x_t)_{t=1}^{\infty}$ наблюдаемых переменных, являющихся вещественными векторами, $x_t \in \mathbb{R}^n$;
* Вероятностное распределение $p(h_t \, \vert \, h_{t-1})$, описывающее переходы между скрытыми состояниями, причём предполагается, что это распределение является гауссовским и имеет вид $p(h_t \, \vert \, h_{t-1}) = \mathcal{N}_{Ah_{t-1}, \, \Gamma}(h_t)$, где $A$ и $\Gamma$ — матрицы размера $m \times m$;
* Вероятностное распределене $p(x_t \, \vert \, h_t)$, задающее модель шума, из-за которого наблюдаемые значения отличаются от скрытых значений, причём предполагается, что это распределение тоже является гауссовским и имеет вид $p(x_t \, \vert \, h_t) = \mathcal{N}_{Ch_t, \, \Sigma}(x_t)$, где $C$ — матрица размера $n \times m$, а $\Sigma$ — матрица размера $n \times n$;
* Начальное распределение скрытого состояния $p(h_1) = \mathcal{N}_{\lambda_0, \, W_0}(h_1)$, где $\lambda_0$ — вектор-столбец размера $m \times 1$, а $W_0$ — начальная ковариационная матрица размера $m \times m$.

С учётом всего вышесказанного вероятностная модель ЛДС принимает вид:
$$p\left((h_t)_{t=1}^T, (x_t)_{t=1}^T \, \vert \, A, \Gamma, C, \Sigma, \lambda_0, W_0 \right) = \mathcal{N}_{\lambda_0, \, W_0}(h_1) \mathcal{N}_{Ch_1, \, \Sigma}(x_1) \prod_{t=2}^T \left(\mathcal{N}_{Ah_{t-1}, \, \Gamma}(h_t) \mathcal{N}_{Ch_t, \, \Sigma}(x_t)\right),$$
где $T$ — некоторый момент времени. Если расписать все плотности гауссовских распределений, то выкладкой можно получить, что совместное распределение последовательностей $(h_t)_{t=1}^T$ и $(x_t)_{t=1}^T$ и само является гауссовским. Как следствие, его маржинальные и условные распределения тоже являются гауссовскими.

По сути, линейные динамические системы можно считать аналогами [скрытых марковских моделей](__home_url__/notes/Скрытые марковские модели) (СММ) для того случая, когда скрытые переменные являются непрерывными, а не дискретными. Что для ЛДС, что для СММ граф взаимозависимости переменных выглядит одинаково.

Для ЛДС можно сформулировать четыре следующих задачи:
* Зная параметры $A$, $\Gamma$, $C$, $\Sigma$, $\lambda_0$ и $W_0$, провести восстановление скрытого состояния в режиме реального времени, то есть научиться вычислять условные вероятности $p(h_t \, \vert \, (x_i)_{i=1}^{t-1})$ и $p(h_t \, \vert \, (x_i)_{i=1}^{t})$, где первую условную вероятность можно использовать для прогнозирования $t$-го скрытого состояния на $(t-1)$-м шаге, а вторую — для уточнения такого прогноза, после того как поступило $t$-e наблюдаемое значение.
* Зная параметры $A$, $\Gamma$, $C$, $\Sigma$, $\lambda_0$ и $W_0$, найти наиболее вероятную последовательность скрытых состояний $(h_t)_{t=1}^T$ по заранее известной последовательности наблюдаемых состояний $(x_t)_{t=1}^T$.
* Имея последовательность наблюдаемых состояний $(x_t)_{t=1}^T$ и последовательность скрытых состояний $(h_t)_{t=1}^T$ обучить с учителем параметры $A$, $\Gamma$, $C$, $\Sigma$, $\lambda_0$ и $W_0$, то есть найти их значения, максимизирующие полное правдоподобие.
* Имея последовательность наблюдаемых состояний $(x_t)_{t=1}^T$ обучить без учителя параметры $A$, $\Gamma$, $C$, $\Sigma$, $\lambda_0$ и $W_0$, то есть найти их значения, максимизирующие неполное правдоподобие.

#### Задача восстановления скрытого состояния в режиме реального времени

Эту задачу будем решать индуктивно.

На нулевом шаге прогноз для шага $t=1$ дан в виде начального распределения скрытого состояния: $p(h_1) = \mathcal{N}_{\lambda_0, \, W_0}(h_1)$. Коррекция этого прогноза после поступления значения $x_1$, а именно $p(h_1 \, \vert \, x_1)$, является некоторым гауссовским распределением в силу того, что, по определению, это условное матожидание гауссовской величины при условии другой гауссовской величины, получаемой из первой добавлением гауссовского шума. Матожидание и ковариационную матрицу для $p(h_1 \, \vert \, x_1)$ можно найти по известным формулам для такого случая:
$$p(h_1 \, \vert \, x_1) = \mathcal{N}_{\mu_1, \, V_1}(h_1),$$
$$V_1 = (W_0^{-1} + C^T\Sigma^{-1}C)^{-1},$$
$$\mu_1 = V_1(C^T\Sigma^{-1}x_1 + W_0^{-1}\lambda_0),$$
где символ $T$ в верхнем регистре обозначает транспонирование, а не возведение в степень, равную некоторому моменту времени.

Окончив с базой индукции, перейдём к шагу индукции. Предположим, что известно распределение $p(h_{t-1} \, \vert \, (x_i)_{i=1}^{t-1})$, и по нему выведем распределение $p(h_t \, \vert \, (x_i)_{i=1}^{t-1})$. Для начала отметим, что все такие распределения являются гауссовскими, то есть допускают представление в виде $p(h_{t} \, \vert \, (x_i)_{i=1}^{t}) = \mathcal{N}_{\mu_{t}, \, V_{t}}(h_{t})$, где через $\mu_{t}$ и $V_{t}$ обозначены матожидание и ковариационная матрица.

Вычисление прогноза скрытого состояния на следующий шаг выглядит так:
$$p(h_t \, \vert \, (x_i)_{i=1}^{t-1}) = \int p(h_t, h_{t-1} \, \vert \, (x_i)_{i=1}^{t-1}) \, dh_{t-1} = \int p(h_t \, \vert \, h_{t-1}) p(h_{t-1} \, \vert \, (x_i)_{i=1}^{t-1}) \, dh_{t-1} = \int  \mathcal{N}_{Ah_{t-1}, \, \Gamma}(h_t) \mathcal{N}_{\mu_{t-1}, \, V_{t-1}}(h_{t-1}) \, dh_{t-1} = \mathcal{N}_{A\mu_{t-1}, \, \Gamma + AV_{t-1}A^T}(h_{t}).$$

Глядя на получившуюся правую часть, введём пару новых обозначений:
$$\overline{\mu}_t = A\mu_{t-1},$$
$$\overline{V}_t = \Gamma + AV_{t-1}A^T.$$

Тем самым через $\mu_{t-1}$, $V_{t-1}$ и параметры ЛДС, предполагающиеся известными, удалось выразить $\overline{\mu}_t$ и $\overline{V}_t$. Они отличаются от нужных для проведения шага индукции $\mu_t$ и $V_t$ тем, что в них не учитывается информация о текущем наблюдаемом значении $x_t$ (в частности, можно считать, что $\lambda_0 = \overline{\mu}_1$ и $W_0 = \overline{V}_1$). Чтобы выразить $\mu_t$ и $V_t$ через $\overline{\mu}_t$ и $\overline{V}_t$, проведём коррекцию, вносимую получением информации об $x_t$.

Для данной коррекции введём ещё одно обозначение, сокращающее размер формул:
$$K_t = \overline{V}_tC^T(C\overline{V}_tC^T + \Sigma)^{-1}.$$

С использованием этого обозначения можно получить такие формулы коррекции:
$$\mu_t = \overline{\mu}_t + K_t(x_t - C\overline{\mu}_t),$$
$$V_t = (I - K_tC)\overline{V}_t,$$
где $I$ — единичная матрица размера $m \times m$. Вывод этих формул опущен из-за громоздкости.

Напомним, что начальные значения $\mu_1$ и $V_1$ были вычислены при описании базы индукции, так что теперь можно вычислить $\mu_t$ и $V_t$ для любого $t$. Для этого, начав с прогноза на шаг $t=1$, можно в цикле сначала прогнозировать на следующий шаг, а потом корректировать такой прогноз на соответствующее наблюдаемое значение.

Подобная процедура называется проходом вперёд (forward algorithm) — так же, как алгоритм для скрытых марковских моделей, который вычислял аналогичные распределения (правда, с другой конечной целью: с целью вычислительно эффективно найти вероятность последовательности наблюдаемых состояний). Также данный алгоритм называют фильтром Калмана. Слово "фильтр" здесь интерпретируется в том ключе, что наблюдаемые значения предполагаются зашумлёнными версиями истинных скрытых значений и требуется сгладить временной ряд, отфильтровав шум.

#### Задача нахождения наиболее вероятной последовательности скрытых состояний

В отличие от скрытых марковских моделей для решения этой задачи отдельный алгоритм (в случае СММ называющийся алгоритмом Витерби) не нужен: достаточно алгоритма прохода вперёд-назад. Для ЛДС наиболее вероятная последовательность скрытых значений является последовательностью, составленной из индивидуально наиболее вероятных скрытых значений. Это так, потому что, как отмечалось выше, совместное распределение $p\left((h_t)_{t=1}^T, (x_t)_{t=1}^T \, \vert \, A, \Gamma, C, \Sigma, \lambda_0, W_0 \right)$ является нормальным, а для многомерного нормального распределения проекции моды на ось какой-либо компоненты совпадает с модой проекции распределения на эту ось (можно ещё добавить, что мода совпадает с матожиданием).

Тем самым задача сводится к такой: по известной последовательности $(x_i)_{i=1}^T$ найти моду распределения $p(h_t \, \vert \, (x_i)_{i=1}^T)$, где $1 \le t \le T$. Иными словами, теперь требуется восстановить скрытое состояние, зная не только прошлые и настоящие наблюдения, но и некоторые последующие наблюдения. Интуитивно говоря, проход назад для того и нужен, чтобы учесть информацию о более поздних наблюдаемых значениях.

Алгоритм прохода вперёд-назад состоит из двух этапов:
* Сначала алгоритмом прохода вперёд, разобранным ранее, вычисляются вектор среднего и ковариационная матрица для распределения $p(h_T \, \vert \, (x_i)_{i=1}^{T}) = \mathcal{N}_{\mu_T, \, W_T}(h_T)$.
* Затем алгоритмом прохода назад, который будет разобран ниже, вычисляются векторы среднего и ковариационные матрицы для распределений $p(h_t \, \vert \, (x_i)_{i=1}^{T})$ в порядке от $t = T-1$ к $t=1$.

Введём обозначения $\tilde{\mu}_t$ и $\tilde{V}_t$: $p(h_t \, \vert \, (x_i)_{i=1}^{T}) = \mathcal{N}_{\tilde{\mu}_t, \, \tilde{V}_t}(h_t)$. В соответствии с этими обозначениями $\tilde{\mu}_T = \mu_T$ и $\tilde{V}_T = V_T$, а для остальных $t$ эти параметры ищутся при проходе назад.

Для $t$, лежащего между $T-1$ и 1 шаг в алгоритме прохода назад выглядит так:
$$J_t = V_t A^T \overline{V}_t^{-1},$$
$$\tilde{\mu}_t = \mu_t + J_t (\tilde{\mu}_{t+1} - A\mu_t),$$
$$\tilde{V}_t = V_t + J_t (\tilde{V}_{t+1} - \overline{V}_t)J_t^T,$$
где $J_t$ — вспомогательное обозначение, сокращающее формулы, а $V_t$ и $\overline{V}_t$ были вычислены при проходе вперёд. Выписанную тройку формул называют РТС-уравнениями по первым буквам имён её авторов.

## Вариационный автокодировщик

#### Введение

Говоря верхнеуровнево, вариационный автокодировщик является примером нейробайесовского метода, а именно метода, где нейронные сети в качестве инструмента встраиваются в байесовский вывод, чтобы оптимизация, нужная для байесовского вывода, проводилась над нейронными сетями теми средствами, которые разработаны для них.

Основные отличия вариационного автокодировщика от классического автокодировщика таковы:
* И кодировщик, и декодировщик на своих выходных слоях выдают параметры вероятностных распределений;
* Между кодировщиком и декодировщиком сущестует специальная прослойка, нужная для перепараметризации;
* Обучение проводится с функцией потерь, восходящей к вариационному байесовскому выводу.

#### Вероятностная модель

Вариационный автокодировщик связан с вероятностной графической моделью, описываемой следующими предположениями:
* Имеются объекты, каждый из которых обладает $n$ наблюдаемыми признаками, и дана обучающая выборка в виде матрицы $X$ размера $l \times n$, где $l$ — количество объектов;
* Имеется скрытое пространство размерности $h$ (причём с точки зрения предметной области считается, что в этом пространстве в лучшей степени проявляются какие-то важные характеристики объектов), и существует (но не дана) матрица $T$ скрытых признаков, имеющая размеры $l \times h$;
* Имеется вектор $\theta$, интерпретируемый как вектор неизвестных параметров вероятностной модели; забегая вперёд, отметим, что это будет вектор весов декодирующей нейронной сети;
* Все объекты являются независимыми друг от друга, причём как в смысле своих наблюдаемых признаков $X_{i\cdot}$, так и в смысле своих скрытых признаков $T_{i\cdot}$, благодаря чему вместо совместной условной плотности для всей выборки $p(X, T \, \vert \, \theta)$ можно рассматривать совместную условную плотность для произвольного $i$-го объекта $p(X_{i\cdot}, T_{i\cdot} \, \vert \, \theta) = p(x_i, t_i \, \vert \, \theta)$, где в правой части просто ввели более удобные обозначения $x_i = X_{i\cdot}$ и $t_i = T_{i\cdot}$;
* Скрытые признаки объекта влияют (с точки зрения графических моделей) на его наблюдаемые признаки, но не наоборот, то есть сначала порождается $t_i \sim p(t_i)$ (тут априорное распределение $t_i$ не зависит от $\theta$), а затем порождается $x_i \sim  p(x_i \, \vert \, t_i, \theta)$.

Обозначим кортеж из строк матрицы $T$ и элементов вектора $\theta$ за $Z$ и благодаря перечисленным предположениям перепишем вероятностную модель так:
$$p(X, Z) = p(\theta) p(X, T \, \vert \, \theta) = p(\theta) \prod_{i=1}^l p(x_i \, \vert \, t_i, \theta) p(t_i).$$
Определение задачи статистического вывода требует явного аналитического вида для $p(X, Z)$ и, чтобы его получить, опишем каждый из сомножителей правой части:
* Предполагается, что параметры вероятностной модели (а именно веса нейронной сети) имеют атомарное распределение, $p(\theta) = \delta(\theta - \theta_0)$, где $\delta$ — дельта-функция Дирака; введение такого ограничивающего предположения вызвано тем, что с точки зрения нейронных сетей обучаются только сами веса, но не их распределение, а с точки зрения вариационного байесовского вывода тем, что вовлечение нейронных сетей оставляет лишь более слабый аналог сопряжённости;
* Распределение $p(x_i \, \vert \, t_i, \theta)$ является распределением из заданного параметрического семейства, предсказываемым некоторой нейронной сетью, обладающей следующими свойствами:
    - на вход подаётся $t_i$,
    - все веса извлекаются из $\theta$,
    - на выходе получаются параметры распределения $p(x_i \, \vert \, t_i, \theta)$ (например, если считать, что речь идёт о картинках размера $d \times d$ пикселей, каждый из которых чёрный или белый, то можно положить, что распределение $p(x_i \, \vert \, t_i, \theta)$ параметризовано $d^2$ вероятностями того, что соответствующий пиксель является чёрным),
    - прочие детали архитектуры могут быть произвольными (например, размерность выходного слоя в общем случае не обязана быть равна $n$, если распределение $n$-мерных строк $x_i$ матрицы $X$ параметризовано не $n$ параметрами);
* Для вариационных автокодировщиков распределение $p(t_i)$ принято полагать стандартным нормальным распределением. Делается это, чтобы вариационный автокодировщик выучивал нетривиальные компактные представления, а не сопоставлял бы одному и тому же скрытому классу точки из совершенно разных регионов скрытого простанства, а потом запоминал бы, какому классу какие регионы приписывал.

В рамках так описанной вероятностной модели $p(X, Z)$ можно поставить задачу статистического вывода, то есть задачу оценивания $p(Z \, \vert \, X) = p(T, \theta \, \vert \, X)$. Из-за вовлечения вариационного вывода оценка будет найдена в виде $\delta(\theta - \theta_0 \, \vert \, X)p(T \, \vert \, X)$.

#### Использование нейронных сетей

Когда специфицировалась вероятностная модель, уже была построена одна нейронная сеть, а именно сеть, возвращающая параметры, задающие распределение $p(x_i \, \vert \, t_i, \theta)$. Эту нейронную сеть называют декодировщиком (decoder), потому что она отображает вектор, лежащий в скрытом пространстве, в вероятностное распределение, откуда можно сэмплировать наблюдаемые признаковые описания объектов. Также её называют порождающей нейронной сетью (generative network), поскольку она позволяет порождать признаковые описания объектов.

Вторая нейронная сеть возникает из-за поиска приближения апостериорного распределения $p(T \, \vert \, X)$ в специальном классе распределений $Q$. Это класс $h$-мерных гауссовских распределений с диагональной ковариационной матрицей. Любое такое распределение $q \in Q$ параметризуется при помощи $2h$ скаляров, а именно $h$ компонент вектора среднего $\mu$ и $h$ диагональных элементов ковариационной матрицы $S$. Однако в вариационном автокодировщике вместо параметризации этими параметрами используется параметризация весами $\phi$ нейронной сети, архитектура которой может быть произвольной с точностью до двух ограничений: вход нейронной сети имеет размер $n$, а её выход имеет размер $2h$, причём выход именно как среднее и диагональ ковариационной матрицы и интерпретируется. Такая нейронная сеть называется кодировщиком (encoder), потому что она отображает вектор наблюдаемых признаков объекта в вероятностное распределение вектора из скрытого пространства. Также эту нейронную сеть называют нейронной сетью вывода (inference network), потому что по её выдаче строится аппроксимация распределения $p(Z \, \vert \, X)$. Далее через $q_\phi(t_i \, \vert x_i)$ будем обозначать распределение $h$-мерного вектора $t_i$, являющееся гауссовским распределением со средним и диагональной ковариационной матрицей, получающимися, если кодировщику с весами $\phi$ подать на вход $n$-мерный вектор $x_i$. 

Две описанных нейронных сети собираются в одну, являющуюся автокодировщиком, при помощи добавления между ними соединительной прослойки, осуществляющей приём перепараметризации (reparametrization trick). Данный приём позволяет так соединить кодировщик с декодировщиком, чтобы можно было вычислять производные выхода всего автокодировщика по параметрам кодировщика. По умолчанию взятие таких производных затруднено тем, что для получения входа кодировщика нужно просэплировать вектор из распределения, получаемого на базе выхода кодировщика, но в сэмпле, строго говоря, нет явной зависимости от весов кодировщика, потому что просэмплированный вектор является константой. Итак, опишем соединительную прослойку. Пусть она будет юнитом, который получает на вход весь выход кодировщика (из которого однозначно получаются вектор средних $\mu$ и ковариационная матрица $S$) и дополнительно некоторый случайный вектор $\nu$, просэмплированный из $h$-мерного стандартного нормального распределения. Юнит-прослойка не обучается и всегда выполняет одно и то же преобразование: возвращает $h$ компонент вектора $S\nu + \mu$, то есть вектора, который распределён уже по нормальному распределению со средним $\mu$ и ковариационной матрицей $S$. Обозначим этот вектор за $V(q_\phi(t_i \, \vert x_i), \nu)$. Его компоненты подаются как входы декодировщику. Так и устроен автокодировщик, собираемый из кодировщика и декодировщика.

Назван же получившийся автокодировщик вариационным, потому что при его обучении в функции потерь участвует вариационная регуляризация. Выглядит сама функция потерь так:
$$-\mathrm{ELBO}(\phi, \theta) = \sum_{i=1}^l \left(-\mathbb{E}_{\nu \sim \mathcal{N}_{0, I_{h}}}[\log p(x_i \, \vert \, V(q_\phi(t_i \, \vert x_i), \nu), \theta)] + D_{KL}(q_\phi(t_i \, \vert x_i) \Vert p(t_i)) \right),$$
где $I_h$ — единичная матрица размера $h \times h$. На дивергенцию Кульбака-Лейблера $D_{KL}(q_\phi(t_i \, \vert x_i) \Vert p(t_i))$ можно смотреть как на регуляризатор, а на $\mathbb{E}_{\nu \sim \mathcal{N}_{0, I_{h}}}[\log p(x_i \, \vert \, V(q_\phi(t_i \, \vert x_i), \nu), \theta)]$ можно смотреть как на ожидаемую ошибку восстановления $x_i$ после прогона его же самого через автокодировщик, куда в юнит-прослойку подали случайный вектор $\nu$. Точнее же, это ожидаемый логарифм правдоподобия известного $x_i$ относительно распределения, возвращаемого нейронной сетью, параметры которой равны $\theta$, при подаче ей на вход вектора $V(q_\phi(t_i \, \vert x_i), \nu)$.

Обучение с указанной функцией потерь возможно теми же методами, какими обычно обучаются нейронные сети, то есть, например, стохастическим градиентным спуском с какими-нибудь дополнительными приёмами по управлению темпом обучения. Результаты же получаются интересными, потому что функция потерь взята не с потолка, а является умноженной на минус один нижней вариационной оценкой (evidence lower bound, ELBO) для $\log p(X)$. Иными словами, за этой функцией потерь стоит теория байесовского вывода в приведённой в предыдущем разделе графической модели.

Кстати, на стохастический градиентный спуск можно посмотреть как на то, что приводит к стохастическому EM-алгоритму, являющемуся существенно более быстрым, чем детерминированный EM-алгоритм, если $l \gg n$.

#### Применение

Вариационный автокодировщик можно использовать в качестве порождающей модели. Подача на вход декодировщику вектора, просэмплиованного из априорного распределения $p(t)$, то есть из $h$-мерного стандартного нормального распределения, приведёт к появлению на выходе декодировщика априорного предсказательного распределения (prior predictive distribution), откуда можно сэмплировать объекты, похожие на объекты из генеральной совокупности, породившей $X$. Подача же на вход декодировщику вектора, просэмплированного из распределения $q_\phi(t_i \, \vert x_i)$, где $x_i$ как-то взят, приведёт к появлению на выходе декодировщика апостериорного предсказательного распределения (posterior predictive distribution), откуда можно сэмплировать объекты, похожие на $x_i$.

## Калибровка предсказанных вероятностей

Некоторые алгоритмы классификации (например, метод опорных векторов) возвращают только класс, к которому, по их предсказаниям, принадлежит объект, но не в состоянии вернуть вероятности принадлежности к классам. Другие алгоритмы классификации способны возвращать также степени своей уверенности в принадлежности объекта к классам, однако зачастую с этими степенями уверенности надлежит обращаться лишь как с порядковыми величинами, а не как с количественными. Хотя такие величины иногда и называют предсказанными вероятностями принадлежности к классу, в общем случае нет никаких гарантий того, что они объективно отражают степени неопределённости. В частности, в статье [Niculescu-Mizil, Caruana, 2004](http://www.datascienceassn.org/sites/default/files/Predicting%20good%20probabilities%20with%20supervised%20learning.pdf) утверждается, что:
* Наивный байесовский классификатор смещает вероятности ближе к краям, то есть ближе к 0% или к 100%;
* Метод опорных векторов и градиентный бустинг над классификационными деревьями смещают вероятности от краёв к центру.

С точки зрения метрик, зависящих только от возвращаемого моделью упорядочивания объектов (скажем, таких метрик, как ROC-AUC), описанная проблема не имеет значения. Однако в логарифмической функции потерь присутствуют уже сами предсказанные вероятности, а не только порядковые номера объектов, так что если бы истинные вероятности предсказывались более точно, значения метрик могли бы быть выше. Также истинные вероятности бывают важны с точки зрения предметной области: например, в задаче кредитного скоринга потенциальный убыток пропорционален вероятности невозврата кредита. В связи со всем вышесказанным нужны техники, способные откалибровать предсказанные вероятности, сделав их более похожими именно на количественные вероятности.

Простым примером такой техники является калибровка Платта. В случае бинарной классификации она выглядит так:
* Обучающая выборка (то есть то, что изначально предлагалось использовать только для настройки параметров, а не для подбора гиперпараметров и слепой финальной оценки) разбивается на то, на чём и будет проходить обучение (скажем, 90% по размеру), и на выборку для калибрации (стало быть, 10% по размеру);
* На подвыборке, выделенной под обучение, настраивается модель;
* Если модель возвращает предсказанные вероятности, то в качестве нового признака объектов из калибрационной выборки берётся вероятность положительного класса. Однако даже если модель не возвращает вероятности, достаточно лишь того, чтобы модель возвращала что-то, по чему объекты можно отранжировать: например, для линейных классификаторов это будет значение функции $g(Xw)$, по знаку которого определяется предсказанный класс;
* Обучается логистическая регрессия без какой-либо регуляризации, по единственному новому признаку объектов калибрационной выборки (и константе) предсказывающая их класс. Предсказанная ей вероятность уже и будет считаться откалиброванной вероятностью. Эффект калибровки заключается в том, что методом максимального правдоподобия будут найдены коэффициент масштаба $a$ и коэффициент сдвига $b$, такие что значения $\sigma(ax+b)$, где $\sigma$ — сигмоидная функция, станут более содержательными. Работает это в основном тогда, когда искажение предсказанных вероятностей относительно истинных вероятностей действительно может быть исправлено сигмоидальным преобразованием.

## Особенности архитектуры DCGAN

Аббревиатура DCGAN расшифровывается как Deep Convolutional GAN (Generative Adversarial Network). Однако все или почти все GAN'ы, работающие с изображениями, являются глубокими и свёрточными, так что название, данное авторами, слишком размыто. Более мелкие детали отличают архитектуру DCGAN от прочих архитектур, таких как LAPGAN. Эти детали таковы:
* Отсутствует [субдискретизация](__home_url__/notes/Субдискретизация) (pooling); вместо этого в дискриминаторе используются свёрточные слои с шагом (stride), равным 2 или более, а в генераторе используются свёрточные слои с дробным шагом, меньшим 1 (fractionally-strided convolution layers), также известные как транспонированные свёрточные слои (transposed convolution layers, ещё их ошибочно называют deconvolution layers, хотя они не осуществляют обратное преобразование к свёртке); никаких слоёв, кроме вышеописанных свёрточных, в DCGAN нет;
* Везде в генераторе используется функция активации ReLU кроме выходного слоя, где используется гиперболический тангенс; в дискриминаторе же везде используется Leaky ReLU.

Дополнительно можно отметить, что для обучения DCGAN применяются недавно набравшие популярность техники, такие как:
* [Пакетная нормализация](__home_url__/notes/Пакетная нормализация) (batch normalization);
* [Оптимизатор Adam](__home_url__/notes/Метод подбора темпа обучения нейронной сети Adam).

## Баланс между изучением и применением в задаче о многоруком бандите

Задача о многоруком бандите (multi-armed bandit) формулируется так: имеется $n$ неизвестных вероятностных распределений, на каждом шаге даётся возможность просэмплировать случайную величину ровно из одного распределения и узнать её значение, а требуется совершить некоторое заранее заданное конечное число шагов так, чтобы сумма просэмплированных величин была наибольшей.

В такой задаче возникает компромиссный выбор между пополнением знаний и применением уже накопленных знаний (exploration-exploitation dilemma). С одной стороны, после того, как из каждого распределения было просэмплировано сколько-то величин, можно просто далее сэмплировать величины только из распределения с наибольшим эмпирическим средним — это будет применение собранных знаний (exploitation). С другой стороны, оценки эмпирического среднего могут неточно приближать истинное среднее, и чем больше величин из распределения просэмплировано, тем точнее в смысле центральной предельной теоремы будет оценка среднего, так что имеет смысл сэмплировать величины из всех распределений — это будет пополнение знаний (exploration). Таким образом, нужно соблюсти оптимальный баланс между изучением и применением.

Существуют следующие стратегии поиска баланса между изучением и применением:
* $\varepsilon$-жадная стратегия: с вероятностью $\varepsilon$ выбирать распределение, откуда сэмплировать значение на текущем шаге, равновероятно, а с вероятностью $(1 - \varepsilon)$ сэмплировать значение из того распределения, которое имеет наибольшую текущую оценку среднего; параметр $\varepsilon$ может быть константой, но лучше сделать, чтобы $\varepsilon$ со временем затухал: например, положить $\varepsilon = \frac{1}{t}$, где $t$ — номер шага.
* Стратегия оптимистичных начальных значений: считать, что до первого шага оценки эмпирического среднего не неопределены, а равны какому-то чрезмерно завышенному значению, а сэмплировать всякий раз "жадно" — поскольку со временем оценки эмпирического среднего в таком случае будут только снижаться, это будет подталкивать изучать распределения, по которым собрано мало статистики, но из-за "жадности" не будет бесконечно долгого изучения.
* UCB, Upper Confidence Bound. Для $j$-го варианта рассчитанная полезность равна
$$\overline{x}_j + \sqrt{\frac{2\log n}{n_j}},$$
где $\overline{x}_j$ — накопленное среднее при использовании $j$-го варианта, $n_j$ — количество предыдущих выборов $j$-го варианта, а $n$ — количество всех уже сделанных шагов. На каждом шаге выбирается тот вариант, у которого наибольшая рассчитанная полезность. На формулу рассчитанной полезности можно посмотреть как на эвристику, где к "жадной" полезности добавляется слагаемое, побуждающее исследовать, но у этой формулы есть и обоснование, восходящее к неравенству Хёффдинга.
* Сэмплирование Томпсона. Считается, что каждое распределение принадлежит к параметрическому семейству, параметры которого имеют своё собственное распределение (желательно такое, чтобы пара соответствующих распределений была сопряжённой). На каждом шаге происходит следующее:
    - для каждого из вариантов сэмплируется значение его параметров;
    - выбирается тот вариант, для которого при просэмплированных на предыдущем шаге параметрах математическое ожидание наибольшее;
    - в зависимости от полученного значения-ответа обновляется распределение параметров выбранного варианта.

## Об итеративности сведения реальной задачи к задаче машинного обучения

При наличии прикладной задачи и относящихся к ней данных далеко не всегда удаётся с первого раза подобрать такую постановку задачи машинного обучения, при которой её решение было бы частью решения исходной прикладной задачи. Ниже этот тезис иллюстрируется примером.

Допустим, стоит такая задача: агрегатор такси хочет находить пользоваталей, которые в ближайшее время откажутся от его услуг, и отправлять им сообщение с купоном на скидку на следующую поездку. Казалось бы, почему нельзя сформулировать задачу машинного обучения как задачу бинарной классификации, где объектами являются пары (клиент, дата), а целевая переменная равна единице, если клиент после соответствующей даты не пользовался услугами компании на протяжении некоторого зафиксированного срока, и нулю иначе? В некоторых случаях может быть так, что данная постановка будет хорошей. Однако предположим, что после разработки решения обнаружилось, что для отправки купона на скидку отбираются только пользователи, оставившие об агрегаторе такси негативные отзывы в интернете или позвонившие с жалобами на горячую линию, причём разбор этих отзывов и жалоб показал, что клиенты недовольны тем, что водители такси создавали аварийные ситуации. Очевидно, скидка здесь будет неэффективным инструментом удержания аудитории, но цель заказчика — именно удержание клиентов, а не просто угадывание тех из них, кто склонен уйти к конкурентам. Решением вскрывшейся проблемы может быть постановка двухэтапной задачи. На первом этапе классификатор разделяет клиентов на тех, которым была бы интересна скидка, и на тех, кому она неинтересна, причём отдельно стоит продумать, как разметить данные для такой классификации. На втором же этапе оставляются только объекты, для которых предсказано, что клиенту интересна скидка, и на прореженных данных происходит обучение и тестирование классификатора, решающего, склонен ли клиент уйти. Обученный классификатор, разумеется, применяется тоже только для тех объектов, для которых предсказано, что скидка может иметь эффект.

Существование риска замены постановки задачи машинного обучения является аргументом в пользу того, что заниматься тонкой настройкой имеет смысл лишь после того, как начальная версия решения уже была опробована и по ней была получена обратная связь.

## Лучевой поиск для порождения последовательностей

Пусть дана порождающая вероятностная модель $\mathbb{P}\!\left(x_i \, \left| \, \{x_j\}_{j=1}^{i-1}, v\right.\right)$, где $x_i$ — текущий элемент последовательности, $\{x_j\}_{j=1}^{i-1}$ — предшествующее ему начало этой последовательности, а $v$ — некоторый вектор (или иной объект), хранящий в себе дополнительную известную информацию (например, если порождается предложение, являющееся переводом заданного предложения на другом языке, то $v$ может быть векторным представлением переводимого предложения). Возникает вопрос: как из такой модели сэмплировать (порождать) последовательности?

Простейшим решением является "жадное" порождение. Можно выбрать начало по правилу:
$$x_1 = \arg \max_x \mathbb{P}\!\left(x \, \left| \, \{\}, v\right.\right),$$
а всякий раз далее выбирать следующий элемент по правилу:
$$x_i = \arg \max_x \mathbb{P}\!\left(x \, \left| \, \{x_j\}_{j=1}^{i-1}, v\right.\right).$$
Как это и бывает с "жадными" алгоритмами, нет никаких гарантий, что полученная таким образом последовательность будет иметь наибольшую вероятность. Например, пусть рассматриваются последовательности длины 2 над символами $a$ и $b$, дополнительной информации нет (так что $v$ можно опустить), а вероятностная модель имеет вид:
$$\mathbb{P}\!\left(a \, \left| \, \{\}\right.\right) = 0.6,$$
$$\mathbb{P}\!\left(b \, \left| \, \{\}\right.\right) = 0.4,$$
$$\mathbb{P}\!\left(a \, \left| \, \{a\}\right.\right) = 0.5,$$
$$\mathbb{P}\!\left(b \, \left| \, \{a\}\right.\right) = 0.5,$$
$$\mathbb{P}\!\left(a \, \left| \, \{b\}\right.\right) = 0.9,$$
$$\mathbb{P}\!\left(b \, \left| \, \{b\}\right.\right) = 0.1.$$
В рамках такой модели наиболее вероятной последовательностью является последовательность $ba$, однако "жадный" поиск закончится ничьёй между $aa$ и $ab$.

Другой крайностью является полный (экспоненциальный) перебор, когда для каждой возможной последовательности считаются её вероятности, а затем выбирается та последовательность, у которой вероятность наибольшая. Разумеется, данный подход является вычислительно неподъёмным для порождения хоть сколько-нибудь длинных последовательностей.

Лучевой поиск (beam search) является компромиссным соединением двух вышеописанных подходов в один новый подход. По сути, это тоже "жадный" алгоритм, но чуть более дальновидный.

Алгоритм лучевого поиска для порождения последовательности длины $l$ (или не более чем $l$, если среди символов есть символ конца последовательности, после появления которого сэмплирование останавливается) получает на вход помимо собственно вероятностной модели и вектора $v$ массив из $l$ чисел $\{n_i\}_{i=1}^l$, интерпретируемых как количества подпоследовательностей, остающихся в рассмотрении после каждого шага.

Устроен же сам алгоритм так. На первом шаге отбираются $n_1$ различных одноэлеметных подпоследовательностей, составленных каждая из одного из $n_1$ элементов с наибольшей $\mathbb{P}\!\left(x \, \left| \, \{\}, v\right.\right)$. На каждом же из последующих шагов (обозначим номер шага за $i$, $i > 1$) для каждой из $n_{i-1}$ отобранных на предыдущем шаге подпоследовательностей рассматриваются все её возможные продолжения на один элемент вперёд и из получившихся таким образом новых подпоследовательностей отбираются $n_i$ наиболее вероятных. Под конец из $n_l$ отобранных последовательностей выбирается одна наиболее вероятная и возвращается как результат.

"Жадный" алгоритм, приводившийся как пример решения в начале этой заметки, является частным случаем лучевого поиска при всех $n_i = 1$, а полный перебор является частным случаем лучевого поиска при значениях $n_i$, равных количествам всех без исключения возникающих на соответствующих шагах последовательностей.

## Архитектура seq2seq

#### Введение

В задачах глубокого обучения, где объекты являются последовательностями, существуют различные варианты того, когда и как принимаются входы и отдаются выходы. В частности, возможны такие варианты:
* принимается вся входная последовательность целиком, а отдаётся одна метка, а не последовательность (пример: задача анализа тональности предложений, то есть определения того, является ли произвольное предложение положительным, отрицательным или нейтральным с точки зрения эмоционального фона);
* входная последовательность принимается элемент за элементом, и до того, как будет принят следующий входной элемент, должен быть отдан текущий элемент выходной последовательности (пример: задача онлайнового/потокового разбора предложения по частям речи);
* принимается вся входная последовательность целиком, а затем порождается выходная последовательность, причём ограничение, что длина выходной последовательности обязана быть равной длине входной последовательности, может налагаться, а может и не налагаться.

Архитектура seq2seq применима именно к последней из этих ситуаций, где ограничения на длину выхода нет. Классическим примером задачи, подпадающей под такое описание, является машинный перевод: во-первых, чтобы перевести предложение, желательно сначала дочитать его до конца и лишь потом начинать писать предложение на другом языке, и, во-вторых, в оригинале и в переводе количество слов может не совпадать. Несовпадение длин входной и выходной последовательностей и является одной из основных причин, по которым нужны особые архитектуры вместо просто рекуррентных нейронных сетей.

Стоит отметить, что помимо seq2seq для генерации новой последовательности по целиком известной входной последовательности могут применяться и другие архитектуры: например, трансформеры.

#### Вероятностная интерпретация

Вероятностная модель, стоящая за seq2seq, имеет следующий вид. Пусть $\{x_i\}_{i=1}^l$ — входная последовательность, $\{y_i\}_{i=1}^m$ — выходная последовательность, а $\mathbb{P}\!\left(\{y_i\}_{i=1}^m \, \left| \, \{x_i\}_{i=1}^l \right.\right)$ — неизвестное условное вероятностное распределение на выходных последовательностях при условии заданной входной последовательности. Общая задача сводится к отображению $\{x_i\}_{i=1}^l$ в $\arg \max_{\{y_i\}_{i=1}^m} \mathbb{P}\!\left(\{y_i\}_{i=1}^m \, \left| \, \{x_i\}_{i=1}^l \right.\right)$. В рамках архитектуры seq2seq вводится дополнительное предположение, что
$$\mathbb{P}\!\left(\{y_i\}_{i=1}^m \, \left| \, \{x_i\}_{i=1}^l \right.\right) \approx \mathbb{P}\!\left(\left.\{y_i\}_{i=1}^m \, \right| \, v \right) \cdot \, \mathbb{P}\!\left(v \, \left| \, \{x_i\}_{i=1}^l \right.\right),$$
где $v$ — вектор фиксированного размера.

В терминологии нейронных сетей сделанное предположение позволяет ввести архитектуру seq2seq, составляющими которой являются:
* кодировщик, который:
    - на стадии обучения оценивает распределение $\mathbb{P}\!\left(v \, \left| \, \{x_i\}_{i=1}^l \right.\right)$,
    - и на стадии обучения, и на стадии применения для каждой входной последовательности $\{x_i\}_{i=1}^l$ произвольной длины $l$ оценивает $\hat{v} = \arg \max_v \hat{\mathbb{P}}\!\left(v \, \left| \, \{x_i\}_{i=1}^l \right.\right)$;
* декодировщик, который:
    - на стадии обучения оценивает распределение $\mathbb{P}\!\left(\left.\{y_i\}_{i=1}^m \, \right| \, v \right)$, где длина вектора $v$ фиксирована,
    - и на стадии обучения, и на стадии применения по переданному кодировщиком значению вектора $v$ "жадным" образом, [лучевым поиском](__home_url__/notes/Лучевой поиск для порождения последовательностей) или ещё как-либо порождает выходную последовательность из выучиваемого или выученного распределения $\mathbb{P}\!\left(\left.\{y_i\}_{i=1}^m \, \right| \, v \right) \approx \prod_{i=0}^{m-1} \mathbb{P}\!\left(y_{i+1} \, \left| \, \{y_j\}_{j=1}^i, v \right.\right)$, где равенство выписано в предположении присутствия марковского свойства.
    
Вектор $v$, передаваемый кодировщиком, можно интерпретировать как векторное представление последовательности, на которой он был получен. Например, в случае с машинным переводом кодировщик становится чем-то вроде sentence2vec.
    
#### Детали реализации

Первый вопрос, возникающий в связи с деталями реализации seq2seq, звучит так: как можно обеспечить работу с переменными значениями длин $l$ и $m$? Ответ на него таков. Предполагается, что одним из элементов последовательностей может быть специальный символ "EOS", интерпретируемый как конец последовательности, после появления которого работа с последовательностью прекращается. Кстати, при обучении пакетным градиентным спуском можно разбивать данные на пакеты так, чтобы в одном пакете все входные последовательности имели одинаковую длину (такой приём по-английски называется bucketing) — советуют так делать не потому, что пакеты должны быть представлены прямоугольными матрицами (для этого концы более коротких последовательностей можно просто дозаполнить нулями), а потому, что считается, будто от этого увеличивается качество моделей.

Второй важный вопрос — какой кодировщик использовать. В порядке от простого к сложному можно назвать следующие варианты:
* обычная [LSTM-сеть](__home_url__/notes/Внутреннее устройство LSTM-нейрона),
* двусторонняя (bidirectional) LSTM-сеть,
* обычная или двусторонняя LSTM-сеть, доработанная для поддержки механизма внимания.

Чтобы понять, что даёт каждое из этих усложнений, начнём с самого простого варианта. В случае обычной LSTM-сети через неё прогоняется вся входная последовательность, а затем финальное скрытое состояние и становится отдаваемым значением вектора $v$. Проблема тут в том, что обычно первый элемент выходной последовательности должен во многом зависеть от первого элемента входной последовательности, но если входная последовательность достаточно длинная, то в скрытом состоянии может не остаться информации о первом элементе. Поскольку же все последующие порождаемые элементы зависят от первого, порождение плохого первого элемента может привести к плохому конечному результату. В статье [Sutskever et al, 2014](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf), являющейся одной из первых статей о seq2seq, для задачи машинного перевода использовались односторонние сети, но был предложен приём, улучшающий качество старта декодировщика. Входная последовательность проходилась в обратном порядке: от конца к началу. С другой стороны, из-за этого приёма увеличивается расстояние между последним элементом входной последовательности и последним элементом выходной последовательности. Двусторонние сети как раз и сочетают сильные стороны прохода что в прямом порядке, что в обратном. В случае seq2seq использование двусторонних сетей возможно, ведь вся входная последовательность доступна сразу же. Но и это не предел: добавление внимания позволяет при порождении каждого следующего элемента выходной последовательности использовать ещё более актуальную информацию о входной последовательности. Все скрытые состояния кодировщика (а не только последнее) сохраняются, а на каждом шаге в качестве вектора $v$ берётся их линейная комбинация с весами, зависящими от скрытого состояния декодировщика.

Наконец, третий вопрос касается декодировщика. Он может быть реализован только как односторонняя рекуррентная сеть, потому что выходная последовательность порождается строго от начала к концу, а информация об уже порождённых элементах учитывается далее через передающееся скрытое состояние.

Также необходимо подчеркнуть, что при обучении декодировщика те элементы выходной последовательности, которые он порождает, на следующих шагах всегда заменяются на элементы выходной последовательности обучающего примера и используется скрытое состояние, рассчитанное по ним. Делается так, потому что иначе обучение генерировать неначальные элементы стояло бы на месте. Однако из-за этого, если декодировщик ошибётся на этапе применения и породит странный элемент, с очень высокой вероятностью остаток последовательности будет порождён некачественно. Для борьбы с этой проблемой используют приёмы наподобие word dropout.

## Восстановление плотности при помощи EM-алгоритма

Пусть дана обучающая выборка из $l$ независимых одинаково распределённых объектов, представленная матрицей $X$ размера $l \times n$, где $n$ — количество признаков. Пусть также имеется предположение об аналитическом виде функции плотности распределения, откуда порождена выборка, то есть имеется $p(x \, \vert \, \theta)$, где $x$ — вектор-строка признакового описания объекта, а $\theta$ — вектор неизвестных параметров распределения. Например, вид такой функции может быть известен благодаря представлениям о предметной области.

Если найти $\theta$, то тем самым будет восстановлена плотность, что, с одной стороны, позволит углубить понимание данных и, с другой стороны, даст возможность сэмплировать новые объекты из того же самого распределения.

Одним из возможных решений описанной задачи восстановления плотности является метод максимального правдоподобия. Впрочем, если распределение $p(x \, \vert \, \theta)$ не принадлежит к классу экспоненциальных распределений, могут возникнуть трудности, связанные с многоэкстремальностью возникающей оптимизационной задачи.

Есть и более интересное решение. Можно дополнительно ввести $h$ скрытых признаков объекта. Обозначим вектор-строку скрытых признаков объекта за $t$ и переопределим $\theta$ как вектор параметров следующей модели:
$$p(x, t \, \vert \, \theta) = p(x \, \vert \, t, \theta) p(t \, \vert \, \theta).$$
Если сделать это удачным образом, то можно добиться того, что распределение $p(x, t \, \vert \, \theta)$ окажется в экспоненциальном классе. Правда, из-за присутствия скрытых переменных найти $\theta$ методом максимального правдоподобия уже не получится. Придётся использовать [EM-алгоритм](__home_url__/notes/Общий взгляд на EM-алгоритм).

Выше было употреблено размытое словосочетание "удачным образом". Проиллюстрируем его значение на примерах. Допустим, исходная $p(x \, \vert \, \theta)$ соответствует смеси нормальных распределений. Это распределение не принадлежит к экспоненциальному классу. Однако предположим, что теперь введён один дополнительный категориальный признак $t$, кодирующий номер компоненты смеси, откуда был порождён объект. Тогда $p(x, t \, \vert \, \theta)$ будет лежать в экспоненциальном классе, потому что разлагается в произведение двух множителей, из которых один, а именно $p(x \, \vert \, t, \theta)$, является плотностью нормального распределения, а другой, то есть $p(t \, \vert \, \theta)$, является плотностью дискретного распределения. Сам по себе такой шаг хорош. Однако и его можно реализовать неправильно. Количество компонент смеси $k$ должно быть подобрано, и здесь есть опасность недоподгонки или переподгонки. Например, в вырожденном случае $k = l$ каждый объект окажется в своей собственной компоненте, ковариационную матрицу которой, кстати, нельзя будет достоверно оценить.

## Обучение на выборке с малым количеством положительных примеров

Допустим, предстоит решить задачу бинарной классификации, где объектов положительного класса не более нескольких десятков (а объектов отрицательного класса заметно больше). Опасность здесь кроется в том, что при хоть сколько-нибудь большом количестве исходных признаков резко возрастает риск переподгонки (overfitting), вызванной обнаружением иллюзорных закономерностей. Высоки шансы того, что какая-либо комбинация признаков чисто случайно позволит хорошо отделить объекты положительного класса. На то же самое можно посмотреть и под таким углом: для отделения небольшого количества точек в пространстве высокой размерности подойдут многие довольно простые разделяющие поверхности.

Иногда решить проблему удаётся при помощи вовлечения автокодировщика. Пусть автокодировщик со скрытым слоем ощутимо меньшей размерности будет обучен по признакам и классу объекта восстанавливать их же. Тем самым будет выучено признаковое представление, от которого ожидается, что все новые признаки будут информативными и отражающими важные для задачи стороны реального мира. Если выйдет получить такое признаковое представление, риск переподгонки сократится. 

## Регрессия на основе гауссовских процессов

#### Необходимые определения

Регрессия на основе гауссовских процессов опирается на предположение о том, что целевая переменная порождена гауссовским процессом. Значит, уместно дать определение того, что такое гауссовский процесс. Сделаем это поэтапно.

Определим случайное поле как множество мощности континуум, состоящее из случайных величин $\xi(x, \omega)$, где $x \in \mathbb{R}^n$ индексирует множество, $n \in \mathbb{N}$, а $\omega$ обозначает тот или иной исход случайности и далее ради краткости будет опускаться. В контексте машинного обучения каждый индексирующий элемент $x$ интерпретируется как признаковое описание некого объекта, и тогда $n$ становится количеством признаков. Если считать, что целевая переменная задаётся случайным полем, то для некоторого объекта $x$ его пронаблюдавшаяся целевая переменная является реализацией случайной величины $\xi(x, \omega)$ при выпавшем исходе $\omega$. Если же зафиксировать $\omega$ на исходе $\overline{\omega}$, то тогда $\xi(x, \overline{\omega})$ будет детерминированной функцией, описывающей зависимость реализовавшейся целевой переменной от признаков объекта.  

Гауссовским процессом назовём случайное поле, такое что $\forall k \in \mathbb{N}$ $\forall x_1, \dots, x_k$, таких что все эти $x_i \in \mathbb{R}^n$, $k$-мерная случайная величина $(\xi(x_1), \dots, \xi(x_k))$ имеет $k$-мерное гауссовское распределение. На самом деле, иногда гауссовские процессы называют гауссовскими полями, и такой термин звучит более корректно, потому что отсылает к пространственной протяжённости (к изменению каждого из признаков), а не к протяжённости во времени.

Подобно тому, как гауссовское распределение полностью задаётся математическим ожиданием и ковариационной матрицей, гауссовский процесс полностью задаётся парой из двух функций, из которых первая называется функцией среднего, а вторая — ковариационной функцией:
$$m(x) = \mathbb{E}\xi(x),$$
$$K(x_1, x_2) = \mathrm{Cov}(\xi(x_1), \xi(x_2)).$$

Важным классом гауссовских процессов являются стационарные процессы. Стационарностью (в слабом смысле) называют соблюдение следующих двух свойств: $m(x)$ не зависит от $x$, а $K(x_1, x_2)$ зависит только от $x_1 - x_2$, то есть представима в виде $K(x_1 - x_2)$.

#### Формулы Андерсона

Пусть целевая переменная порождена стационарным гауссовским процессом $\xi(x)$ с $m(x) \equiv 0$. Предположение о стационарности введено для упрощения выкладок, и оно не ограничивает общность, потому что произвольный гауссовский процесс можно остационарить. Опять же, из стационарного процесса всегда можно вычесть его среднее, чтобы получить процесс с нулевым средним.

Далее, предположим, что ковариционная функция $K(x_1 - x_2)$ известна в аналитическом виде. На практике эту функцию можно просто выбрать, и такой выбор будет в чём-то похож на выбор ширины окна при использовании метода окна Парзена.

Задача заключается в том, что, имея обучающую выборку $x_1, \dots, x_l$, для которой известны целевые переменные $\xi(x_1), \dots, \xi(x_l)$, требуется для новой выборки $x_{l+1}, \dots, x_s$ при сделанных предположениях найти условное распределение $p(\xi(x_{l+1}), \dots, \xi(x_s) \, \vert \, \xi(x_1), \dots, \xi(x_l))$.

Ничего сложного в такой задаче нет:
* искомая условная плотность записывается как отношение совместной плотности к плотности условия;
* эти две плотности, по определению гауссовского процесса, являются плотностями $s$-мерного и $l$-мерного гауссовских распределений;
* отношением двух гауссиан является гауссиана, параметры который находятся арифметической выкладкой.

Если проделать все выкладки, получится, что распределение $p(\xi(x_{l+1}), \dots, \xi(x_s) \, \vert \, \xi(x_1), \dots, \xi(x_l))$ является $(s - l)$-мерным гауссовским распределением с вектором среднего $S^TC^{-1}\overrightarrow{\xi}$ и ковариационной матрицей $Q - S^TC^{-1}S$, где:
* $C$ — матрица размера $l \times l$, элементы которой определены как $c_{ij} = K(x_i - x_j)$, то есть $C$ является ковариационной матрицей проекции гауссовского процесса на объекты обучающей выборки;
* $S$ — матрица размера $l\times(s-l)$, элементы которой определены как $s_{ij} = K(x_i - x_{l+j})$, то есть $S$ отражает ковариацию между объектами обучающей выборки и новыми объектами;
* $Q$ — матрица размера $(s-l)\times(s-l)$, которой определены как $q_{ij} = K(x_{l+i} - x_{l+j})$, то есть $Q$ является ковариационной матрицей проекции гауссовского процесса на новые объекты;
* $\overrightarrow{\xi}$ — вектор-столбец значений целевой переменной на объектах обучающей выборки.

Формулы для среднего и ковариационной матрицы условного гауссовского процесса при условии обучающей выборки называют формулами Андерсона.

#### Частные случаи

Если выбрана ковариационная функция $K(x_1 - x_2)$, являющаяся непрерывной в 0, то обусловленный гауссовский процесс, восстановленный по точкам обучающей выборки, будет иметь непрерывную функцию среднего, проходящую через точки обучающей выборки, а дисперсии случайных величин, соответствующих точкам обучающей выборки, будут нулевыми: $\mathrm{Var} \, \xi(x_i) = 0$ $\forall i \in \{1, \dots, l\}$. Это следует из формул Андерсона.

Если же $K(x_1 - x_2)$ непрерывна всюду кроме 0, где имеет устранимый разрыв (то есть имеет предел слева при стремлении к 0, имеет предел справа при стремлении к 0, и эти два предела равны), то обусловленный гауссовский процесс будет иметь разрывную функцию среднего, имеющую устранимые разрывы в точках обучающей выборки и проходящую через точки обучающей выборки благодаря этим разрывам. Любая функция $K(x_1 - x_2)$ с приведёнными выше свойствми является суммой некоторой непрерывной ковариационной функции и ковариационной функции белого шума, так что исходный гауссовский процесс можно рассматривать как гауссовский процесс с зашумлением. Таким образом, для обусловленной $m(x)$ пределы $\lim_{x \rightarrow x_i} m(x)$ могут пониматься как очищенные от шума значения целевой переменной в точках обучающей выборки.

#### Выбор ковариационной функции через принцип наибольшей обоснованности

Допустим, ковариационная функция ищется в каком-либо параметрическом семействе. Часто используют следующее семейство: $$K(x_1 - x_2) = Ae^{-\gamma\Vert x_1 - x_2\Vert^2} + \sigma[x_1 = x_2],$$
где запись $\Vert\cdot\Vert$ обозначает евклидово расстояние, а квадратные скобки обозначают нотацию Айверсона. Параметры этого семейства интерпретируются так:
* $A$ задаёт общий уровень ковариации;
* $\gamma$ задаёт степень затухания ковариации;
* $\sigma$ задаёт степень зашумлённости, ведь слагаемое с этим параметром является ковариационной функцией белого шума.

Возникает вопрос, как подобрать параметры ковариационной функции. Ответ на него может быть дан байесовским [принципом наибольшей обоснованности](__home_url__/notes/Принцип наибольшей обоснованности). Согласно этому принципу, следует выбрать ту тройку $(A, \gamma, \sigma)$, для которой максимальна обоснованность, то есть вероятность обучающей выборки. Поскольку исходный гауссовский процесс является стационарным с нулевым средним, обоснованность равна плотности в точке $(\xi(x_1), \dots, \xi(x_l))$ $l$-мерного нормального распределения с нулевым вектором среднего и ковариационной матрицей $C$. Решение соответствующей оптимизационной задачи даёт рекомендованные параметры ковариационной функции.

#### Непараметричность

Даже если ковариационная функция зафиксирована заранее или ищется в параметрическом семействе, регрессия на основе гауссовских процессов является непараметрическим методом.

Здесь предлагается посмотреть на непараметричность не в терминах того, что множество зависимостей, могущих быть выраженными алгоритмом обучения, всюду плотно во множестве всех зависимостей, а в терминах того, что сложность восстановленной зависимости определяется не только гиперпараметрами, но и размером выборки.

Возьмём, скажем, случайный лес, у которого зафиксировали количество деревьев на уровне $q$ и ограничили максимальную глубину каждого из деревьев уровнем $d$. Такой случайный лес заведомо не сможет выдавать более $2^{qd}$ различных предсказаний. Однако если обучать регрессию на основе гауссовских процессов, чем больше обучающая выборка, тем более сложным может быть обусловленный процесс. Например, если взять какую-то одну непрерывную ковариационную функцию и зафиксировать её, предсказания будут содержать не меньше уникальных значений, чем их было у целевой переменной в обучающей выборке (по той хотя бы причине, что на объектах обучающей выборки предсказывается сама их целевая переменная).

## Распределение Дирихле и случайные процессы Дирихле

#### Определение и свойства распределения Дирихле

Для $k \in \mathbb{N}$ рассмотрим множество $P$ векторов $\pi$ длины $k$, таких что $\sum_{i=1}^k \pi_i = 1$ и $\forall i: 1 \le i \le k,$ соблюдается неравенство $\pi_i \ge 0$. Любой такой вектор $\pi \in P$ может интерпретироваться как вектор вероятностей, которым параметризуется категориальное распределение с $k$ исходами.

На множестве $P$ и определено распределение Дирихле. Это распределение параметризуется вектором $\alpha$ длины $k$, таким что $\forall i: 1 \le i \le k,$ верно неравенство $\alpha_i > 0$. Обозначается распределение Дирихле как $\mathrm{Dir}_{\alpha}(\pi)$, и его плотность в точке $\pi$ имеет следующий вид:
$$\mathrm{Dir}_{\alpha}(\pi) = \frac{\Gamma(\sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma(\alpha_i)}\prod_{i=1}^k\pi_i^{\alpha_i - 1},$$
где $\Gamma$ — гамма-функция.

Первый множитель правой части является нормировочной константой, и поведение функции плотности распределения Дирихле определяется множителем $\prod_{i=1}^k\pi_i^{\alpha_i - 1}$. Из вида этого множителя вытекает, что если все $\alpha_i = 1$, то распределение Дирихле является равномерным распределением на множестве $P$, если все $\alpha_i > 1$, то распределение Дирихле имеет плотность, сконцентрированную около векторов $\pi$, таких что одна их компонента равна 1, а остальные — 0. Наконец, если все $\alpha_i < 1$, плотность распределения Дирихле имеет моду в векторе $\pi$, таком что в соответствующем ему категориальном распределении все исходы более-менее вероятны.

Из свойств распределения Дирихле можно отметить следующие:
* Агрегационное свойство: если $\pi \sim \mathrm{Dir}_{\alpha}(\pi)$, то $(k-1)$-мерный вектор $(\pi_1 + \pi_2, \pi_3, \dots, \pi_k)$ имеет распределение Дирихле, параметризованное $(k-1)$-мерным вектором $(\alpha_1 + \alpha_2, \alpha_3, \dots, \alpha_k)$:
$$(\pi_1 + \pi_2, \pi_3, \dots, \pi_k) \sim \mathrm{Dir}_{(\alpha_1 + \alpha_2, \alpha_3, \dots, \alpha_k)}((\pi_1 + \pi_2, \pi_3, \dots, \pi_k)).$$
Предельный случай этого свойства принимает вид:
$$(\pi_1, \sum_{i=2}^k \pi_i) \sim \mathrm{Beta}_{\alpha_1, \sum_{i=2}^k \alpha_i}(\pi_1) = \frac{1}{B(\alpha_1, \sum_{i=2}^k \alpha_i)}\pi_1^{\alpha_1 - 1}(1 - \pi_1)^{\sum_{i=2}^k \alpha_i - 1},$$
где $\mathrm{Beta}$ — бета-распределение (его плотность выписана после знака равенства), а $B$ — бета-функция. Отметим, что бета-распределение является двумерным аналогом распределения Дирихле.
* Свойство условной независимости: если $\pi \sim \mathrm{Dir}_{\alpha}(\pi)$, то его условное распределение при условии одной компоненты по-прежнему является распределением Дирихле:
$$p(\pi_2, \dots, \pi_k \, \vert \, \pi_1) = \frac{p(\pi)}{p(\pi_1)} = \frac{\mathrm{Dir}_{\alpha}(\pi)}{ \mathrm{Beta}_{\alpha_1, \sum_{i=2}^k \alpha_i}(\pi_1)} = \frac{1}{\mathrm{const}}\prod_{i=1}^k\left(\frac{\pi_i}{1 - \pi_1}\right)^{\alpha_i - 1} = \mathrm{Dir}_{(\alpha_2, \dots, \alpha_k)}\left((\frac{\pi_2}{1 - \pi_1}, \dots, \frac{\pi_k}{1 - \pi_1})\right).$$

#### Сэмплирование из распределения Дирихле

Чтобы сэмплировать векторы $\pi$ из распределения Дирихле с параметром $\alpha$, можно сгенерировать $k$ величин из гамма-распределения, а потом их отнормировать. Однако в контексте случайных процессов Дирихле более интересен другой способ, называющийся stick breaking.

Устроен этот способ так. Сначала сэмплируются $k$ величин $v_1, \dots, v_k$ из следующих бета-распределений:
$$v_1 \sim \mathrm{Beta}_{\alpha_1, \sum_{i=2}^k\alpha_i}(v_1),$$
$$v_2 \sim \mathrm{Beta}_{\alpha_2, \sum_{i=3}^k\alpha_i}(v_2),$$
$$\dots,$$
$$v_{k-1} \sim \mathrm{Beta}_{\alpha_{k-1}, \alpha_k}(v_{k-1}),$$
$$v_{k} = 1.$$
Далее $i$-я компонента сэмплируемого вектора $\pi$ находится по формуле:
$$\pi_i = \prod_{j=1}^{i-1}(1-v_j) v_i.$$

Корректность работы такого способа доказывается через агрегационное свойство распределения Дирихле. Интуитивно говоря, с каждым $v_i$ определяется, какая доля ещё нераспределённой вероятности достанется $i$-му исходу.

#### Определение и свойства случайного процесса Дирихле

Ниже даётся неконструктивное определение процесса Дирихле. Иными словами, это определение не позволяет построить сам процесс, из-за чего для такого определения доказана сложная теорема, гласящая, что случайный процесс с перечисленными свойствами существует и единственен.

Итак, случайный процесс Дирихле — это случайный процесс $\xi$, задаваемый следующими свойствами:
* индексирующим элементом континуального множества случайных величин является множество $A$, являющееся подмножеством некоторого множества $U$. Для любого $A \subset U$ определена случайная величина $\xi(A, \omega)$, задающая значение процесса на $A$ при исходе $\omega$ (далее зависимость от исхода иногда будет опускаться). Поскольку процесс индексируется подмножеством, а не одномерным действительным числом, интерпретируемым как время, иногда его называют случайным полем Дирихле;
* каким бы ни оказался зафиксированный исход $\overline{\omega}$, реализация процесса Дирихле при этом исходе будет вероятностной мерой на $U$. Иными словами, функция $A \mapsto \xi(A, \overline{\omega})$ является вероятностной мерой на $U$;
* случайный процесс Дирихле параметризуется следующими двумя параметрами:
    - $G$ — вероятностная мера на $U$, интерпретируемая как аналог среднего,
    - $\alpha > 0$ — неотрицательное число, контролирующее разреженность;
* одномерной проекцией процесса Дирихле на некоторое зафиксированное подмножество $\overline{A}$ является следующая случайная величина:
$$\xi(\overline{A}) \sim \mathrm{Beta}_{\alpha G(\overline{A}), \alpha(1 - G(\overline{A}))}(\xi(\overline{A})),$$
откуда видно, что $G$ интерпретируется как среднее, потому что математическое ожидание $\xi(\overline{A})$ равно $G(\overline{A})$;
* если взять $m$ подмножеств $A_1, \dots, A_m$, таких что $\bigcup_{i=1}^m A_i = U$ и $\forall i, j: 1 \le i < j \le m$, $A_i \cap A_j = \emptyset$, то $m$-мерной проекцией процесса Дирихле на $(A_1, \dots, A_m)$ будет $m$-мерная случайная величина из распределения Дирихле:
$$(\xi(A_1), \dots, \xi(A_m)) \sim \mathrm{Dir}_{(\alpha G(A_1), \dots, \alpha G(A_m))}((\xi(A_1), \dots, \xi(A_m))).$$

Из определения процесса Дирихле вытекает одно его важное свойство: мера $\xi(A)$, являющаяся некоторой реализацией, не может иметь непрерывной части. Доказывается это от противного. Если бы у этой меры была непрерывная часть, можно бы было подобрать два подмножества $A_1$ и $A_2$ так, чтобы знание $\xi(A_1)$ давало бы оценку на $\xi(A_2)$, а это противоречило бы свойству условной независимости распределения Дирихле. Оставив в стороне существование сингулярных мер, будем считать, что отсюда выводится то, что мера $\xi(A)$ является атомарной, то есть вся вероятность сосредоточена в некотором счётном множестве элементов $U$. Это свойство процесса Дирихле позволяет сэмплировать из него.

#### Сэмплирование из случайного процесса Дирихле

Сэмплирование из процесса Дирихле — получение различных реализаций процесса Дирихле при каких-либо исходах. Как было показано в предыдущем разделе, любую реализацию процесса Дирихле можно представить в виде счётного множества пар $\{(\theta_i, \pi_i)\}_{i=1}^{+\infty}$, где $\theta_i \in U$, а $\pi_i$ образуют вероятности, то есть $\sum_{i=1}^{+\infty}\pi_i = 1$ и $\pi_i > 0$. Поскольку на практике невозможно ждать бесконечно долго, критерием остановки порождения какой-либо реализации является получение множества пар такой длины, что сумма его $\pi_i$ отличается от 1 менее чем на пределы машинной точности.

Существует два наиболее известных способа сэмплировать из случайного процесса Дирихле:
* схема китайских ресторанов;
* stick breaking.

Схема китайских ресторанов устроена следующим образом:
* $\theta_1 \in U$ сэплируется из $G$, этому элементу приписывается счётчик $n_1$, инициализированный 1, а вероятность этого элемента $\pi_1$ будет вычисляться по формуле $\pi_1 = \frac{n_1}{n_1 + \alpha}$.
* Целочисленная переменная $k_*$ инициализируется 2; интерпретируется эта переменная как количество имеющихся на текущем этапе различных $\theta_i$, увеличенное на 1;
* Далее вплоть до достижения условия остановки вида $\frac{\alpha}{\sum_{i=1}^{k_* - 1} n_i + \alpha} < \varepsilon$, где $\varepsilon$ — порог точности, происходит следующее:
    - с вероятностью $\pi_i = \frac{n_i}{n_i + \alpha}$ может быть выбран один из ранее просэмплированных $\theta_i$, тогда его счётчик $n_i$ увеличивается на 1 (что повышает вероятность выбора этого элемента в дальнейшем);
    - с вероятностью $\frac{\alpha}{\sum_{i=1}^{k_* - 1} n_i + \alpha}$ может быть выбрано сэмплирование нового элемента, тогда $\theta_{k_*}$ сэмплируется из $G$, ему приписывается счётчик $n_{k_*}$, инициализированный 1, а сама переменная $k_*$ увеличивается на 1, чтобы обозначать уже потенциальный следующий элемент.
    
Схема stick breaking напоминает аналогичную схему для распределения Дирихле:
* счётчик $i$ инициализируется 1;
* пока $i = 1$ или не стало верным, что $\prod_{j=1}^{i-1}(1 - v_j) < \varepsilon$:
    - сэмплируется $v_i \sim \mathrm{Beta}_{1, \alpha}(v_i)$,
    - сэпмлируется $\theta_i \sim G$,
    - вычисляется соответствующая $\theta_i$ вероятность $\pi_i = \prod_{j=1}^{i-1}(1 - v_j) \, v_i$,
    - счётчик $i$ увеличивается на 1.

## Разделение смеси неопределённого количества распределений

#### Формализация задачи

Пусть имеется смесь неопределённого количества распределений, такая что $i$-е распределение параметризовано вектором $\theta_i \in \mathbb{R}^d$ и имеет вид $p(x \, \vert \, \theta_i)$, где зависимость плотности от $x$ и $\theta_i$ дана в аналитическом виде. Обозначим за $\theta$ бесконечную последовательность неизвестных параметров $\theta_i$, $\theta = \{\theta_i\}_{i=1}^{+\infty}$, а за $\pi$ обозначим бесконечную последовательность вероятностей соответствующих компонент смеси, $\pi = \{\pi_i\}_{i=1}^{+\infty}$. Если на самом деле количество компонент смеси конечное, можно считать, что с какого-то момента все $\pi_i$ становятся равны 0.

Чисто теоретически, задачу можно бы было поставить как максимизацию по $\theta$ и $\pi$ следующего правдоподобия:
$$p(X, Z \, \vert \, \theta, \pi) = \prod_{i=1}^{l}p(x_i \, \vert \, \theta_{z_i})p(z_i \, \vert \pi) = \prod_{i=1}^{l}p(x_i \, \vert \, \theta_{z_i})\pi_{z_i},$$
где $X$ — матрица наблюдений размером $l \times n$, $l$ — количество объектов, $n$ — количество признаков, $Z$ — вектор размера $l \times 1$ номеров компонент смеси, породивших соответствующие наблюдения, $x_i$ — признаковое описание $i$-го объекта, $z_i$ — номер компоненты, породившей $i$-й объект. На самом деле, поскольку вектор $Z$ не дан, речь идёт о максимизации не полного правдоподобия, а частичного правдоподобия (правдоподобия только $X$). Однако проблема не в этом. С такой постановкой проблема в том, что она допускает вырожденное решение: можно каждый объект поместить в свою собственную компоненту. Для борьбы с этим часто считают, что количество компонент смеси зафиксировано и что оно меньше количества объектов. Но что делать, если хочется иметь возможность варьировать число компонент смеси по мере поступления новых данных?

Чтобы избежать вырожденных решений, вводится своего рода регуляризция, а именно априорное распределение на пару $(\theta, \pi)$. Пара $(\theta, \pi)$ может быть проинтерпретирована как вероятностная мера на $\mathbb{R}^d$. Следовательно, на ум тут же приходит взять в качестве априорного распределения [процесс Дирихле](__home_url__/notes/Распределение Дирихле и случайные процессы Дирихле). Итак, пусть исходя из априорных знаний были подобраны мера $G$ на $\mathbb{R}^d$ и число $\alpha > 0$, которыми параметризуется процесс Дирихле $\xi_{G, \alpha}$. Будем считать, что именно из такого $\xi_{G, \alpha}$ сэмплируется пара $(\theta, \pi)$. Тогда может быть выписана уже не условная, а совместная плотность:
$$p(X, Z, \theta, \pi) = \prod_{i=1}^{l}\left(p(x_i \, \vert \, \theta_{z_i})\pi_{z_i}\right)p(\theta, \pi),$$
где $p(\theta, \pi)$ — вероятность реализации $(\theta, \pi)$ случайного процесса Дирихле $\xi_{G, \alpha}$. Как это видно из схемы сэмплирования из процесса Дирихле, называемой stick breaking, $p(\theta, \pi)$ факторизуется до $p(\theta)p(\pi)$, где $p(\theta) = \prod_{i=1}^{+\infty}G(\theta_i)$.

Задача заключается в том, что по выборке $X$ и выписанному аналитическому виду $p(X, Z, \theta, \pi)$ требуется найти апостериорное распределение $p(Z, \theta, \pi \, \vert \, X)$.

#### Сколлапсированное сэмплирование Гиббса

Как это часто и бывает в байесовских методах, поставленная задача не решается аналитически. Один из численных способов поиска приближённого решения опирается на марковские цепи Монте-Карло. А поскольку MCMC сходятся долго, хочется сократить количество переменных. Отсюда и возникает коллапсирование. По переменной $\pi$ проводится схлопывание, и выбрана $\pi$ просто по той причине, что интеграл возьмётся аналитически:
$$p(X, Z, \theta) = \int p(X, Z, \theta, \pi) d\pi = \int \prod_{i=1}^{l}\left(p(x_i \, \vert \, \theta_{z_i})p(z_i \, \vert \pi)\right) p(\theta)p(\pi) d\pi = \prod_{i=1}^{l}p(x_i \, \vert \theta_{z_i}) \; p(\theta) p(Z),$$
где плотности образующих смесь распределений $p(x_i \, \vert \theta_{z_i})$ были даны исходно, $p(\theta)$ определена выбором параметра процесса Дирихле $G$, а $p(Z) = \int \prod_{i=1}^{l}p(z_i \, \vert \pi)p(\pi)d\pi = \int \prod_{i=1}^{l}\pi_{z_i}p(\pi)d\pi$.

После коллапсирования сэмплирование Гиббса позволит генерировать пары $(Z, \theta)$ из распределения $p(Z, \theta \, \vert \, X)$. При этом сама конструкция сэмплирования Гиббса в этой задаче окажется похожей на схему китайских ресторанов.

Величина $(Z, \theta)$, которую можно рассмотреть как конкатенацию вектора $Z$ длины $l$ и бесконечной последовательности $\theta$, формально является бесконечномерной, так что даже один шаг сэмплирования Гиббса должен бы длиться бесконечно долго. Чтобы этого избежать, $\theta$ понимается как вектор переменной длины, такой что в него входят лишь параметры тех компонент, к которым приписан хотя бы один объект. Если на каком-либо шаге из компоненты перенезначаются в другие компоненты все её объекты, параметры этой компоненты удаляются из вектора $\theta$.

Итак, пусть схема Гиббса была инициализирована каким-то начальным значением $(Z, \theta)$. Тогда каждое следующее сэмплирование проводится в три этапа, описываемых ниже.

На первом этапе для каждого из $i$ от 1 до $l$ фиксируются все $z_j$, $j \ne i$, и весь конечномерный вектор $\theta$. Обозначим за $Z_{-i}$ вектор длины $l-1$, составленный всеми компонентами $Z$ кроме $i$-й. Нужно просэмплировать новое значение $z_i$ из одномерного распределения $p(z_i \, \vert \, Z_{-i}, \theta, X)$, известного с точностью до константы. Делается это так:
* $z_i := k$, где $k$ от 1 до текущей длины вектора $\theta$, с вероятностью, пропорциональной $n_k^{-i}p(x_i \, \vert \theta_k)$, где $n_k^{-i}$ — количество на текущий момент приписанных к $k$-й компоненте объектов без учёта $i$-го объекта (если он к ней приписан). На то, чему вероятность пропорциональна, можно посмотреть как на популярность компоненты, скорректированную на правдоподобие объекта относительно неё.
* $z_i := k_*$, где $k_*$ — увеличенная на 1 длина вектора $\theta$, с вероятностью, пропорциональной $\alpha\int p(x_i \, \vert \, \theta) G(\theta) d\theta$. Если выпадает этот исход, длина вектора $\theta$ увеличивается на 1, и в его конец дописывается компонента $\theta_{k_*}$, просэмплированная из распределения с плотностью $G(\theta_{k_*})p(x_i \, \vert \, \theta_{k_*})$, понимаемого как априорное распределение $G$, перевзвешенное на эмпирические данные.

На втором этапе для каждого из $i$ от 1 до текущей длины $\theta$ фиксируются все $\theta_j$, $j \ne i$, и весь вектор $Z$. Процедура же обновления каждого $\theta_i$ выглядит так: новое значение $\theta_i$ сэмплируется из распределения с плотностью $G(\theta_i) \prod_{j: z_j = i}p(x_j \, \vert \, \theta_i)$. 

На третьем этапе вектор $\theta$ модифицируется, а именно из него удаляются все его компоненты, такие что у соответствующих им распределений после первого этапа не осталось приписанных объектов, и оставшиеся компоненты перенумеровываются.

## Латентное размещение Дирихле

#### Введение

Латентное размещение Дирихле (Latent Dirichlet Allocation, LDA) является одним из методов тематического моделирования, опирающегося на подход "мешка слов". В рамках такого подхода порядок слов внутри документов не учитывается, а учитывается только то, сколько раз то или иное слово вошло в тот или иной документ.

LDA является довольно простым методом тематического моделирования, и это даёт ему высокую интерпретируемость, так что можно провести такую аналогию, что LDA для задачи тематического моделирования является примерно тем же, чем линейные модели для задачи регрессии.

#### Вероятностная модель

Здесь и далее рассматривается модель без априорного распределения на матрицу тем, а про вариант с априорным распределением будет написано ближе к концу. В рассматриваемом варианте есть следующие сущности:
* $W$ — наблюдаемый набор документов; у $W$ есть два индекса, но это не матрица, потому что её строки могут иметь разную длину; запись $W_{di}$ пусть обозначает идентификатор слова (далее для краткости будет писаться, что просто слово), стоящего в $d$-м документе на $i$-й позиции; $W_{di} \in \{1, \dots, \vert V \vert\}$, где $V$ — множество всех слов, а запись $\vert \cdot \vert$ применительно ко множеству обозначает количество элементов в нём. Таким образом, чтобы элемент $W_{di}$ существовал, необходимо и достаточно, чтобы индекс $d$ имел значение от 1 до $\vert D \vert$, где $D$ — множество всех документов, а индекс $i$ в зависимости от значения индекса $d$ имел значение от 1 до количества слов в $d$-м документе.
* $Z$ — ненаблюдаемая информация о темах, из которых порождались слова; у $Z$ есть два индекса, и множество допустимых значений этой пары индексов такое же, как у $W$; запись $Z_{di}$ пусть обозначает идентификатор темы (далее для краткости будет писаться, что просто тему), из которой в $d$-м документе было порождено слово, стоящее на $i$-й позиции; $Z_{di} \in \{1, \dots, \vert T \vert\}$, где $T$ — множество всех тем, размер которого $\vert T\vert$ является гиперпараметром и должен быть подобран.
* $\Theta$ — неизвестная матрица тематических профилей документов, её размеры $\vert D\vert \times \vert T \vert$; элемент этой матрицы $\Theta_{dt}$ показывает вероятности встретить тему $t$ в документе $d$; как следствие, эта матрица не может быть произвольной: на неё накладываются ограничения, что $\Theta_{dt} \ge 0$ и для любой $d$-й строки $\sum_{t=1}^{\vert T \vert}\Theta_{dt} = 1$.
* $\Phi$ — неизвестная матрица тем, её размеры $\vert T \vert \times \vert V\vert$; элемент этой матрицы $\Phi_{tw}$ показывает вероятность встретить слово $w$ в теме $t$; как следствие, эта матрица не может быть произвольной: на неё накладываются ограничения, что $\Phi_{tw} \ge 0$ и для любой $t$-й строки $\sum_{w=1}^{\vert V \vert}\Phi_{tw} = 1$.
* $\alpha$ — вектор длины $\vert T \vert$, такой что все его компоненты строго больше нуля; этот вектор является гиперпараметром и должен быть подобран.

Говоря верхнеуровнево, вероятностная модель для LDA устроена так:
$$p(W, Z, \Theta \, \vert \, \Phi, \alpha) = \prod_{d=1}^{\vert D \vert}\left(p(\Theta_{d\cdot} \, \vert \, \alpha) \prod_{i=1}^{n_d} \left(p(Z_{di} \, \vert \, \Theta_{d\cdot}) p(W_{di}\, \vert \, \Phi, Z_{di})\right) \right),$$
где $\Theta_{d\cdot}$ обозначает $d$-ю строку матрицы $\Theta$, а $n_{d}$ — количество слов в $d$-м документе. Выписанная формула концептуально интерпретируется так: документы порождаются независимо друг от друга, для каждого документа сначала порождается его тематический профиль, затем в документе поочерёдно порождаются слова, причём для каждого слова сначала порождается его тема, а уже потом из этой темы порождается само слово. Кстати, такую логику можно перерисовать в виде графической модели.

Чтобы специфицировать вероятностную модель для LDA, определим $p(\Theta_{d\cdot} \, \vert \, \alpha)$ как [распределение Дирихле](__home_url__/notes/Распределение Дирихле и случайные процессы Дирихле) с параметром $\alpha$. Также заметим, что остальные вероятности можно записать в аналитическом виде исходя из определений введённых сущностей:
$$p(Z_{di} \, \vert \, \Theta_{d\cdot}) = \Theta_{dZ_{di}},$$
$$p(W_{di}\, \vert \, \Phi, Z_{di}) = \Phi_{Z_{di}W_{di}}.$$
Теперь вероятностную модель для LDA можно записать в явном аналитическом виде:
$$p(W, Z, \Theta \, \vert \, \Phi, \alpha) = \prod_{d=1}^{\vert D \vert}\left(\mathrm{Dir}_{\alpha}(\Theta_{d\cdot}) \prod_{i=1}^{n_d}\left(\Theta_{dZ_{di}} \Phi_{Z_{di}W_{di}}\right)\right) = \prod_{d=1}^{\vert D \vert}\left(\frac{\Gamma(\sum_{t=1}^{\vert T \vert}\alpha_t)}{\prod_{t=1}^{\vert T \vert}\Gamma(\alpha_t)}\prod_{t=1}^{\vert T \vert}\Theta_{dt}^{\alpha_t - 1} \cdot \prod_{i=1}^{n_d} \prod_{t=1}^{\vert T \vert} \left(\Theta_{dt}^{[Z_{di} = t]} \Phi_{tW_{di}}^{[Z_{di} = t]}\right)\right),$$
где квадратные скобки соответствуют нотации Айверсона.

#### Обучение

При обучении LDA важен не столько сам гиперпараметр $\alpha$, сколько его длина, то есть количество тем. Если все компоненты $\alpha$ достаточно малы, то обучение пройдёт успешно. Поэтому будет считать, что гиперпараметр $\alpha$ сразу подобран удачно и оптимизировать его не нужно.

Обучение LDA сводится к тому, что, имея корпус текстов $W$ и какой-то вектор $\alpha$, необходимо найти оптимальную структуру тем, то есть матрицу $\Phi$, количество строк которой задано благодаря $\alpha$ и количество столбцов которой задано благодаря $W$:
$$p(W \, \vert \, \Phi, \alpha) \to \max_{\Phi}.$$
Это задача максимизации неполного правдоподобия, ведь правдоподобие ненаблюдаемых переменных $Z$ и $\Theta$ не учитывается. На самом деле, получится так, что оценки для этих переменных в качестве полезного побочного эффекта будут даны в ходе поиска решения.

Как решать поставленную задачу? Задачи максимизации неполного правдоподобия часто решают при помощи [EM-алгоритма](__home_url__/notes/Общий взгяд на EM-алгоритм). Здесь он тоже позволит получит решение, но с одним нюансом. Точный EM-алгоритм оказывается неприменим, потому что отсутствует сопряжённость по кортежу скрытых переменных, не относящися к параметрам (то есть по объединению $Z$ и $\Theta$). Из-за этого на Е-шаге задача статистического вывода не решается аналитически. Однако вариационный EM-алгоритм будет работать, потому что условная сопряжённость по кортежу скрытых переменных, не относящихся к параметрам, есть, и, значит, приближённые E-шаги можно делать при помощи mean-field approximation.

Убедимся в этом.

Кортеж наблюдаемых переменных состоит только из $W$, больше наблюдаемых переменных нет. Кортеж скрытых переменных состоит из $\Phi$, $Z$ и $\Theta$. Из них $\Phi$ относится к параметрам, а $Z$ и $\Theta$ нет. Значит, на E-шаге при текущем зафиксированном значении $\overline{\Phi}$ будет оцениваться апостериорное распределение $p(Z, \Theta \, \vert \, W, \overline{\Phi}, \alpha)$. В вариационном EM-алгоритме оценка ищется в виде $q(Z)q(\Theta)$, и искусственно вводимая таким выбором факторизация приводит к приближённости.

Теперь перейдём непосредственно к проверке условной сопряжённости по кортежу скрытых переменных, не относящихся к параметрам (в [заметке про mean-field approximation](__home_url__/notes/Вариационный байесовский вывод и mean-field approximation) она называлась условная $T$-сопряжённость, потому что в той заметке символ $T$ обозначал не множество тем, а кортеж скрытых переменных, не относящихся к параметрам). Применительно к LDA определение сводится к наличию следующих свойств:
* При зафиксированных значениях $\overline{\Phi}$ и $\overline{\Theta}$ условное априорное распределение $p(Z \, \vert \, \overline{\Theta}, \overline{\Phi}, \alpha)$ является сопряжённым к условному правдоподобию $p(W \, \vert \, Z, \overline{\Theta}, \overline{\Phi}, \alpha)$.
* При зафиксированных значениях $\overline{\Phi}$ и $\overline{Z}$ условное априорное распределение $p(\Theta \, \vert \, \overline{Z}, \overline{\Phi}, \alpha)$ является сопряжённым к условному правдоподобию $p(W \, \vert \, \Theta, \overline{Z}, \overline{\Phi}, \alpha)$.

Для примера проверим первое свойство, второе проверяется аналогично. Условное априорное распределение $p(Z \, \vert \, \overline{\Theta}, \overline{\Phi}, \alpha)$ имеет плотность, принадлежащую следующему классу функций от $Z$: $\prod_{d=1}^{\vert D \vert} \prod_{i=1}^{n_d} \prod_{t=1}^{\vert T \vert} \mathrm{const}^{[Z_{di} = \mathrm{const}]}$. После умножения этой плотности на условное правдоподобие $p(W \, \vert \, Z, \overline{\Theta}, \overline{\Phi}, \alpha) = \prod_{d=1}^{\vert D \vert} \prod_{i=1}^{n_d} \prod_{t=1}^{\vert T \vert} \left(\overline{\Phi}_{tW_{di}}^{[Z_{di} = t]}\right)$ произведение останется в том же самом классе функций от $Z$. Значит, сопряжённость имеется.

Тем самым было показано, что при применении вариационного EM-алгоритма к задаче обучения LDA никаких непреодолимых вычислительных препятствий не возникнет, и решение можно будет найти.

#### Модификации

В вероятностную модель для LDA можно включить априорное распределение $\Phi$. Для этого добавляют ещё один гиперпараметр: $\beta$, вектор длины $\vert V \vert$, которым параметризуется распределение Дирихле, из которого порождаются строки матрицы $\Phi$.

Вероятностная модель в таком варианте принимает вид:
$$p(W, \Phi, Z, \Theta \, \vert \, \alpha, \beta) = \left(\prod_{t=1}^{\vert T \vert} p(\Phi_{t\cdot} \, \vert \, \beta)\right)\prod_{d=1}^{\vert D \vert}\left(p(\Theta_{d\cdot} \, \vert \, \alpha) \prod_{i=1}^{n_d} \left(p(Z_{di} \, \vert \, \Theta_{d\cdot}) p(W_{di}\, \vert \, \Phi, Z_{di})\right) \right),$$
где $p(\Phi_{t\cdot} \, \vert \, \beta) = \mathrm{Dir}_{\beta}(\Phi_{t\cdot})$.

Отдельно стоит заметить, что обучать LDA можно не только вариационным EM-алгоритмом, но и при помощи сколлапсированного сэмплирования Гиббса.

## Об уместности A/B-тестирования

Рассмотрим задачу обучения с учителем, для которой есть несколько обученных моделей. Выбирая между ними одну для регулярного использования, можно принять решение, основываясь на тестировании этих моделей на отложенной выборке (так называемый оффлайн-тест). При таком подходе для каждой из моделей вычисляется некоторая [метрика качества](__home_url__/notes/Обзор некоторых метрик оценки качества моделей), а выбирается модель с наилучшей метрикой. Однако иногда исторических данных бывает недостаточно, и вместо вычислительного эксперимента необходимо провести A/B-тесты, то есть эксперименты, подразумевающие взаимодействие с реальной средой и сбор обратной связи от неё (так называемые онлайн-тесты).

Причины, по которым может потребоваться A/B-тестирование, таковы:
* Внедрение модели изменяет среду, и поэтому на исторических данных можно измерить только качество модели на объектах, пришедших не из того распределения, из которого они будут приходить к внедрённой и какое-то время проработавшей модели;
* Модели получены при разных постановках задачи машинного обучения, из-за чего значения одной и той же метрики нельзя сравнивать друг с другом (разные постановки могут возникнуть, в частности, потому, что бывает не до конца понятно, что именно относить к положительному классу, так что используется несколько разметок);
* Целевой показатель отсутствует в исторических данных, и поэтому метрикой качества служит лишь приближение к нему (это можно сравнить с тем, что иногда метрику качества нельзя использовать в качестве функции потерь, и поэтому функцией потерь служит приближение метрики качества).

Проиллюстрируем последнюю ситуацию на примере. Пусть целью является увеличить продажи автомобилей определённой модели при помощи показа интернет-рекламы этой модели. По историческим данным можно узнать, кто кликал на рекламу этой модели, а кто нет. Однако наличие клика не означает, что человек готов приоберсти автомобиль и обладает достаточными для этого сбережениями. Нельзя исключать, что из двух сравниваемых классификаторов, предсказывающих вероятность клика, один покажет лучший результат при оффлайн-тесте за счёт того, что точнее будет выявлять тех, кто кликает на рекламу просто так, но при этом будет хуже выявлять тех, кто кликает на рекламу, держа в уме возможность покупки рекламируемого автомобиля.

С другой стороны, нельзя забывать и о недостатках A/B-тестирования:
* Проведение реального эксперимента иногда бывает затратным по ресурсам или по времени ожидания результатов;
* Если две группы (группа контроля и группа воздействия) подобраны с различиями, результаты теста могут оказаться смещёнными;
* Даже если две группы на момент начала теста были равнозначны, остаётся риск того, что во время теста какое-либо внешнее воздействие сделает их неравнозначными и сместит результаты теста;
* Разбиение на группы иногда может включать некоторый субъективный выбор, из-за чего становится сложно доказать кому-либо объективность результатов такого теста.

Стало быть, применять A/B-тесты стоит только тогда, когда это действительно необходимо.

## Обзор некоторых метрик оценки качества моделей

#### Коэффициент детерминации

Коэффициент детерминации $R^2$ тесно связан со среднеквадратичной ошибкой (MSE): $R^2 = 1 - \frac{\mathrm{MSE}}{\mathrm{Var}[y]}$, где запись $\mathrm{Var}[y]$ обозначает дисперсию целевой переменной. Из выписанной формулы вытекает, что с точки зрения оптимизации $R^2$ и MSE эквивалентны друг другу.

Основное преимущество, благодаря которому смотреть на $R^2$ удобнее, чем на MSE, заключается в том, что он проще в трактовке результатов. Не зная данных, сложно сказать, насколько хорошим или плохим является среднеквадратичная ошибка, равная, скажем, 12. А коэффициент детерминации не может быть больше 1, поэтому можно заранее сказать, что значения, близкие к 1, хороши. Второй опорной точкой является 0: если $R^2 < 0$, то это означает, что предсказания модели отклоняются от фактических значений сильнее, чем фактические значения отклоняются от своего среднего по выборке.

По сути, $R^2$ говорит об отмасштабированной среднеквадратичной ошибке. Можно бы было смотреть на метрику $\frac{\mathrm{MSE}}{\mathrm{Var}[y]}$. Вычитание из единицы этого отношения имеет исторические причины, восходящие к тому, что $R^2$ стал популярен благодаря эконометрике. Эконометристы, как правило, не проводят тестирование на отложенной выборке, потому что им неинтересно предсказание на новых объектах, а интересно объяснение зависимостей в существующей выборке. Если обучать методом наименьших квадратов линейную регрессию со включённой константой, то $R^2$ на обучающей выборке заведомо будет в диапазоне от 0 до 1, и в эконометрическом контексте можно считать, что $R^2$ показывает, какую долю дисперсии целевой переменной удалось отразить в модели.

Наконец, стоит отметить, что с трактовкой значений $R^2$ стоит быть аккуратнее. Например, пусть выборка разнородна и в ней есть объекты двух типов, таких что у каждого типа значения целевой переменной лежат в своём обособленном диапазоне. Тогда $R^2$, подсчитанный на всей выборке, будет выше, чем среднее двух коэффициентов детерминации, подсчитанных каждый только на объектах одного типа. Дело в том, что поменяется знаменатель, ведь дисперсия целевой переменной внутри каждого из двух диапазонов ниже, чем дисперсия целевой переменной, подсчитанная на всех объектах. Подобный эффект может привести к тому, что даже у слабых моделей $R^2$, вычисленный на всей выборке, будет близок к 1. В этом случае высокие значения $R^2$ говорят не о том, что MSE мала, а о том, что MSE мала относительно высокой дисперсии, значительный вклад в которую вносит разнородность выборки.

#### ROC AUC

Название этой метрики в полном виде звучит как площадь под ROC-кривой (area under curve, AUC). Как следует из названия, эта метрика может быть определена через ROC-кривую (receiver operating characteristic, ROC). Однако есть и альтернативная интерпретация данной метрики.

Звучит эта интерпретация так. Пусть имеется модель бинарной классификации, способная ранжировать объекты по степени своей уверенности в их принадлежности к положительному классу (грубо говоря, [предсказывающая вероятности](__home_url__/notes/Калибровка предсказанных вероятностей)). Тогда ROC AUC показывает вероятность того, что при равновероятном выборе одного объекта из всех объектов нулевого класса, представленных в выборке, и равновероятном выборе одного объекта из всех объектов положительного класса, представленных в выборке, объект положительного класса будет отранжирован моделью выше, чем объект отрицательного класса.

Из данного выше определения следует, что ROC AUC не зависит от баланса классов. Детализация этого утверждения такова: при зафиксированной  модели ни сэмплирование новых объектов одного из классов из того же распределения, из какого они были порождены, ни удаление объектов одного из классов с вероятностями, сохраняющими распределение объектов в этом классе, не поменяют математическое ожидание ROC AUC.

Также в отличие от полноты ROC AUC нельзя сделать равной 1, просто приписывая все объекты к положительному классу, а в отличие от точности ROC AUC нельзя сделать равной 1, почти всё кроме очевидно положительных объектов приписывая к нулевому классу.

Однако ROC AUC не всегда является уместной метрикой для оценки качества бинарной классификации. Дело в том, что ROC AUC зависит от ранжирования всей без исключения выборки. Если же стоит задача, имея $n$ объектов, выбрать из них $k$, где $k < n$, так, чтобы среди выбранных было как можно больше объектов положительного класса, то метрика precision@k (возможно, подсчитанная в разбивке на какие-то группы из $n$ объектов и затем усреднённая) более уместна. Все улучшения или ухудшения в ранжировании ниже $k$-й позиции не изменят полезность модели для поставленной задачи, но на ROC AUC в отличие от precision@k они скажутся.

Другим следствием того, что ROC AUC зависит от ранжирования всей выборки (а не лишь интересующей её части), является то, что порой конкретные значения ROC AUC бывают обманчивы. Например, пусть дана выборка, где объекты нулевого класса разнородны: часть из них модель всегда ранжирует ниже всех объектов положительного класса, а часть не отличает от объектов положительного класса (т.е. ранжирует вперемежку с ними). Чем выше доля той части, которую модель ранжирует максимально низко, тем выше ROC AUC. Иными словами, ROC AUC можно сколь угодно приближать к 1, включая в выборку объекты, для которых даже без машинного обучения очевиден их класс. И это не противоречит тому, что ROC AUC не зависит от баланса классов: описанное изменение ROC AUC вызывается не изменением баланса классов, а изменением структуры класса, куда добавляют очевидные объекты.

## Регрессионные значения Шэпли и SHAP-значения

#### Постановка задачи

Рассмотрим задачу объяснения предсказания обученной модели на каком-то конкретном объекте. Допустим, хочется получить результат в следующем виде:
$$\hat{y} = \phi_0 + \sum_{i=1}^n \phi_i,$$
где:
* $\hat{y}$ — предсказание модели на выбранном объекте (для краткости индекс объекта здесь и далее опущен),
* $\phi_0$ — некоторое базовое предсказание, не зависящее от признаков (например, для регрессии, обучаемой с MSE в качестве функции потерь, это может быть среднее значение целевой переменной на обучающей выборке, а для классификации, обучаемой с логарифмической функцией потерь, это может быть вектор долей классов в обучающей выборке),
* $\phi_i$ — вклад $i$-го признака в предсказание модели на рассматриваемом объекте, этот вклад требуется каким-то образом найти,
* $n$ — количество признаков, используемых моделью.

#### Регрессионные значения Шэпли

Для сформулированной выше задачи можно в качестве стартовой точки предложить следующую попытку решения. Обучим модель, во всём аналогичную исходной модели кроме того, что $i$-й признак не использовался, и положим $\phi_i = \hat{y} - \hat{y}_{\backslash i}$, где $\hat{y}_{\backslash i}$ — предсказание модели, обученной на всех признаках кроме $i$-го. Однако у этого способа есть существенные недостатки. Во-первых, нет никаких гарантий, что будет соблюдаться желаемое условие $\hat{y} = \phi_0 + \sum_{i=1}^n \phi_i$. Во-вторых, интерпретация полученных величин вызывает вопросы: например, сильный признак, такой что информация, содержащаяся в нём, косвенно отражена и в других признаках, может получить маленький по модулю $\phi_i$, но, казалось бы, модуль $\phi_i$ должен быть большим, раз признак сильный.

Разовьём предыдущую попытку путём обобщения. Расставим все признаки в каком-то произвольном порядке и обозначим за $P$ множество всех признаков, которые при таком упорядочивании оказались перед $i$-м признаком. Тогда решение определим так: $\phi_i = \hat{y}_{P \cup \{i\}} - \hat{y}_P$, где $\hat{y}_P$ — предсказание модели, обученной на всех признаках, принадлежащих $P$ (если $P = \emptyset$, то $\hat{y}_P = \phi_0$), а $\hat{y}_{P \cup \{i\}}$ — предсказание модели, обученной на $i$-м признаке и на всех признаках, принадлежащих $P$. Отметим три момента:
* если упорядочивание признаков зафиксировать для всех $i$, то условие $\hat{y} = \phi_0 + \sum_{i=1}^n \phi_i$ будет соблюдаться;
* для признака, оказавшегося последним, получится то же самое, что получилось бы при первом способе;
* недостатком является то, что результат зависит от упорядочивания признаков, а оно, как отмечалось выше, берётся произвольно.

Устраним имеющийся недостаток за счёт усреднения по всем возможным упорядочиваниям множества из $n$ признаков. Обозначим за $N_{\backslash i}$ множество всех подмножеств множества исходных признаков, таких что $i$-й признак в них не входит. Тогда определим искомые вклады признаков так:
$$\phi_i = \sum_{S \in N_{\backslash i}} \frac{(\vert S \vert)! (n - \vert S \vert - 1)!}{n!} (\hat{y}_{S \cup \{i\}} - \hat{y}_S),$$
где множитель перед разностью показывает для каждого конкретного подмножества $S$, какую долю из всех $n!$ упорядочиваний $n$ признаков составляют такие, где на первых $\vert S \vert$ позициях стоят признаки из $S$, а на $(\vert S \vert + 1)$-й позиции стоит $i$-й признак.

В [литературе](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions.pdf) доказывается, что предложенный способ определить вклады признаков в предсказание на конкретном объекте является единственным способом, удовлятворяющим ряду естественных условий. При этом впервые это было доказано не в контексте машинного обучения, а в контексте теории игр (грубо говоря, решалась [задача](https://en.wikipedia.org/wiki/Shapley_value) о том, как разделить награду за результат между нескольким участникам одной команды), и по имени экономиста, доказавшего это, такие коэффициенты $\phi_i$ называют регрессионными значениями Шэпли.

#### SHAP-значения

Теоретически регрессионные значения Шэпли решают поставленную задачу, но на практике их вычисление может оказаться вычислительно неподъёмным. Модель потребуется обучать $O(n!)$ раз, что при хоть сколько-нибудь больших $n$ затратно.

Для получения результата за разумное время используют приближения к регрессионным значениям Шэпли. Тут можно применить несколько приёмов:
* Оценить сумму из формулы для $\phi_i$ лишь по части слагаемых.
* Заменить многократное обучение модели на усреднение по различным вариантам исключённых признаков. Делается это так. Берётся только исходная модель (обученная на всех признаках). Её предсказание в случае исключения каких-либо признаков определяется как среднее по обучающей выборке её предсказаний на синтетических объектах, оставленные признаки которых равны оставленным признакам того исходного объекта, для которого ищется вклад признаков, а исключённые признаки равны исключённым признакам какого-либо объекта из обучающей выборки. Преимуществом этого приёма является то, что устраняется влияние случайности на обучение разных моделей.
* В предыдущем приёме можно усреднять не по всей обучающей выборке, а лишь по её случайному подмножеству.
* Если речь идёт о некоторой конкретной модели (например, об ансамбле деревьев или нейронной сети), можно использовать и какие-то специфичные для этой модели техники (наподобие DeepLIFT).

Инструмент, объединяющий различные способы приближённо вычислить регрессионные значения Шэпли, назван [SHAP](https://github.com/slundberg/shap). Возвращаемые им вклады признаков в предсказание модели на некотором объекте называют SHAP-значениями.

## Прунинг решающих деревьев

Обучению решающего дерева присущи следующие недостатки:
* Чем дальше какой-либо узел от корня, тем меньше количество данных, на которых подбирается критерий разбиения в этом узле.
* Структура обучаемого дерева нестабильна. Малое изменение обучающей выборки может привести к построению сильно отличающегося дерева (например, потому что могут измениться признак и его пороговое значение, по которым происходит какое-то более раннее разбиение).

Две этих особенности приводят к тому, что решающее дерево, обученное каким-либо жадным алгоритмом наподобие CART, скорее всего, окажется переобученным. Чтобы сделать дерево менее переобученным, его подрезают, выкидывая часть узлов большой глубины. Соответствующие техники называют прунингом.

Существует два основных вида прунинга:
* reduced error pruning;
* cost-complexity pruning.

Остановимся на них подробнее.

Reduced error pruning так же, как и, скажем, ранняя остановка, требует наличия дополнительной валидационной выборки. Для каждой пары листьев, имеющих общего прямого родителя, на валидационной выборке оценивается качество дерева, где этих листьев нет (и, стало быть, зато листом является их родитель). Если качество на валидационной выборке не ниже, эти листья убираются. Процесс продолжается до тех пор, пока больше не останется соседних листьев, которые было бы можно объединить.

Reduced error pruning считается методом, слишком сильно обрезающим дерево. Так, в частности, если в какой-либо узел не придёт ни одного объекта из валидационной выборки, этот узёл и всё, что идёт после него, будут удалены.

А вот в cost-complexity pruning есть специальный гиперпараметр $\alpha$, регулирующий агрессивность обрезки дерева.

Задача для этого прунинга ставится в виде минимизации регуляризованной ошибки на обучающем множестве, а именно:
$$E_{\mathrm{train}}(T) + \alpha \, s(T)\to \min_T,$$
где $T$ — какой-то обрезанный вариант исходного дерева, $E_{\mathrm{train}}(T)$ — значение функции потерь на тренировочном множестве для дерева $T$, $s(T)$ — количество листьев в дереве $T$. Таким образом, сложностью дерева считается количество листьев в нём.

Решение поставленной задачи можно найти итеративно. На $i$-м шаге будем удалять из дерева всё, что идёт после какого-либо одного узла $t$. Этот узел ищется перебором по следующему критерию:
$$\frac{E_{\mathrm{train}}(T_{i-1}^{(t)}) - E_{\mathrm{train}}(T_{i-1})}{s(T_{i-1}) - s(T_{i-1}^{(t)})} \to \min_t,$$
где $T_{i-1}$ — дерево, оставшееся после $(i-1)$-го шага (в этой нотации $T_0$ — исходное дерево), а $T_{i-1}^{(t)}$ — дерево, получающееся из него удалением всего, что идёт после узла $t$. Обозначим за $\alpha_i$ то, чему равна левая часть формулы выше при оптимальном узле $t$. Итеративный процесс заканчивается, как только $\alpha_i$ оказывается больше, чем $\alpha$ (или как только дерево обрезается под корень). Критерий остановки взят таким, потому что для него продолжение процесса означает, что новое дерево не хуже старого с точки зрения минимизируемой функции. 

При этом рассуждением от противного можно показать, что ряд $\alpha_i$ является неубывающим. Действительно, если бы для какого-то $i$ было бы верно, что $\alpha_i > \alpha_{i+1}$, то это означало бы, что на $i$-м шаге дерево было обрезано неоптимально, ведь вариант обрезки по узлу из $(i+1)$-го шага лучше.

Говоря верхнеуровнево, описанный итеративный процесс решения минимизационной задачи выглядит так: сначала совершается обрубание "ветки", максимально улучшающее минимизируемую величину, затем совершается лучшее обрубание "ветки" для оставшейся части дерева, и так до тех пор, пока обрубание не перестанет приводить к улучшениям.

## Обзор подходов к задаче выдачи рекомендаций

#### Введение

Задача выдачи рекомендаций в общем случае подразумевает, что есть:
* пользователи, о которых что-то может быть известно или нет;
* объекты, свойства которых могут быть известны или нет (далее эти объекты будут называться предметами по аналогии с англоязычным «items», хотя они и не обязаны быть материальными);
* (опционально) история взаимодействия пользователей с предметами (взаимодействием могут быть покупки, выставление оценок, просмотры и т.д.).

В задаче выдачи рекомендаций требуется сопоставить пользователей и предметы. Чаще всего это конкретизируется до того, что нужно отобрать и/или отранжировать несколько предметов для текущего пользователя так, чтобы эти предметы были оптимальны с точки зрения некоторого критерия.

Рекомендательные системы нужны, чтобы автоматизировать или хотя бы облегчить пользовательский выбор, а также чтобы направить его в нужное русло. Выбирать самостоятельно пользователи могут не желать по целому ряду причин:
* это может быть скучным или отвлекающим;
* качественный выбор может требовать анализа слишком большого количества информации;
* качественный выбор может требовать вовремя вспомнить что-то, что часто забывается.

Хорошие рекомендательные системы не просто подбирают наиболее подходящие варианты, но и имеют дополнительные эффекты в виде открытия для пользователей новых предметов, о которых те сами навряд ли бы узнали.

Ниже каждый из разделов посвящён какому-либо одному подходу к решению задачи выдачи рекомендаций.

#### Простые подходы, не требующие никакой информации о пользователях

Допустим, ведётся подсчёт некоторого показателя, характеризующего популярность или приоритетность каждого из предметов.

Для примера возьмём в качестве предметов товары в интернет-магазине. Тогда вышеописанным показателем могут быть:
* конверсия из просмотра информации о товаре в его покупку или в его откладывание в корзину;
* произведение какой-либо конверсии на маржу (наценку магазина);
* средние за день продажи (в штуках или денежном выражении), подсчитанные по $D$ последним дням или по всем дням, когда товар был в наличии;
* средняя оценка пользователей, купивших данный товар.

Что именно брать в качестве такого показателя, зависит от особенностей задачи и целей внедрения рекомендательной системы. Скорее всего, знание предметной области позволит включить в формулу для показателя и дополнительные факторы. Например, если разрабатывается система, посетителю новостного сайта рекомендующая новости, необходимо учитывать время с момента публикации, чтобы не рекомендовать устаревшие материалы.

Теперь перейдём к тому, как можно давать рекомендации.

Наивное решение заключается в том, что предметы можно просто отранжировать по значениям их показателя. Главный недостаток этого решения — риск выдачи некачественных рекомендаций из-за нехватки данных по части предметов. Например, предположим, что рекомендации даются на основании пользовательских оценок и один товар получил среднюю оценку 4,9 по тысяче отзывам, а другой товар получил всего один отзыв с оценкой 5. Формально говоря, второй товар имеет более высокий показатель и, значит, рекомендован должен быть он, однако ясно, что надёжнее было бы рекомендовать первый.

Для решения обозначенной проблемы можно посмотреть на задачу как на задачу о «многоруком бандите» и применить соответствующий [аппарат](__home_url__/notes/Баланс между изучением и применением в задаче о многоруком бандите).

К плюсам рассматриваемого подхода (неважно, наивного или модифицированного) относятся:
* простота и прозрачность;
* отсутствие широких требований к доступности разнообразных данных.

К минусам же относятся:
* неперсонализированность (все пользователи видят одно и то же);
* тривиальность рекомендаций (возможно, о том, что рекомендуется, пользователи и так знают).

#### Подход со сведением задачи к задаче обучения с учителем

Допустим, данные есть как минимум о взаимодействии пользователей с предметами. Тогда рекомендательную систему можно построить на базе модели, обученной с учителем.

Объектами (в терминологии машинного обучения) будут ситуации, в которых некоторый пользователь провзаимодействовал с некоторым предметом. Целевая переменная может быть вещественной или категориальной (в частности, бинарной). Как следствие, задача может быть сформулирована как задача регресии или как задача классификации. Однако и в том, и в другом случае задачу можно переформулировать и как задачу ранжирования. Даже если задача не была поставлена как задача ранжирования, ранжирование можно провести на базе предсказаний, возвращаемых моделью. Например, если задача ставилась как задача бинарной классификации с целевой переменной, кодирующей исходы «покупки не будет» и «покупка будет», то ранжирование происходит по предсказанной вероятности положительного класса. Впрочем, если задача исходно ставилась как задача ранжирования, то тогда будет оптимизироваться функция потерь, учитывающая больше аспектов ранжирования.

Действительно, в примере из предыдущего абзаца для задачи классификации логистическая функция потерь чувствительна к тому, отличает ли модель тех, кто более склонен покупать всё подряд, от тех, кто в принципе не склонен к покупкам. Однако для задачи ранжирования это не важно, и поэтому функции потерь наподобие YetiRank или StochasticRank, аппроксимирующие метрики ранжирования PFound или NDCG, сфокусированы лишь на том, как расставлены предметы отдельно для каждого из пользователей.

Если же пользователи оценивают предметы, скажем, по пятибалльной шкале, то с этой оценкой в качестве целевой переменной задача может быть задачей регрессии. Но если для одного и того же пользователя для всех предметов ошибиться на одну и ту же константу, то ранжирование предметов не изменится, а, значит, и штрафовать за такую ошибку не надо. Это соображение отражено в функциях потерь для задачи ранжирования, таких как QueryRMSE.

Среди признаков, возникающих при сведении выдачи рекомендаций к обучению с учителем, можно выделить следующие:
* признаки, описывающие пользователя:
    - социально-демографические характеристики (пол, возраст и т.д.);
    - информация из специализированных форм и анкет (интересы, список подписок и т.д.);
* признаки, описывающие предмет:
    - параметры предмета из его табличного описания;
    - признаки, извлечённые из текстового описания предмета;
    - векторные вложения изображений с предметом;
* признаки, описывающие взаимодействие пользователей с предметами:
    - векторные вложения предметов, полученные из истории транзакций при помощи аналогов word2vec;
    - признаки, описывающие текущую корзину пользователя и соответствие предмета ей;
    - признаки, описывающие прошлую историю действий пользователя (средний чек, количество предыдущих покупок того же самого предмета и т.д.);
    - признаки, являющиеся рангами или оценками, выдаваемыми какой-либо другой рекомендательной системой (например, рекомендательной системой на базе коллаборативной фильтрации или на базе матричных разложений);
* прочие признаки:
    - время суток, день недели, время года и т.д.;
    - погода;
    - курсы валют.

К сильным сторонам данного подхода относятся:
* разносторонность (т.е. способность учитывать данные разной природы);
* универсальность (в принципе, при помощи данного подхода можно строить мета-модели, дающие рекомендации по оценкам от других рекомендательных систем);
* поддержка работы с пользователями или предметами без собранной истории по ним.

К слабым сторонам данного подхода можно отнести то, что:
* при обучении для вычисления целевой переменной обязательно требуются данные о взаимодействии пользователей с предметами;
* иногда детализация постановки задачи машинного обучения может требовать высокой квалификации от того, кто этим занимается.

#### Коллаборативная фильтрация

Начнём с расшифровки понятия. Прилагательное «коллаборативная» в данном контексте указывает на то, что рассматриваются только схожие ситуации, а, например, сами свойства предметов или характеристики пользователей игнорируются.

Одним из форматов данных, поддерживаемых коллаборативной фильтрацией, является кросс-табуляционная матрица пользователей и предметов. Пусть $U$ — множество пользователей, данные по действиям которых хранятся, а $I$ — множество предметов, данные по действиям с которыми хранятся. Тогда речь идёт о матрице $R$ размера $\vert U \vert \times \vert I \vert$, на пересечении $i$-й строки и $j$-го столбца которой стоит число $R_{ij}$, прямо или косвенно показывающее степень соответствия $j$-го предмета $i$-му пользователю. При этом важно отметить, что матрица $R$ может содержать пропущенные значения. На практике она оказывается сильно разреженной, но это и придаёт смысл задаче, потому что если бы она была целиком известна, никакая рекомендательная система не была бы нужна.

В матрице $R$ могут храниться оценки предметов, проставляемые пользователями. Тогда матрица $R$ содержит информацию, прямо указывающую на степени соответствия предметов пользователям. Но может быть и так, что в матрице $R$ хранится количество покупок пользователем какого-либо предмета или бинарный индикатор того, что пользователь хотя бы раз покупал предмет. Тогда информация из матрицы $R$ лишь косвенно указывает на степени соответствия предметов пользователям, а именно делает это лишь в той мере, в какой можно считать, что всё, что пользователь приобретает, ему нравится.

Коллаборативная фильтрация бывает двух типов:
* «пользователь-пользователь» (она же коллаборативная фильтрация от пользователя, user-based collaborative filtering);
* «предмет-предмет» (она же коллаборативная фильтрация от предмета, item-based collaborative filtering).

Оба типа сводятся к тому, что пропущенные значения в матрице $R$ заполняются, а потом рекомендуется то, что имеет наибольшее оценённое значение. Разница только в том, как это делается.

В варианте «пользователь-пользователь» формула для заполнения матрицы $R$ имеет вид:
$$\hat{R}_{ij} = \frac{\sum_{i' \in U_j} w_{ii'} R_{i'j}}{\sum_{i' \in U_j} |w_{ii'}|},$$
где $U_j$ — множество всех пользователей $i'$, для которых известно $R_{i'j}$, а веса $w_{ii'}$ показывают степень схожести $i$-го пользователя и $i'$-го пользователя. Эти веса могут быть как положительными, так и отрицательными, причём большие по модулю отрицательные значения говорят о том, что пользователи противоположны друг другу по своим предпочтениям и что поэтому действия одного из них всё равно проливают свет на предпочтения другого. Один из вариантов, как можно определить веса, таков:
$$w_{ii'} = \frac{(1 / |I_{ii'}|) \sum_{j \in I_{ii'}} (R_{ij} - \overline{R}_{i\cdot})(R_{i'j} - \overline{R}_{i'\cdot})}{\sqrt{(1 / |I_{i}|) \sum_{j \in I_i} (R_{ij} - \overline{R}_{i\cdot})^2} \sqrt{(1 / |I_{i'}|) \sum_{j \in I_i'} (R_{i'j} - \overline{R}_{i'\cdot}})^2},$$
где $I_i$ — множество предметов, для которых есть значение в $i$-й строке матрицы $R$, $I_{ii'}$ — множество предметов, для которых есть значения и в $i$-й, и в $i'$-й строках матрицы $R$, а $\overline{R}_{i\cdot}$ — среднее (разумеется, без учёта пропусков) $i$-й строки матрицы $R$. По сути, так определённый вес $w_{ii'}$ — это коэффициент корреляции Пирсона между $i$-й и $i'$-й строками матрицы $R$, модифицированный для случая, когда часть значений пропущена.

На практике формулу для $w_{ii'}$ немного меняют. Во-первых, можно положить $w_{ii'} = 0$, если множество $I_{ii'}$ содержит слишком мало элементов. Тогда из формирования рекомендаций для $i$-го пользователя будут исключены те пользователи, о схожести или несхожести которых с $i$-м недостаточно информации. Во-вторых, для увеличения скорости работы веса $w_{ii'}$ можно предрассчитать заранее, а потом для каждого $i$ оставить только сколько-то из них с наибольшими модулями, положив остальные равными нулю.

Ещё один момент, имеющий значение на практике, — это предобработка данных. Допустим, изначально в матрице $R$ хранятся оценки, проставляемые пользователями, однако эти оценки неоткалиброваны: скажем, кто-то ставит 8 баллов из 10, если предмет более-менее понравился, а кто-то в таком случае ставит лишь 5 баллов из 10. Чтобы подход коллаборативной фильтрации заработал, подобную матрицу $R$ нужно преобразовать, из каждой строки вычтя её среднее. Новые значения, и впрямь, будут показывать уже именно то, понравился ли предмет пользователю или нет, и не будут содержать субъективной интерпретации шкалы.

Вариант же «предмет-предмет» во всём похож на вариант «пользователь-пользователь» кроме того, что строки и столбцы меняются местами. С точки зрения интерпретации это означает, что если в варианте «пользователь-пользователь» рекомендовались предметы, которые предпочитают похожие пользователи, то в варианте «предмет-предмет» рекомендуются предметы, похожие на те, что предпочитаются пользователем.

Если говорить о сравнении двух вариантов («пользователь-пользователь» и «предмет-предмет»), то строгих доказательств того, что какой-то один имеет более высокое качество, нет. В некоторых [экспериментах](http://www.diva-portal.org/smash/get/diva2:1111865/FULLTEXT01.pdf) вариант «пользователь-пользователь» победил.

Впрочем, помимо качества важно ещё и быстродействие на стадии выдачи рекомендаций. Тут, введя дополнительные предположения, можно получить строгие результаты. Например, если предположить, что пользователей сильно больше, чем предметов, а ещё предположить, что каждый пользователь провзаимодействовал лишь с малым количеством предметов, то для варианта «предмет-предмет» вполне можно заранее предрассчитать матрицу схожести между предметами. Если даже появится какое-либо новое взаимодействие, из-за большой длины столбцов матрицы $R$ оно не сильно изменит результаты. Однако для варианта «пользователь-пользователь» может потребоваться каждый раз заново рассчитывать матрицу схожести между пользователями, потому что любое новое взаимодействие способно сильно изменить веса для пользователя, совершившего это взаимодействие.
 
#### Матричные разложения

Методы выдачи рекомендаций с использованием матричного разложения различаются в первую очередь по тому, какая именно матрица раскладывается.

Одну из групп таких методов образуют методы, где раскладывается кросс-табуляционная матрица пользователей и предметов $R$ (та же самая, что используется для коллаборативной фильтрации).

Искать её разложение можно, в частности, в следующих видах:
* $R \approx WV^T$, где $W$ — матрица размера $\vert U \vert \times k$, $V$ — матрица размера $\vert I \vert \times k$, а $k$ — гиперпараметр (как правило, $k$ берут много меньше, чем и $\vert U \vert$, и $\vert I \vert$). Эта форма разложения интерпретируется так. В матрице $W$ находятся $k$ скрытых признаков пользователей, в матрице $V$ находятся $k$ скрытых признаков предметов, а чем больше скалярное произведение вектора скрытых признаков пользователя на вектор скрытых признаков предмета, тем выше для этого пользователя должен быть отранжирован этот предмет.
* $R \approx WV^T + b + c + m$, где $m$ — среднее значение известных ячеек матрицы $R$, $b$ — вектор-столбец размера $\vert U \vert \times 1$, $c$ — вектор-строка размера $1 \times \vert I \vert$, а операция сложения матрицы с вектором, одна из размерностей которого меньше, определена через бродкастинг. По сравнению с предыдущей формой интерпретация меняется так. Предполагается, что каждый пользователь дополнительно имеет свой собственный сдвиг, выраженный в $b$, а каждый предмет имеет свой собственный сдвиг, выраженный в $c$.

Чтобы найти $W$, $V$ и, если они есть, $b$ и $c$ ($m$ находится по своему определению), можно ввести какую-либо функцию потерь (чаще всего берут MSE или MSE с $L_2$-регуляризацией), а потом, считая все неизвестные величины оптимизируемыми параметрами, применить какой-нибудь численный метод:
* градиентный спуск (или его модификации);
* покоординатный спуск (в контексте выдачи рекомендаций часто называемый alternating least squares): значения $W$, $V$, $b$ и $c$ как-то инициализируются, а затем по очереди все эти переменные кроме одной рассматриваются как константы и ищется значение одной незафиксированной, минимизирующее функцию потерь, после чего её старое значение заменяется на найденное, очередь переходит к следующей переменной и так далее вплоть до выполнения некоторого критерия остановки.

Иногда находимое таким образом разложение матрицы $R$ называют её сингулярным разложением (SVD). Более того, некоторые конкретные спецификации обсуждаемого метода получили названия FunkSVD и SVD++. На самом деле, если понимать под SVD ровно то, что понимается под этим термином в линейной алгебре, то никакого отношения к SVD это не имеет. В частности, линейно-алгебраическое SVD не может работать, если в раскладываемой матрице есть пропущенные значения, а эти методы работают. Поэтому в контексте таких методов правильнее говорить просто о матричном разложении, а не о сингулярном разложении.

Тем не менее SVD тоже может использоваться для выдачи рекомендаций. Раскладывать можно и другие матрицы, если они доступны. Например, можно раскладывать матрицу транзакций, где строки кодируют транзакции, а столбцы — предметы, которые могут быть приобретены в соответствующей транзакции в нулевом или положительном количестве. В такой матрице уже нет пропущенных значений, а нули — это настоящие нули. При применении SVD в одной из получившихся матриц для каждого предмета будет найден вектор его скрытых признаков. При помощи таких векторов можно найти веса сходства между предметами, чтобы затем использовать уже именно их в коллаборативной фильтрации «предмет-предмет», применяемой не к пользователям, а к незавершённым транзакциям (например, в ситуациях, когда пользователь отложил товары в корзину, но ещё не оформил заказ). 

#### Нейросетевые методы

Если нейронная сеть обучается напрямую предсказывать какой-либо исход взаимодействия пользователя с предметом, то это частный случай подхода со сведением задачи выдачи рекомендаций к задаче обучения с учителем. Однако можно обучать нейронные сети и для других задач, таких что предсказания будут иметь отношение к выдаче рекомендаций, но при этом в отличие от разбиравшегося в одном из предыдущих разделов подхода объектами не будут ситуации взаимодействия пользователя с предметом. Более того, если в разбиравшемся ранее подходе нейронную сеть, в принципе, можно заменить любым другим алгоритмом обучения с учителем, в этом разделе речь пойдёт о подходах, опирающихся именно на свойства нейронных сетей.

Один из подходов предполагает обучение нейронной сети со следующей архитектурой:
* на вход подаётся вектор длины $\vert U \vert + \vert I \vert$, такой что он является конкатенацией one-hot-encoded вектора $i$-го пользователя и one-hot-encoded вектора $j$-го предмета;
* за входным слоем идёт слой вложений, где one-hot-encoded вектор пользователя преобразуется в непрерывный вектор и one-hot-encoded вектор предмета преобразуется в непрерывный вектор;
* далее идёт сколько-то полносвязных слоёв;
* выходом является одно число, интерпретирующееся как оценка $R_{ij}$.

Для обучения такой нейронной сети требуется только кросс-табуляционная матрица $R$, а логику, стоящую за выбранной архитектурой, можно описать так: для пользователей и предметов извлекаются их скрытые признаки, а по этим скрытым признакам многослойный перцептрон решает задачу предсказания $R_{ij}$.

Ещё один [подход](http://users.cecs.anu.edu.au/~u5098633/papers/www15.pdf) предполагает обучение очищающего от шума автокодировщика. Объектами в нём являются строки матрицы $R$, то есть то, что описывает пользователя. Шумом же считается замена каких-либо значений на пропуски (или на плэйсхолдер пропуска, если пропуски не поддерживаются конкретным программным инструментом). Получив на вход строку с пропусками, автокодировщик должен вернуть строку без пропусков.

Важно отметить, как проходит обучение и оценка качества рекомендательных автокодировщиков. Что тренировочным, что тестовым множеством являются все строки матрицы $R$. Разница же между ошибкой на обучении и ошибкой на валидации в том, по каким ячейкам они считаются. Ячейки, где стоят пропуски, игнорируются. Часть ячеек, где значения известны, отбирается для подсчёта функции потерь во время обучения. А уже по оставшимся заполненным ячейкам подсчитывается оценка качества.

#### Открытые практические вопросы

Иногда бывает важно, чтобы после рекомендаций происходило то, чего без них не произошло бы. Иными словами, рекомендации должны привносить некоторый инкремент/аплифт в целевые метрики. Строго говоря, вышеперечисленные методы не гарантируют, что этот инкремент появится и не произойдёт того, что пользователю будет порекомендовано то, что он и сам бы купил или и сам бы просмотрел. Как добиться от рекомендаций аплифта — во многом всё ещё не решённый вопрос.

## Вложение по подсловам fastText

Алгоритм fastText — способ отображать слова в векторные вложения, базирующийся на word2vec-SkipGram.

Основным отличием от word2vec является то, что теперь обучение происходит не на словах, а на $n$-граммах из символов. Среди гиперпараметров fastText есть два, определяющие, какие именно $n$-граммы рассматриваются: их минимальная длина $n_{\min}$ и их максимальная длина $n_{\max}$. В [оригинальной статье](https://arxiv.org/pdf/1607.04606.pdf) знаки препинания (в том числе пробелы) игнорируются, но зато в начало каждого слова добавляется специальный символ "<", а в конец каждого слова добавляется специальный символ ">". Также независимо от длины слово само оно, заключённое в угловые скобки, тоже считается рассматриваемой $n$-граммой.

При помощи word2vec-SkipGram, обученной по $n$-грамме предсказывать $n$-граммы, встречающиеся в её контексте, каждая $n$-грамма отображается в непрерывный вектор. Вложение же слова определяется как сумма векторных вложений $n$-грамм, целиком входящих в это слово.

По сравнению с word2vec у fastText есть следующие преимущества:
* слова, не встречавшиеся на этапе обучения, всё равно могут быть отображены в векторы;
* однокоренные слова и разные формы одного и того же слова будут отображаться в схожие векторы, даже если обучающих данных недостаточно, чтобы word2vec могла это вывести.

Что касается недостатков fastText, то к ним можно отнести скорость обучения. Дело в том, что при хоть сколько-нибудь большом $n_{\max}$ количество $n$-грамм будет существенно превышать количество слов в языке. Как следствие, и количество обучаемых параметров будет больше, чем у word2vec.

## Обзор возможных ошибок при проведении A/B-экспериментов

Ошибки, которые можно допустить при A/B-экспериментах, бывают следующими:
* Дефекты планирования:
  - Отсутствие заранее предрассчитанных сроков проведения и процента, вовлечённого в эксперимент. Ещё до запуска эксперимента необходимо выбрать статистическую значимость $\alpha$ (как правило, её полагают равной 0,95 или, что лучше, 0,99) и статистическую мощность, обозначаемую как $(1 - \beta)$ (ниже 0,8 её обычно не берут), а также определиться, двусторонняя или односторонняя альтернатива будет проверяться (если неизвестно, как повлияет экспериментальное изменение, нужно взять двустороннюю альтернативу; если есть основания считать, что эксперимент заведомо улучшающий или ухудшающий, то можно взять соответствующую одностороннюю альтернативу). Далее необходимо оценить дисперсию целевой метрики и предположительный размер эффекта, вносимого тестируемым изменением. Наконец, сроки проведения эксперимента и процент, отводимый под контрольную группу и тестовую группу, надлежит подобрать так, чтобы добиться выбранной мощности. Все эти расчёты нужны, потому что если продолжительность и охват выбрать наугад, то потом в случае отсутствия статистической значимости нельзя будет отличить ситуацию, когда эффекта нет или он сопоставим с шумом, от ситуации, когда эксперимент слишком рано завершили.
  - Смещение тестовой группы относительно контрольной по каким-либо параметрам. По самой концепции A/B-экспериментов такого смещения быть не должно.
  - Отличие контроля от того, что на текущий момент есть по умолчанию. Строго говоря, A/B-эксперимент проверяет не универсальное наличие эффекта от воздействия, а наличие эффекта от воздействия, применённого в конкретном состоянии. Значит, во время эксперимента контроль должен быть репрезентативен по отношению к состоянию по умолчанию. Пример, как это условие может быть нарушено, таков. Допустим, провели ухудшающий эксперимент, в котором половина пользователей испытала воздействие, эффект от которого продолжается и по окончании эксперимента. Если сразу же после этого эксперимента на той же самой выборке, но с независимым разбиением на группы проводить какой-то другой эксперимент, то в его контроле половина пользователей будет не такой, какой она обычно бывает.
  - Добавление дополнительных воздействий к тому, которое якобы исследуется. Например, если внедряется какая-либо рекомендательная модель, заполняющая блок на странице интернет-магазина товарами, которые могли бы быть интересны пользователю, то, чтобы протестировать эффект от собственно модели, необходимо в контроле заполнить блок случайными и/или наиболее популярными товарами, а не убирать весь блок. Если убрать весь блок, будет тестироваться совместный эффект от наличия на странице блока и его заполнения в соответствии с моделью.
  - Игнорирование эффекта новизны. Иногда оказывается так, что экспериментальное воздействие улучшает метрики не из-за своей сути, а просто из-за того, что появляется что-то новое. Если есть основания считать, что так может произойти, лучше не учитывать в обсчёте эксперимента сколько-то первых дней. Также, если изменение будет принято, через какое-то время можно провести обратный эксперимент, чтобы эффект новизны играл уже против этого изменения.
  - Несоответствие идентификаторов, по которым объекты разбиваются на группы, этим реальным объектам. Например, если идентификатор пользователя на сайте меняется в зависимости от того, авторизован он или нет, то при разбиении по такому идентификатору вполне возможно, что некоторые пользователи будут попадать то в контрольную группу, то в группу воздействия. Наличие пересечения между этими группами создаст ряд проблем. Например, наблюдения в контрольной группе и в тестовой группе уже не будут независимы. Кроме того нельзя будет оценить отложенный эффект от воздействия.
  - Использование для оказания воздействия способа, вносящего искажения. Под искажениями имеется в виду, что воздействие может получаться то сильнее, то слабее, чем запланировано, или что воздействие может быть оказано на не совсем те объекты. В предельном случае из-за какой-либо ошибки может оказаться так, что воздействие не будет оказано ни на один объект, но если этого не заметить, то будет сделан вывод, что воздействие неэффективно, хотя на самом деле произошёл сбой в способе его оказания.
* Нарушение процедуры проведения, зафиксированной ещё до начала эксперимента:
  - Прекращение эксперимента при первом же появлении статистически значимой разницы в интересующей метрике. Если не ждать заранее установленного конца эксперимента, а остановить его сразу же, как только $p$-значение опустится ниже уровня $(1 - \alpha)$, полученное $p$-значение окажется заниженным. Чтобы убедиться в этом, обозначим за 0 момент старта эксперимента, за $T$ — момент конца эксперимента, а за $p(t, \omega)$ — случайный процесс, задающий $p$-значение в момент времени $t$ при исходе случайности $\omega$. Очевидно, что множество $\{\omega: p(T, \omega) \le 1 - \alpha\}$ является подмножеством множества $\{\omega: \min_{t \in [0, T]} p(t, \omega) \le 1 - \alpha\}$. По определению, статистическая значимость относится к первому множеству, а то, что, будет получено при досрочной остановке, подменит его на второе.
  - Попытки продержать эксперимент дольше, чем планировалось, чтобы $p$-значения, близкие к критическим, могли бы опуститься ниже критического порога. Опять же, как и в предыдущем пункте это приводит к тому, что $p$-значение, измеренное в момент некорректного конца, является заниженным, то есть на самом деле вероятность пронаблюдать подобный исход в условиях отсутствия разницы выше, чем $1 - \alpha$. Если есть основания считать, что изначально необходимая продолжительность эксперимента была недооценена, лучше запустить новый эксперимент и держать его положенное время.
  - Извлечение гипотез из тех же данных, на которых они должны проверяться. Например, нельзя добавлять новые метрики после того, как по ним или по предыдущим метрикам поступили данные, чтобы задним числом найти такие, по которым эксперимент можно бы было принять. Аналогично, после запуска эксперимента нельзя искать срезы, в которых результат был бы желаемым.
*  Некорректное применение статистических тестов и неверная интерпретация их результатов:
  - Если тест показывает, что нет статистически значимой разницы между контрольной группой и группой воздействия, это не означает, что доказано, что воздействие не имеет эффекта — это означает только то, что не обнаружено свидетельств наличия эффекта.
  - Можно неоправданно завысить или занизить чувствительность статистического теста, если проявить непоследовательность в применении вспомогательных методов. Рассмотрим это на примере, где такими методами являются бутстрэп и стратификация групп. Допустим, объекты бывают двух классов и математические ожидания целевой метрики отличаются в зависимости от класса. Тогда для разбиения на контрольную группу и группу воздействия можно поровну и случайно разделить сначала объекты одного класса, а потом другого, добившись тем самым равного соотношения классов в группах. Если теперь из таких групп сэмплировать выборки для бутстрэпа просто случайно (т.е. игнорируя соотношение классов), то статистическая значимость окажется заниженной, ведь в дисперсию среднего по выборкам вклад внесёт и случайная неравномерность распределения классов, тогда как для средних по группам такого вклада нет.
  - Если смотреть сразу на $n$ метрик, то шансы, что хотя бы для одной из них за время эксперимента появится статистическая значимость на уровне $\alpha$, будут равны не $(1 - \alpha)$, а, грубо говоря, $(1 - \alpha^n)$. Поэтому, если метрик больше одной, нужно использовать специальные процедуры множественной проверки гипотез.
  - Бывает так, что тест Манна-Уитни говорит о статистически значимой разнице в пользу одной из двух групп, тогда как среднее значение метрики может быть лучше в другой группе. Это связано с тем, что среднее чувствительно к выбросам, тогда как тест Манна-Уитни учитывает только ранги. Если не смотреть на статистику теста Манна-Уитни, а смотреть только на разницу в средних и на $p$-значение, может быть сделан неправильный вывод о знаке эффекта, оказываемого экспериментальным воздействием.

## Основные понятия обучения с подкреплением

**Состояние (state), действие (action), награда (reward), агент (agent), среда (environment), наблюдение (observation), эпизод (episode)** — базовые понятия. В некоторый момент времени $t$ среда находится в состоянии $s_t$, а агент знает это состояние или по крайней мере получает наблюдение $o_t$, отражающее часть информации об $s_t$. На основании $s_t$ или $o_t$ агент совершает действие $a_t$, в результате чего момент времени становится равным $t+1$. В этот новый момент времени агент получает награду $r_{t+1} \in \mathbb{R}$, полностью или частично обусловленную действием $a_t$ (аналогично, в момент времени $t$ была награда $r_t$), а среда переходит в новое состояние $s_{t+1}$ и предоставляет наблюдение $o_{t+1}$. Может оказаться так, что в какой-то момент среда придёт в конечное состояние, из которого дальше нет ходов. Тогда говорят, что эпизод завершился.

**Модель среды (environment model)** — формальное описание того, как устроена среда. Модель среды может быть известна, а может быть и неизвестна. Как правило, известность модели среды считают эквивалентной тому, что можно узнать любой элемент трёхмерных таблиц $\mathcal{P}_{ss^{\prime}}^a$ и $\mathcal{R}_{ss^{\prime}}^a$. Первая из них содержит вероятность перейти в состояние $s^{\prime}$ при условии, что предыдущим состоянием было $s$ и было совершено действие $a$. Вторя содержит математические ожидания награды, которая выдаётся сразу же после того, как из состояния $s$ совершается действие $a$ и это приводит к приходу в состояние $s^{\prime}$.

**Стратегия (policy)** — формализация того, как агент выбирает действия. Начальная стратегия может быть взята произвольно; в дальнейшем целью является найти оптимальную стратегию. Обычно считают, что стратегия $\pi$ — это отображение, которое состоянию среды (или наблюдению, доступному агенту, если состояние среды скрыто) сопоставляет вероятностное распределение на множестве всех действий.

**Выигрыш (return, gain)** — то, математическое ожидание чего необходимо максимизировать. Пусть есть какая-то последовательность шагов взаимодействия агента со средой, то есть имеется последовательность действий $\{a_t\}$, совершённых агентом, и имеется последовательность наград $\{r_t\}$, полученных непосредственно после этих действий. Если среда предполагает конечные эпизоды, последовательность $\{r_t\}$ является конечной и состоит из всех наград, полученных от старта эпизода и вплоть до его заверешния. Если среда не предполагает наличие эпизодов, последовательность $\{r_t\}$ может быть бесконечной. Так вот, для некого момента времени $t$ определим ассоциированный с ним выигрыш как $G_t = \sum_{i=0}^{+\infty}\gamma^i r_{t+i+1}$, где $0 < \gamma \le 1$ — гиперпараметр, называемый коэффициентом дисконтирования, а в случае конечных последовательностей суммирование прекращается с их концом. 

**Функции ценности (value functions)** — промежуточные понятия, отличающие обучение с подкреплением в узком смысле от других методов решения задач, где есть среда, агент и награда. Функция ценности состояния определяется как $V_\pi(s) = \mathbb{E}_\pi [G_t \, \vert \, s_t = s]$, где обозначение $\mathbb{E}_\pi$ указывает на то, что действия, награды за которые формируют $G_t$, совершаются в соответствии со стратегией $\pi$. Функция ценности пары состояния и действия определяется как $Q_\pi(s, a) = \mathbb{E}_\pi [G_t \, \vert \, s_t = s, a_t = a]$. Зная $Q_\pi$, всегда можно найти $V_\pi$ по формуле $V_\pi(s) = \mathbb{P}_\pi[a \, \vert \, s] Q(s, a)$. Зная $V_\pi$, получить $Q_\pi$ можно, только если также известна модель среды: $Q_\pi(s, a) = \sum_{s^{\prime}} \mathcal{P}_{ss^{\prime}}^a (\mathcal{R}_{ss^{\prime}}^a + \gamma V_\pi(s^{\prime}))$. Хотя $Q_\pi$ и сложнее оценить из-за большей размерности области определения, в задачах, где недоступна модель среды, оценивать приходится её, а не $V_\pi$, потому что $V_\pi$ может не позволить улучшить стратегию.

**Задача предсказания (prediction problem)** — одна из двух задач, на которые разбивается задача обучения с подкреплением в узком смысле. В этой задаче есть возможность в соответствии с некоторой стратегией $\pi^{\prime}$ отыгрывать до конца эпизоды или, если среда не предполагает наличие эпизодов, просто совершать последовательности шагов. По собранным вышеописанным образом данным требуется оценить $V_\pi$ или $Q_\pi$. Если метод решения задачи предполагает, что $\pi = \pi^{\prime}$, то такой метод называется методом с интегральной оценкой ценности (on-policy method). Если же не предполагается, что стратегия, порождающая данные, совпадает со стратегией, для которой ищется функция ценности, то такой метод называется методом с разделённой оценкой ценности (off-policy method).

**Задача управления (control problem)** — вторая из двух задач, на которые разбивается задача обучения с подкреплением в узком смысле. В этой задаче есть возможность отыгрывать эпизоды или хотя бы совершать последовательности шагов, а на собранных таким образом данных решать задачу предсказания. Требуется же найти оптимальную стратегию. Примером подхода к решению является итерация по стратегии (policy iteration) — алгоритм, напоминающий EM-алгоритм. На вход подаётся некоторая детерминированная стратегия $\pi_0$. На каждом этапе работы алгоритма для текущей стратегии $\pi_i$ собираются данные и по ним оценивается $Q_{\pi_i}$, а затем текущая стратегия обновляется до $\pi_{i+1}$, являющейся жадной относительно $Q_{\pi_i}$. Иными словами, берётся $\pi_{i+1}(s) = \arg\max_a Q_{\pi_i}(s, a)$. Если предположить, что $Q_{\pi_i}$ оценена достаточно точно, будет применима теорема об улучшении стратегии и из неё будет следовать, что $\pi_{i+1}$ не хуже $\pi_i$, причём если они одинаковы, то обе они оптимальны.

## Методы численной оптимизации и обучение с подкреплением

Границы того, что относится к обучению с подкреплением, можно устанавливать по-разному. В относительно узкой трактовке, которая предлагается, в частности, в книге Саттона и Барто, в обучении с подкреплением обязательно должны присутствовать следующие составляющие:
* стратегия (policy), то есть то, что отображает наблюдение текущего состояния в действие (необязательно детерминированным образом);
* награда (reward), то есть то, что эпизоду (если они есть) или просто какой-то последовательности шагов (действий и получений обратной связи от среды) сопоставляет вещественное число (опять же, необязательно детерминированно);
* функция ценности (value function), то есть то, что отображает состояние в ожидаемую награду, которая будет получена, если прийти в это состояние и далее следовать имеющейся стратегии.

На самом деле, при поиске стратегии, максимизирующей ожидаемую награду, можно обойтись и без функции ценности. Допустим, стратегия ищется в каком-либо параметрическом семействе. Тогда оптимальные значения параметров можно найти любым численным методом, предназначенным для так называемой black-box optimization.

К задачам, решаемым при помощи обучения с подкреплением, успешно применяли метод [эволюционных стратегий](https://arxiv.org/pdf/1703.03864.pdf) и похожий на него метод [кросс-энтропии](http://web.mit.edu/6.454/www/www_fall_2003/gew/CEtutorial.pdf), а также различные генетические алгоритмы. Эти три метода можно отнести к методам, отталкивающимся от улучшения популяций. Помимо них можно применить и любые другие численные методы: например, метод имитации отжига (simulated annealing).

Есть ли смысл вводить функцию ценности или нет, зависит от того, насколько ценна промежуточная информация, и от того, насколько велико пространство стратегий (если не очень, то функция ценности рискует стать переусложнением).

## TD($\lambda$)-методы и следы приемлемости

#### Введение

TD($\lambda$)-методы — методы решения задачи предсказания, то есть задачи оценки функции ценности для заданной стратегии. Эти методы образуют параметрическое семейство, включающее в себя в том числе TD(1)-метод, являющийся обобщением метода Монте-Карло. Для краткости в этой заметке будет разбираться оценивание функции ценности состояния $V_\pi$. Для функции ценности пары состояния и действия $Q_\pi$ всё выглядит аналогично.

#### Прямой (концептуальный) подход

Если задача предсказания решается методом Монте-Карло, то сначала собирается набор отыгранных в соответствии со стратегией $\pi$ эпизодов, а затем по собранной статистике оценивается функция ценности $V_\pi$. Но на это можно посмотреть и по-другому. Предположим, что $V_\pi$ была инициализирована произвольно, а каждое посещение состояния $s$ (если речь идёт о методе Монте-Карло всех посещений) вносит инкрементальное обновление $V_\pi(s)$. Тогда, чтобы получился именно метод Монте-Карло, такое обновление должно иметь вид:
$$V_\pi(s) \leftarrow V_\pi(s) + c\left(\sum_{k=0}^{T - t - 1}\gamma^k r_{t+k+1} - V_\pi(s)\right),$$
где $c$ — константа, зависящая в том числе от количества эпизодов и количества посещений $s$, $t$ — момент времени внутри эпизода, когда произошло рассматриваемое посещение $s$, $T$ — длина этого эпизода, а $\{r_\tau : 1 \le \tau \le T\}$ — последовательность наград, которые были получены после каждого из шагов этого эпизода. Иными словами, посещение состояния $s$ в момент времени $t$ сдвигает оценку $V_\pi(s)$ в сторону следующей величины:
$$R_t = \sum_{k=0}^{T - t - 1}\gamma^k r_{t+k+1}.$$
Эта величина подсчитана исключительно по фактическим наградам, и, как следствие, она неизвестна до тех пор, пока эпизод не завершён.

Рассмотрим оценку величины $R_t$, которую можно подсчитать спустя $n$ шагов после момента времени $t$:
$$R_t^{(n)} = \sum_{k=0}^{n - 1}\gamma^k r_{t+k+1} + \gamma^n V_\pi(s_{t+n}).$$
Уточним также, что если $t+n > T$, то по определению $R_t^{(n)} = R_t$. Так определённая оценка $R_t^{(n)}$ отличается от $R_t$ тем, что приведённая сумма наград за поздние шаги заменяется значением функции ценности того состояния, из которого эти шаги были совершены.

Можно делать обновления не в сторону $R_t$, а в сторону $R_t^{(n)}$. Например, то, что называется TD-метод (он же TD(0)-метод, поскольку является частным случаем TD($\lambda$)-метода при $\lambda = 0$), делает обновления в сторону $R_t^{(1)}$.

Тут уместно обратить внимание на один нюанс. Вообще-то, классический метод Монте-Карло не предполагает инкрементальных обновлений: вместо них происходит усреднение уже после того, как отыграны все эпизоды (поэтому этот метод можно назвать оффлайновым). Как следствие, не возникает вопроса о том, когда обновлять $V_\pi$. Однако в определении $R_t^{(n)}$ участвует $V_\pi$, так что то, когда она обновляется, становится важным. Помимо оффлайнового варианта возможны и онлайновые — например, такой: обновлять сразу же, как только $R_t^{(n)}$ становится известной. Тем самым $V_\pi$ будет меняться в процессе отыгрывания эпизодов. Преимущество этого варианта в том, что в отличие от классического метода Монте-Карло его можно применять и к средам, где нет конечных эпизодов и последовательности шагов могут быть бесконечными.

Наконец, перейдём к концептуальному определению TD($\lambda$)-метода. Это метод, в котором есть гиперпараметр $0 \le \lambda \le 1$ и в котором обновления функции ценности происходят в сторону следующей величины:
$$(1 - \lambda)\sum_{n=1}^{+\infty} \lambda^{n-1} R_t^{(n)}.$$
Чем такое определение хорошо, так это тем, что оно даёт интерпретацию, в какую сторону происходят обновления: в сторону средневзвешенного между обновлениями TD-метода, метода Монте-Карло и промежуточных между этими двумя крайностями вариантов. Чем такое определение плохо, так это тем, что напрямую по нему нельзя создать алгоритм, способный совершать обновления после каждого шага, ведь большая часть слагаемых к тому времени будет неизвестна.

#### Обратный (механистический) подход

У TD($\lambda$)-метода есть и альтернативное определение, позволяющее обновлять $V_\pi$ на каждом шаге.

В виде псевдокода это определение записывается так:
* Инициализировать $V_\pi$ произвольно
* Для каждого состояния $s$ положить $e(s) = 0$
* Для каждого эпизода:
    - Инициализировать $s$ начальным состоянием
    - Пока эпизод не завершился:
        - Совершить действие $a$ в соответствии с заданной стратегией $\pi$
        - Получить вознаграждение $r$ и следующее состояние $s^{\prime}$
        - $\delta \leftarrow r + \gamma V_\pi(s^{\prime}) - V_\pi(s)$
        - Для всех состояний $z$:
            - $V_\pi(z) \leftarrow V_\pi(z) + \alpha e(z) \delta$, где $\alpha$ — гиперпараметр темпа обучения
            - $e(z) \leftarrow \gamma\lambda e(z)$
        - $e(s) \leftarrow e(s) + 1$
        - $s \leftarrow s^{\prime}$
        
Из этой онлайновой версии можно получить оффлайновую, на протяжении эпизода суммируя обновления для $V_\pi$, но применяя их только после конца эпизода.

Величина $e(s)$ называется следом приемлемости (eligibility trace) состояния $s$. Она, образно говоря, показывает, какая доля текущего обновления $\delta$ должна быть применена к состоянию $s$, чтобы это обновление было учтено с тем же весом, с каким оно учитывается в прямом подходе. Выкладкой можно показать, что прямой и обратный подход дают одинаковые результаты.

## Некоторые табличные методы обучения с подкреплением

#### Введение

Метод обучения с подкреплением называется табличным, если функция ценности для него ищется в виде таблицы, где в одном столбце содержится значение состояния (для функции ценности состояния $V_\pi$) или пары из состояния и действия (для функции ценности пары состояния и действия $Q_\pi$), а в другом столбце содержится соответствующее значение функции. Разумеется, табличные методы работают, только если количество возможных состояний или пар состояния и действия невелико. В случае, когда их количество велико, используют аппроксимационные методы. Впрочем, для многих табличных методов легко получить их аппроксимационные версии.

#### SARSA

Для решения задачи предсказания в методе SARSA функция $Q_\pi$ оценивается интегрально (on-policy) TD(0)-методом со следующей формулой обновления:
$$Q_\pi(s_t, a_t) \leftarrow Q_\pi(s_t, a_t) - \alpha(r_{t+1} + \gamma Q_\pi(s_{t+1}, a_{t+1}) - Q_\pi(s_t, a_t)),$$
где последовательности $\{s_t\}$, $\{a_t\}$ и $\{r_t\}$ были получены при следовании текущей стратегии $\pi$.

Способ решения задачи управления является частным случаем обобщённой итерации по стратегии. После каждого шага (действия и вызванного им перехода в новое состояние) происходит обновление ровно одного значения $Q$-таблицы, а текущая стратегия $\pi$ тоже обновляется после каждого шага.

В виде псевдокода алгоритм SARSA записывается так:
* Инициализировать $Q$-таблицу для всех пар $(s, a)$ произвольным образом с единственным условием, что если $s$ — конечное состояние, из которого больше нет ходов, то $Q(s, \cdot) = 0$
* Для каждого эпизода:
    - Инициализировать $s$ начальным состоянием
    - Положить стратегию $\pi$ равной некоторой стратегии, основанной на текущей $Q$-таблице (например, стратегии, $\varepsilon$-жадной относительно $Q$)
    - Выбрать (но пока не совершать) действие $a$ в соответствии со стратегией $\pi$, применённой к состоянию $s$
    - Пока эпизод не завершился:
        - Совершить действие $a$
        - Получить награду $r$ и новое состояние $s^{\prime}$
        - Выбрать (но пока не совершать) действие $a^{\prime}$ в соответствии со стратегией $\pi$, применённой к состоянию $s^{\prime}$
        - Обновить одну ячейку $Q$-таблицы: $Q(s, a) \leftarrow Q(s, a) + \alpha(r + \gamma Q(s^{\prime}, a^{\prime}) - Q(s, a))$
        - Положить стратегию $\pi$ равной некоторой стратегии, основанной на обновлённой $Q$-таблице (например, стратегии, $\varepsilon$-жадной относительно $Q$)
        - $a \leftarrow a^{\prime}$, $s \leftarrow s^{\prime}$
    
#### Q-обучение

Для решения задачи предсказания в методе Q-обучения $Q$-таблица оценивается раздельно (off-policy) TD(0)-методом со следующей формулой обновления:
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) - \alpha(r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)),$$
где последовательности $\{s_t\}$, $\{a_t\}$ и $\{r_t\}$ были получены при следовании некой стратегии $\pi$, не обязанной быть оптимальной относительно $Q$-таблицы.

Как и для SARSA способ решения задачи управления является частным случаем обобщённой итерации по стратегии. Обновление одного из значений $Q$-таблицы и обновление стратегии $\pi$ происходят после каждого шага. Однако в отличие от SARSA $Q$-таблица представляет собой оценку функции ценности оптимальной стратегии, а не оценку функции ценности текущей стратегии $\pi$.

В виде псевдокода алгоритм Q-обучения записывается так:
* Инициализировать $Q$-таблицу для всех пар $(s, a)$ произвольным образом с единственным условием, что если $s$ — конечное состояние, из которого больше нет ходов, то $Q(s, \cdot) = 0$
* Для каждого эпизода:
    - Инициализировать $s$ начальным состоянием
    - Пока эпизод не завершился:
        - Положить стратегию $\pi$ равной некоторой стратегии, основанной на текущей $Q$-таблице (например, стратегии, $\varepsilon$-жадной относительно $Q$)
        - Выбрать и совершить действие $a$ в соответствии со стратегией $\pi$, применённой к состоянию $s$
        - Получить награду $r$ и новое состояние $s^{\prime}$
        - Обновить одну ячейку $Q$-таблицы: $Q(s, a) \leftarrow Q(s, a) + \alpha(r + \gamma \max_{a^{\prime}} Q(s^{\prime}, a^{\prime}) - Q(s, a))$
        - $s \leftarrow s^{\prime}$

## DSSM (Deep Semantic Similarity Model)


#### Введение

Как и word2vec DSSM примечательна тем, что строит векторные вложения слов, хотя и обучается решать другую задачу. С этой точки зрения основным отличием DSSM от word2vec можно назвать то, что она обучается с учителем, а не в рамках концепции predictive learning (обучение через предсказание контекста). Объектами в постановке задачи для DSSM являются пары из поискового запроса и документа, где что запрос, что документ являются каким-либо текстом. Целевая переменная содержит информацию о том, релевантен ли документ запросу. В [оригинальной статье](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/cikm2013_DSSM_fullversion.pdf) предполагалось, что целевая переменная бинарная, причём к каждому запросу брался один документ, считавшийся релевантным, и также сэмплировалось сколько-то документов для отрицательных примеров.

#### Формат входных данных

Из соображений минимизации количества весов нейронной сети в качестве представления текстов (неважно, запрос это или документ) выбран мешок триграмм (переход к нему авторы статьи о DSSM назвали word hashing). Если число слов в языке измеряется сотнями тысяч, то триграмм из 30 символов меньше 30 тысяч, так что размерность входа, действительно, сокращается на порядок.

#### Архитектура нейронной сети

Архитектура у DSSM следующая:
- Есть несколько полносвязных слоёв, размер входа первого из которых равен количеству триграмм, а размер выхода последнего из которых равен размерности искомого векторного вложения.
- Поскольку объектами являются пары из запроса и документа, есть две копии вышеописанных полносвязных слоёв, через одну из которых прогоняется запрос, а через другую — документ. В оригинальной статье веса в этих копиях были общими (т.е. одними и теми же).
- Если обозначить за $q$ запрос, за $d$ документ, за $y_q$ получившееся вложение запроса, а за $y_d$ получившееся вложение документа, то далее эти вложения как входы подаются юниту, возвращающему вещественное число, равное косинусу угла между ними: $\frac{y^T_q y_d}{\|y_q\| \|y_d\|}$. Обозначим его за $R(q, d)$ и будем считать, что оно оценивает степень соответствия документа запросу.

#### Обучение

Функция потерь для DSSM вычисляется не для одного объекта, а, как и в задачах ранжирования, для группы объектов. В данном случае группу образуют положительный пример и подобранные к нему отрицательные примеры. Если обозначить за $(q, d^+)$ положительный пример, а за $D$ — множество из $d^+$ и документов из соответствующих отрицательных примеров, то на группе с примером $(q, d^+)$ значение функции потерь равно:
$$\log P(d^+ \, \vert \, q) = \log \frac{e^{\gamma R(q, d^+)}}{\sum_{d \in D} e^{\gamma R(q, d)}},$$
то есть $P(d^+ \, \vert \, q)$ — предсказанная вероятность того, что для запроса $q$ документом из положительного примера является $d^+$ — полагается равной результату применения операции softmax с температурой $\gamma$ (это гиперпараметр) к предсказанным мерам релевантности $R(q, d)$.

Поскольку такая функция потерь является дифференцируемой относительно $R(q, d)$, а $R(q, d)$ дифференцируема относительно весов нейронной сети, для обучения можно использовать любые градиентные методы оптимизации.

## Способы получения вложений объектов по вложениям связанных объектов

Допустим, есть объекты двух видов, между которыми существуют взаимосвязи: например, есть тексты и есть пользователи, причём про пользователей известно, какие тексты они читали, а какие нет. Также допустим, что для одного из видов объектов уже есть готовые векторные представления: например, вложения для текстов были получены через word2vec. Предположим, что требуется получить вложения для объектов второго вида на базе вложений объектов первого вида.

Сделать это можно следующими способами:
* Проведя ALS-шаг (один шаг попеременного метода наименьших квадратов). По сути, этот шаг является решением $n$ независимых задач линейной регрессии, где $n$ — количество объектов второго вида. В $i$-й задаче выборка основывается на том, с какими объектами первого вида было взаимодействие $i$-го объекта второго вида: скажем, можно взять все тексты, которые читал пользователь, и просэмплировать сколько-то текстов, которые пользователь не читал. Целевой переменной является результат взаимодействия (например, 1, если текст был прочтён, и 0, если текст попал в выборку в результате отрицательного сэмлирования). Признаками являются векторные вложения выбранных объектов первого вида. Вектор коэффициентов, найденных при решении получившейся задачи линейной регрессии, и станет вложением $i$-го объекта.
* Проагрегировав векторные представления тех объектов первого вида, с которыми взаимодействовал объект второго вида. При этом можно использовать не только сумму, среднее, максимум или минимум: например, если известно время взаимодействия, то можно рассчитать взвешенную сумму с весами, затухающими в зависимости от прошедшего с момента взаимодействия времени.